In [ ]:
# ============================
# Cell 1 — Imports + Custom Env (Continuous CartPole Swing-Up) + EDGE RESPAWN
# + TensorFlow GPU setup (for GPflow/TF compute)
# ============================

import warnings
warnings.filterwarnings("ignore", message="pkg_resources is deprecated as an API.*", category=UserWarning)

import os
import math
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from gymnasium.wrappers import TimeLimit
from gymnasium.utils import seeding

# ---- numpy bool8 compat ----
if not hasattr(np, "bool8"):
    np.bool8 = np.bool_

# ============================================================
# TensorFlow GPU setup (THIS is what makes TF/GPflow use GPU)
# ============================================================
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # quieter TF logs (optional)

import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    # Prevent TF from grabbing all VRAM at startup
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("✅ TF version:", tf.__version__)
    print("✅ Built with CUDA:", tf.test.is_built_with_cuda())
    print("✅ GPUs:", gpus)
else:
    print("⚠️ TF version:", tf.__version__)
    print("⚠️ Built with CUDA:", tf.test.is_built_with_cuda())
    print("⚠️ GPUs: [] (TensorFlow will run on CPU)")

# ============================================================
# Dtypes for GPU-friendly pipeline
# ============================================================
# Env can stay float32; GP features should also be float32 to avoid slow casts.
DTYPE_NP = np.float32

# ============================================================
# Angle helpers
# ============================================================
def wrap_pi(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def obs_to_state(obs):
    """
    obs = [x, x_dot, theta, theta_dot]
    Wrap theta to (-pi, pi] for stability.
    """
    x, xdot, th, thdot = float(obs[0]), float(obs[1]), float(obs[2]), float(obs[3])
    th = wrap_pi(th)
    return x, xdot, th, thdot
U_MIN, U_MAX = -1.0, 1.0

def state_to_features(x, xdot, theta, thetadot, u,
                      x_scale=2.4, v_scale=3.0, w_scale=8.0,
                      dtype=DTYPE_NP):
    """
    GP features (D=6), bounded:
        [ tanh(x/x_scale),
          tanh(xdot/v_scale),
          sin(theta),
          cos(theta),
          tanh(thetadot/w_scale),
          u ]
    Returns float32 by default (GPU-friendly).
    """
    x_feat = np.tanh(x / x_scale)
    xdot_feat = np.tanh(xdot / v_scale)
    w_feat = np.tanh(thetadot / w_scale)
    return np.array(
        [x_feat, xdot_feat, np.sin(theta), np.cos(theta), w_feat, float(u)],
        dtype=dtype
    )

# ============================================================
# Custom Continuous CartPole Swing-Up Env (CPU physics; that's OK)
# ============================================================
class ContinuousCartPoleSwingUpEnv(gym.Env):
    metadata = {"render_modes": ["rgb_array", "human"], "render_fps": 50}

    def __init__(self, render_mode=None, start_down=True):
        super().__init__()

        # physics
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = self.masspole + self.masscart
        self.length = 0.5
        self.polemass_length = self.masspole * self.length

        # control
        self.force_mag = 30.0
        self.tau = 0.02
        self.min_action = -1.0
        self.max_action = 1.0

        # track limits
        self.x_threshold = 2.4

        # reset mode
        self.start_down = bool(start_down)

        # render
        self.render_mode = render_mode
        self.state = None
        self.np_random = None
        self.seed()

        # spaces
        high = np.array(
            [
                self.x_threshold * 2,
                np.finfo(np.float32).max,
                np.finfo(np.float32).max,
                np.finfo(np.float32).max,
            ],
            dtype=np.float32,
        )

        self.action_space = spaces.Box(
            low=np.array([self.min_action], dtype=np.float32),
            high=np.array([self.max_action], dtype=np.float32),
            shape=(1,),
            dtype=np.float32,
        )
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def stepPhysics(self, force):
        x, x_dot, theta, theta_dot = self.state
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta * costheta / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * xacc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * thetaacc
        return (x, x_dot, theta, theta_dot)

    def step(self, action):
        action = np.asarray(action, dtype=np.float32).reshape(1,)
        assert self.action_space.contains(action), f"{action} invalid"

        u = float(action[0])
        force = self.force_mag * u

        self.state = self.stepPhysics(force)
        x, x_dot, theta, theta_dot = self.state

        terminated = bool(x < -self.x_threshold or x > self.x_threshold)
        truncated = False  # TimeLimit handles truncation

        reward = (
            +1.0 * math.cos(theta)
            -0.01 * (x * x)
            -0.001 * (x_dot * x_dot)
            -0.001 * (theta_dot * theta_dot)
            -0.001 * (u * u)
        )

        obs = np.array([x, x_dot, theta, theta_dot], dtype=np.float32)
        info = dict(x=x, x_dot=x_dot, theta=theta, theta_dot=theta_dot, u=u)
        return obs, float(reward), terminated, truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.seed(seed)

        x = float(self.np_random.uniform(low=-0.05, high=0.05))
        x_dot = float(self.np_random.uniform(low=-0.05, high=0.05))
        theta_dot = float(self.np_random.uniform(low=-0.05, high=0.05))

        if self.start_down:
            theta = float(math.pi + self.np_random.uniform(low=-0.10, high=0.10))
        else:
            theta = float(self.np_random.uniform(low=-0.10, high=0.10))

        self.state = (x, x_dot, theta, theta_dot)
        obs = np.array(self.state, dtype=np.float32)
        info = {}
        return obs, info

    def render(self):
        return None

    def close(self):
        return None

# ============================================================
# Edge respawn wrapper
# ============================================================
class EdgeRespawnWrapper(gym.Wrapper):
    def __init__(self, env, respawn_penalty=-2.0, reset_seed_mode="random", seed=0):
        super().__init__(env)
        self.respawn_penalty = float(respawn_penalty)
        self.reset_seed_mode = str(reset_seed_mode)
        self._rng = np.random.default_rng(seed)

    def reset(self, *, seed=None, options=None):
        if seed is not None:
            self._rng = np.random.default_rng(seed)
        return self.env.reset(seed=seed, options=options)

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)

        if terminated:
            info = dict(info)
            info["respawned"] = True
            reward = float(reward) + self.respawn_penalty

            seed = int(self._rng.integers(0, 10**9)) if self.reset_seed_mode == "random" else None
            obs, _ = self.env.reset(seed=seed)

            terminated = False
            truncated = False

        return obs, float(reward), bool(terminated), bool(truncated), info

# ============================================================
# Environment factory
# ============================================================
MAX_EPISODE_STEPS = 600

def make_env(
    render_mode=None,
    seed=0,
    max_episode_steps=MAX_EPISODE_STEPS,
    start_down=True,
    edge_respawn=True,
    respawn_penalty=-2.0,
):
    env = ContinuousCartPoleSwingUpEnv(render_mode=render_mode, start_down=start_down)
    env = TimeLimit(env, max_episode_steps=max_episode_steps)
    if edge_respawn:
        env = EdgeRespawnWrapper(env, respawn_penalty=respawn_penalty, seed=seed)
    env.reset(seed=seed)
    return env

# ============================================================
# Sanity check
# ============================================================
env = make_env(render_mode=None, seed=0, start_down=True, edge_respawn=True)
obs, _ = env.reset(seed=0)
s = obs_to_state(obs)

print("✅ Env ready (edge_respawn=True)")
print("obs:", obs)
print("state:", s)
print("action space:", env.action_space)
env.close()


In [ ]:

# ============================
# Cell 2 — Render the RANDOM collection path (and collect X,Y)  ✅ CartPole version
#
# Fix:
#   ✅ Adds render_cartpole_frame_from_state() (pure PIL) so the line:
#        frame = render_cartpole_frame_from_state(...)
#      actually works.
#
# What you get:
#   - Runs random actions for n_steps (with resets as needed)
#   - Collects executed transitions:
#       X0: (N,6)  = [x_feat, xdot_feat, sinθ, cosθ, w_feat, u]
#       Ydx0, Ydxdot0, Ydth0, Ydthdot0  (each (N,1))
#   - Records frames (rgb) and displays an inline animation (JS HTML)
#   - Plots trajectories:
#       x(t), xdot(t), theta(t), thetadot(t), action(t)
#       phase plot: x vs xdot
#
# IMPORTANT:
#   - We DO NOT rely on env.render() / pygame.
#   - We render from state using a self-contained PIL renderer below.
#   - If your env does edge respawn and sets info["respawned"]=True, we DROP that transition.
# ============================

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display

from PIL import Image, ImageDraw

# ------------------------------------------------------------
# Pure-PIL renderer: (x, theta) -> RGB frame
# ------------------------------------------------------------
def render_cartpole_frame_from_state(
    x, theta,
    x_threshold=2.4,
    W=720, H=450,
    cart_width=70,
    cart_height=35,
    pole_length_px=160,
    pole_width=10,
):
    """
    Minimal CartPole render (rgb uint8) from state.
    Coordinates:
      - x in [-x_threshold, x_threshold] maps to screen track
      - theta: 0 = upright, pi = down (standard)
    """
    # background
    img = Image.new("RGB", (W, H), (245, 245, 245))
    dr = ImageDraw.Draw(img)

    # track line
    y_track = int(0.72 * H)
    dr.line([(0, y_track), (W, y_track)], fill=(210, 210, 210), width=4)

    # map x -> screen px
    # keep margins so cart stays visible
    margin = 40
    x_clamped = float(np.clip(x, -x_threshold, x_threshold))
    x_norm = (x_clamped + x_threshold) / (2 * x_threshold)  # [0,1]
    cx = int(margin + x_norm * (W - 2 * margin))
    cy = y_track - cart_height // 2

    # cart rectangle
    x0 = cx - cart_width // 2
    y0 = cy - cart_height // 2
    x1 = cx + cart_width // 2
    y1 = cy + cart_height // 2
    dr.rounded_rectangle([x0, y0, x1, y1], radius=8, fill=(60, 90, 160), outline=(30, 30, 30), width=2)

    # axle point on top of cart
    ax = cx
    ay = y0 + 6

    # pole end point
    # IMPORTANT: theta=0 is upright -> pole points UP, so use -cos/sin appropriately
    # Standard cartpole uses theta from vertical; here we draw:
    #   dx =  L * sin(theta)
    #   dy = -L * cos(theta)
    dx = pole_length_px * float(np.sin(theta))
    dy = -pole_length_px * float(np.cos(theta))
    px = ax + dx
    py = ay + dy

    # pole (as thick line)
    dr.line([(ax, ay), (px, py)], fill=(180, 50, 50), width=pole_width)

    # axle circle
    r = 8
    dr.ellipse([ax - r, ay - r, ax + r, ay + r], fill=(30, 30, 30))

    return np.asarray(img, dtype=np.uint8)


# ------------------------------------------------------------
# Random collection (rendered)
# ------------------------------------------------------------
def collect_random_transitions_rendered_cartpole(
    n_steps=500,
    seed=0,
    max_episode_steps=500,
    start_down=True,
    # rendering controls
    frame_stride=2,
    resize=(720, 450),
    fps=20,
    # edge-respawn handling (relies on your Cell 1 env.step() putting info["respawned"]=True)
    edge_respawn=True,
    respawn_penalty=-2.0,
    drop_respawn_transitions=True,
    verbose=True,
):
    rng = np.random.default_rng(seed)

    env_vis = make_env(
        render_mode=None,   # we render ourselves
        seed=seed,
        max_episode_steps=max_episode_steps,
        start_down=start_down,
        edge_respawn=edge_respawn,
        respawn_penalty=respawn_penalty,
    )

    obs, info = env_vis.reset(seed=seed)
    x, xdot, th, thdot = obs_to_state(obs)

    X_list, Ydx_list, Ydxdot_list, Ydth_list, Ydthdot_list = [], [], [], [], []

    traj_x, traj_xdot, traj_th, traj_thdot, traj_u = [], [], [], [], []
    frames = []
    dropped = 0
    resets = 0

    for t in range(n_steps):
        u = float(rng.uniform(U_MIN, U_MAX))

        obs2, reward, terminated, truncated, info = env_vis.step(np.array([u], dtype=np.float32))
        x2, xdot2, th2, thdot2 = obs_to_state(obs2)

        respawned = bool(isinstance(info, dict) and info.get("respawned", False))
        if respawned and drop_respawn_transitions:
            dropped += 1
        else:
            X_list.append(state_to_features(x, xdot, th, thdot, u))
            Ydx_list.append(x2 - x)
            Ydxdot_list.append(xdot2 - xdot)
            Ydth_list.append(wrap_pi(th2 - th))
            Ydthdot_list.append(thdot2 - thdot)

        traj_x.append(x)
        traj_xdot.append(xdot)
        traj_th.append(th)
        traj_thdot.append(thdot)
        traj_u.append(u)

        if (t % frame_stride) == 0:
            W, H = int(resize[0]), int(resize[1])
            frame = render_cartpole_frame_from_state(x2, th2, x_threshold=2.4, W=W, H=H)
            frames.append(frame)

        x, xdot, th, thdot = x2, xdot2, th2, thdot2

        if terminated or truncated:
            resets += 1
            obs, info = env_vis.reset(seed=int(seed + 123 + t))
            x, xdot, th, thdot = obs_to_state(obs)
            if verbose:
                print(f"[t={t:04d}] reset (terminated={terminated}, truncated={truncated})")

    env_vis.close()

    X0 = np.asarray(X_list, dtype=np.float64)
    Ydx0 = np.asarray(Ydx_list, dtype=np.float64).reshape(-1, 1)
    Ydxdot0 = np.asarray(Ydxdot_list, dtype=np.float64).reshape(-1, 1)
    Ydth0 = np.asarray(Ydth_list, dtype=np.float64).reshape(-1, 1)
    Ydthdot0 = np.asarray(Ydthdot_list, dtype=np.float64).reshape(-1, 1)

    traj = dict(
        x=np.asarray(traj_x, dtype=np.float64),
        xdot=np.asarray(traj_xdot, dtype=np.float64),
        theta=np.asarray(traj_th, dtype=np.float64),
        thetadot=np.asarray(traj_thdot, dtype=np.float64),
        u=np.asarray(traj_u, dtype=np.float64),
        dropped_respawn=int(dropped),
        resets=int(resets),
        steps=int(n_steps),
        kept=int(X0.shape[0]),
    )

    # ----------------------------
    # 1) show animation
    # ----------------------------
    if len(frames) > 0:
        fig = plt.figure(figsize=(resize[0] / 100, resize[1] / 100), dpi=100)
        plt.axis("off")
        im = plt.imshow(frames[0])

        def animate_fn(i):
            im.set_data(frames[i])
            return [im]

        ani = animation.FuncAnimation(
            fig,
            animate_fn,
            frames=len(frames),
            interval=1000 / float(fps),
            blit=True
        )
        plt.close(fig)
        display(HTML(ani.to_jshtml()))
    else:
        print("⚠️ No frames collected (check frame_stride / resize).")

    # ----------------------------
    # 2) trajectory plots
    # ----------------------------
    tgrid = np.arange(len(traj["x"]))

    plt.figure(figsize=(9, 3.8))
    plt.plot(tgrid, traj["x"], linewidth=2)
    plt.xlabel("t"); plt.ylabel("x")
    plt.title("Random collection: cart position x(t)")
    plt.grid(True, alpha=0.25); plt.show()

    plt.figure(figsize=(9, 3.8))
    plt.plot(tgrid, traj["xdot"], linewidth=2)
    plt.xlabel("t"); plt.ylabel("xdot")
    plt.title("Random collection: cart velocity xdot(t)")
    plt.grid(True, alpha=0.25); plt.show()

    plt.figure(figsize=(9, 3.8))
    plt.plot(tgrid, traj["theta"], linewidth=2)
    plt.xlabel("t"); plt.ylabel("theta (rad)")
    plt.title("Random collection: pole angle theta(t) (wrapped)")
    plt.grid(True, alpha=0.25); plt.show()

    plt.figure(figsize=(9, 3.8))
    plt.plot(tgrid, traj["thetadot"], linewidth=2)
    plt.xlabel("t"); plt.ylabel("thetadot (rad/s)")
    plt.title("Random collection: pole angular velocity thetadot(t)")
    plt.grid(True, alpha=0.25); plt.show()

    plt.figure(figsize=(9, 3.4))
    plt.plot(tgrid, traj["u"], linewidth=2)
    plt.xlabel("t"); plt.ylabel("u")
    plt.title("Random actions u(t)")
    plt.grid(True, alpha=0.25); plt.show()

    plt.figure(figsize=(6.8, 5.2))
    plt.scatter(traj["x"], traj["xdot"], s=10, alpha=0.55)
    plt.xlabel("x"); plt.ylabel("xdot")
    plt.title("Random collection path in state space (x vs xdot)")
    plt.grid(True, alpha=0.25); plt.show()

    print("Collected X0 shape:", X0.shape, " (6D features)")
    print("Targets shapes:", Ydx0.shape, Ydxdot0.shape, Ydth0.shape, Ydthdot0.shape)
    print(f"Kept={traj['kept']}  Dropped(respawn)={traj['dropped_respawn']}  Resets={traj['resets']}")

    return X0, Ydx0, Ydxdot0, Ydth0, Ydthdot0, frames, traj


# ---- run it ----
SEED = 0
X0, Ydx0, Ydxdot0, Ydth0, Ydthdot0, frames0, traj0 = collect_random_transitions_rendered_cartpole(
    n_steps=500,
    seed=SEED,
    max_episode_steps=500,
    start_down=True,
    frame_stride=2,
    resize=(720, 450),
    fps=20,
    edge_respawn=True,
    respawn_penalty=-2.0,
    drop_respawn_transitions=True,
    verbose=False,
)

# Optional: quick distribution sanity
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Histogram(x=Ydth0.flatten(), nbinsx=60, name="Δtheta"))
fig.add_trace(go.Histogram(x=Ydthdot0.flatten(), nbinsx=60, name="Δthetadot"))
fig.update_layout(
    title="Initial random dataset: distribution of Δtheta and Δtheta_dot",
    barmode="overlay",
    xaxis_title="delta value",
    yaxis_title="count",
)
fig.update_traces(opacity=0.55)
fig.show()


In [ ]:
# ===========================
# Cell 3 — OSGPR-VFE core (Streaming Sparse GP) + training + summaries + anchors
#
# Matches OUR pipeline:
#   - GLOBAL sparse GP lives on an inducing set Z (usually Z_GLOBAL, size M, capped)
#   - Each online update:
#       (a) extract old posterior summary at Z_old
#       (b) build a NEW OSGPR_VFE model on (X_new, Y_new) + old summary
#       (c) train a bit, then cache for fast prediction
#   - Anchors are selected FROM THE CURRENT inducing set Z (so you can reselect after updates)
#
# Provides:
#   - batch_state_to_features(): (B,4)+(B,) -> (B,6)
#   - OSGPR_VFE (single-output)
#   - train_osgpr()
#   - prior_summary(), extract_summary_from_model()
#   - greedy_dopt_anchors_from_K()
#   - rebuild_osgpr_from_old_summary(): returns (model_new, train_time, neg_obj)  ✅ fixes your unpack bug
# ===========================

import time
import copy
import numpy as np
import tensorflow as tf
import gpflow

from gpflow.inducing_variables import InducingPoints
from gpflow.models import GPModel, InternalDataTrainingLossMixin
from gpflow import covariances

# ---- numerics ----
gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_jitter(1e-6)
tf.keras.backend.set_floatx("float64")

print("TF built with CUDA:", tf.test.is_built_with_cuda())
try:
    print("GPUs visible:", tf.config.list_physical_devices("GPU"))
except Exception as e:
    print("GPU query failed:", e)

DTYPE = gpflow.default_float()

# ---------------------------
# helpers
# ---------------------------
def sym_jitter(A, jitter=1e-6):
    """Make symmetric + add jitter (numpy)."""
    A = np.asarray(A, dtype=np.float64)
    A = 0.5 * (A + A.T)
    A = A + float(jitter) * np.eye(A.shape[0], dtype=np.float64)
    return A

def finite_mask(*arrs):
    """Row-wise finite mask across arrays."""
    m = None
    for a in arrs:
        a = np.asarray(a)
        mm = np.isfinite(a).all(axis=1) if a.ndim == 2 else np.isfinite(a)
        m = mm if m is None else (m & mm)
    return m

def clone_kernel(kernel):
    """
    Clone a GPflow kernel (to avoid variable-sharing across models).
    gpflow.utilities.deepcopy exists in many versions; fallback to copy.deepcopy.
    """
    try:
        from gpflow.utilities import deepcopy as gf_deepcopy
        return gf_deepcopy(kernel)
    except Exception:
        return copy.deepcopy(kernel)

# ------------------------------------------------------------
# Batch feature map (FAST) — used by MPPI later
# ------------------------------------------------------------
def batch_state_to_features(S, U, x_scale=2.4, v_scale=3.0, w_scale=8.0):
    """
    Vectorized mapping from physical CartPole state to 6D GP features.

    S: (B,4)  [x, xdot, theta, thetadot]
    U: (B,)   action in [-1,1]
    Returns:
      Xfeat: (B,6) [tanh(x/xs), tanh(xdot/vs), sin(theta), cos(theta), tanh(thetadot/ws), u]
    """
    S = np.asarray(S, dtype=np.float64)
    U = np.asarray(U, dtype=np.float64).reshape(-1)
    assert S.ndim == 2 and S.shape[1] == 4, "S must be (B,4)"
    assert U.shape[0] == S.shape[0], "U must match batch size"

    x     = S[:, 0]
    xdot  = S[:, 1]
    th    = S[:, 2]
    thdot = S[:, 3]

    Xf = np.empty((S.shape[0], 6), dtype=np.float64)
    Xf[:, 0] = np.tanh(x / x_scale)
    Xf[:, 1] = np.tanh(xdot / v_scale)
    Xf[:, 2] = np.sin(th)
    Xf[:, 3] = np.cos(th)
    Xf[:, 4] = np.tanh(thdot / w_scale)
    Xf[:, 5] = U
    return Xf

# ============================================================
# OSGPR-VFE model — regression-only, single-output
# ============================================================
class OSGPR_VFE(GPModel, InternalDataTrainingLossMixin):
    """
    Online Sparse Variational GP Regression (VFE), SINGLE-OUTPUT.

    Provide:
      - current batch data (X, Y)
      - old summary q_old(u)=N(mu_old, Su_old) at Z_old
      - Kaa_old = K(Z_old,Z_old) from old step
      - new inducing Z (usually Z_GLOBAL; you MAY refresh Z over time, but size should be capped)

    Includes:
      - predict_f (correct but slower)
      - build_predict_cache + predict_f_cached (FAST diag predictions)
    """
    def __init__(self, data, kernel, mu_old, Su_old, Kaa_old, Z_old, Z, mean_function=None):
        X, Y = gpflow.models.util.data_input_to_tensor(data)
        self.X, self.Y = X, Y

        likelihood = gpflow.likelihoods.Gaussian()
        num_latent_gps = GPModel.calc_num_latent_gps_from_data(data, kernel, likelihood)
        super().__init__(kernel, likelihood, mean_function, num_latent_gps)

        Z = np.asarray(Z, dtype=np.float64)
        assert Z.ndim == 2, "Z must be (M, D)"
        self.inducing_variable = InducingPoints(Z)
        gpflow.set_trainable(self.inducing_variable, False)
        mu_old  = np.asarray(mu_old, dtype=np.float64).reshape(-1, 1)
        Su_old  = sym_jitter(Su_old, 1e-6)
        Kaa_old = sym_jitter(Kaa_old, 1e-6)
        Z_old   = np.asarray(Z_old, dtype=np.float64)

        self.mu_old  = tf.Variable(mu_old,  trainable=False, dtype=DTYPE)
        self.Su_old  = tf.Variable(Su_old,  trainable=False, dtype=DTYPE)
        self.Kaa_old = tf.Variable(Kaa_old, trainable=False, dtype=DTYPE)
        self.Z_old   = tf.Variable(Z_old,   trainable=False, dtype=DTYPE)

        if self.mean_function is None:
            self.mean_function = gpflow.mean_functions.Zero()

        # cache for fast predict
        self._cache_ready = False
        self._cache_Lb = None
        self._cache_LD = None
        self._cache_rhs = None

    def _common_terms(self):
        """
        Build common matrices used by both ELBO and prediction.

        Z   : new inducing (Mb)
        Za  : old inducing (Ma) == self.Z_old
        X   : current batch inputs

        Kbf = K(Z, X)    [Mb, N]
        Kbb = K(Z, Z)    [Mb, Mb]
        Kba = K(Z, Za)   [Mb, Ma]
        """
        jitter = gpflow.utilities.to_default_float(1e-6)
        sigma2 = self.likelihood.variance

        Saa = self.Su_old  # [Ma,Ma]
        ma  = self.mu_old  # [Ma,1]

        Kbf = covariances.Kuf(self.inducing_variable, self.kernel, self.X)           # [Mb, N]
        Kbb = covariances.Kuu(self.inducing_variable, self.kernel, jitter=jitter)   # [Mb, Mb]
        Kba = covariances.Kuf(self.inducing_variable, self.kernel, self.Z_old)      # [Mb, Ma]

        Kaa_cur = gpflow.utilities.add_noise_cov(self.kernel(self.Z_old), jitter)   # [Ma,Ma]
        Kaa     = gpflow.utilities.add_noise_cov(self.Kaa_old, jitter)              # [Ma,Ma]

        err = self.Y - self.mean_function(self.X)  # [N,1]

        # c = Kbf*(Y/sigma2) + Kba*(Saa^{-1} ma)
        Sainv_ma = tf.linalg.solve(Saa, ma)                                # [Ma,1]
        c = tf.matmul(Kbf, self.Y / sigma2) + tf.matmul(Kba, Sainv_ma)     # [Mb,1]

        # Cholesky(Kbb)
        Lb = tf.linalg.cholesky(Kbb)                                       # [Mb,Mb]
        Lbinv_c   = tf.linalg.triangular_solve(Lb, c,   lower=True)        # [Mb,1]
        Lbinv_Kba = tf.linalg.triangular_solve(Lb, Kba, lower=True)        # [Mb,Ma]
        Lbinv_Kbf = tf.linalg.triangular_solve(Lb, Kbf, lower=True) / tf.sqrt(sigma2)  # [Mb,N]

        d1 = tf.matmul(Lbinv_Kbf, Lbinv_Kbf, transpose_b=True)             # [Mb,Mb]

        # T = (Lb^{-1}Kba)^T  => [Ma,Mb]
        T = tf.linalg.matrix_transpose(Lbinv_Kba)

        # d2
        LSa = tf.linalg.cholesky(Saa)
        LSainv_T = tf.linalg.triangular_solve(LSa, T, lower=True)
        d2 = tf.matmul(LSainv_T, LSainv_T, transpose_a=True)               # [Mb,Mb]

        # d3
        La = tf.linalg.cholesky(Kaa)
        Lainv_T = tf.linalg.triangular_solve(La, T, lower=True)
        d3 = tf.matmul(Lainv_T, Lainv_T, transpose_a=True)                 # [Mb,Mb]

        Mb = self.inducing_variable.num_inducing
        D = tf.eye(Mb, dtype=DTYPE) + d1 + d2 - d3
        D = gpflow.utilities.add_noise_cov(D, jitter)
        LD = tf.linalg.cholesky(D)

        rhs = tf.linalg.triangular_solve(LD, Lbinv_c, lower=True)          # [Mb,1]

        Qff_diag = tf.reduce_sum(tf.square(Lbinv_Kbf), axis=0)             # [N]

        tf.debugging.assert_all_finite(Lb,  "Lb has NaN/Inf")
        tf.debugging.assert_all_finite(LD,  "LD has NaN/Inf")
        tf.debugging.assert_all_finite(rhs, "rhs has NaN/Inf")

        return (Kbf, Kba, Kaa, Kaa_cur, La, Kbb, Lb, D, LD, Lbinv_Kba, rhs, err, Qff_diag)

    def maximum_log_likelihood_objective(self):
        sigma2 = self.likelihood.variance
        N = tf.cast(tf.shape(self.X)[0], DTYPE)

        Saa = self.Su_old
        ma  = self.mu_old
        Kfdiag = self.kernel(self.X, full_cov=False)

        (Kbf, Kba, Kaa, Kaa_cur, La, Kbb, Lb, D, LD,
         Lbinv_Kba, rhs, err, Qff_diag) = self._common_terms()

        LSa = tf.linalg.cholesky(Saa)
        Lainv_ma = tf.linalg.triangular_solve(LSa, ma, lower=True)

        bound = -0.5 * N * np.log(2.0 * np.pi)
        bound += -0.5 * tf.reduce_sum(tf.square(err)) / sigma2
        bound += -0.5 * tf.reduce_sum(tf.square(Lainv_ma))
        bound +=  0.5 * tf.reduce_sum(tf.square(rhs))

        bound += -0.5 * N * tf.math.log(sigma2)
        bound += -tf.reduce_sum(tf.math.log(tf.linalg.diag_part(LD)))

        bound += -0.5 * tf.reduce_sum(Kfdiag) / sigma2
        bound +=  0.5 * tf.reduce_sum(Qff_diag)

        bound += tf.reduce_sum(tf.math.log(tf.linalg.diag_part(La)))
        bound += -tf.reduce_sum(tf.math.log(tf.linalg.diag_part(LSa)))

        # correction term involving Kaa_cur - Qaa
        Kaadiff = Kaa_cur - tf.matmul(Lbinv_Kba, Lbinv_Kba, transpose_a=True)
        Sainv_Kaadiff = tf.linalg.solve(Saa, Kaadiff)
        Kainv_Kaadiff = tf.linalg.solve(Kaa, Kaadiff)

        bound += -0.5 * tf.reduce_sum(
            tf.linalg.diag_part(Sainv_Kaadiff) - tf.linalg.diag_part(Kainv_Kaadiff)
        )
        return bound

    def predict_f(self, Xnew, full_cov=False):
        jitter = gpflow.utilities.to_default_float(1e-6)

        Kbs = covariances.Kuf(self.inducing_variable, self.kernel, Xnew)  # [Mb, Nnew]
        (_, _, _, _, _, _, Lb, _, LD, _, rhs, _, _) = self._common_terms()

        Lbinv_Kbs = tf.linalg.triangular_solve(Lb, Kbs, lower=True)
        LDinv_Lbinv_Kbs = tf.linalg.triangular_solve(LD, Lbinv_Kbs, lower=True)
        mean = tf.matmul(LDinv_Lbinv_Kbs, rhs, transpose_a=True)  # [Nnew,1]

        if full_cov:
            Kss = self.kernel(Xnew) + jitter * tf.eye(tf.shape(Xnew)[0], dtype=DTYPE)
            var = (
                Kss
                - tf.matmul(Lbinv_Kbs, Lbinv_Kbs, transpose_a=True)
                + tf.matmul(LDinv_Lbinv_Kbs, LDinv_Lbinv_Kbs, transpose_a=True)
            )
            return mean + self.mean_function(Xnew), var
        else:
            var = (
                self.kernel(Xnew, full_cov=False)
                - tf.reduce_sum(tf.square(Lbinv_Kbs), axis=0)
                + tf.reduce_sum(tf.square(LDinv_Lbinv_Kbs), axis=0)
            )
            var = tf.maximum(var, tf.cast(1e-12, var.dtype))
            return mean + self.mean_function(Xnew), var

    def build_predict_cache(self):
        """Build cached matrices for fast predict_f_cached(). Call after training / after each update."""
        (_, _, _, _, _, _, Lb, _, LD, _, rhs, _, _) = self._common_terms()
        self._cache_Lb = Lb
        self._cache_LD = LD
        self._cache_rhs = rhs
        self._cache_ready = True

    def predict_f_cached(self, Xnew, full_cov=False):
        """Fast diag prediction using cached Lb, LD, rhs."""
        if not self._cache_ready:
            return self.predict_f(Xnew, full_cov=full_cov)

        jitter = gpflow.utilities.to_default_float(1e-6)
        Lb  = self._cache_Lb
        LD  = self._cache_LD
        rhs = self._cache_rhs

        Kbs = covariances.Kuf(self.inducing_variable, self.kernel, Xnew)  # [Mb,Nnew]
        Lbinv_Kbs = tf.linalg.triangular_solve(Lb, Kbs, lower=True)
        LDinv_Lbinv_Kbs = tf.linalg.triangular_solve(LD, Lbinv_Kbs, lower=True)
        mean = tf.matmul(LDinv_Lbinv_Kbs, rhs, transpose_a=True)

        if full_cov:
            Kss = self.kernel(Xnew) + jitter * tf.eye(tf.shape(Xnew)[0], dtype=DTYPE)
            var = (
                Kss
                - tf.matmul(Lbinv_Kbs, Lbinv_Kbs, transpose_a=True)
                + tf.matmul(LDinv_Lbinv_Kbs, LDinv_Lbinv_Kbs, transpose_a=True)
            )
            return mean + self.mean_function(Xnew), var
        else:
            var = (
                self.kernel(Xnew, full_cov=False)
                - tf.reduce_sum(tf.square(Lbinv_Kbs), axis=0)
                + tf.reduce_sum(tf.square(LDinv_Lbinv_Kbs), axis=0)
            )
            var = tf.maximum(var, tf.cast(1e-12, var.dtype))
            return mean + self.mean_function(Xnew), var

# ----------------------------
# training helper
# ----------------------------
def train_osgpr(model, iters=250, lr=0.02, clip_norm=10.0):
    """Adam optimize the negative ELBO."""
    opt = tf.keras.optimizers.Adam(lr)

    @tf.function
    def step():
        with tf.GradientTape() as tape:
            loss = -model.maximum_log_likelihood_objective()
        grads = tape.gradient(loss, model.trainable_variables)
        if clip_norm is not None:
            grads = [tf.clip_by_norm(g, clip_norm) if g is not None else None for g in grads]
        opt.apply_gradients([(g, v) for g, v in zip(grads, model.trainable_variables) if g is not None])
        return loss

    t0 = time.time()
    last = None
    for _ in range(int(iters)):
        last = step()
    return float(time.time() - t0), float(last.numpy())

# ----------------------------
# summaries (to chain online)
# ----------------------------
def prior_summary(kernel, Z):
    """
    Prior summary at inducing Z for the first model:
      mu0 = 0
      Su0 = Kzz
      Kaa0 = Kzz
    """
    Z = np.asarray(Z, dtype=np.float64)
    Kzz = kernel.K(Z).numpy()
    Kzz = sym_jitter(Kzz, 1e-6)
    mu0 = np.zeros((Z.shape[0], 1), dtype=np.float64)
    return mu0, Kzz, Kzz, Z

def extract_summary_from_model(model):
    """
    Extract q(u)=N(mu,Su) at model's current inducing Z plus Kaa=K(Z,Z).
    """
    Z = model.inducing_variable.Z.numpy().astype(np.float64)

    mu_tf, Sig_tf = model.predict_f(Z, full_cov=True)
    mu = mu_tf.numpy().reshape(-1, 1)

    Su = Sig_tf.numpy()
    if Su.ndim == 3:
        Su = Su[0]
    Su = sym_jitter(Su, 1e-6)

    Kaa = model.kernel.K(Z).numpy()
    Kaa = sym_jitter(Kaa, 1e-6)
    return mu, Su, Kaa, Z

# ============================================================
# Anchors: greedy D-opt (log-det) on Kzz
# ============================================================
def greedy_dopt_anchors_from_K(Kzz, m_anchors=24, lam=1e-6):
    """
    Greedy log-det anchor selection on PSD Kzz using incremental Cholesky updates.
    Returns indices of size m_anchors.
    """
    K = np.asarray(Kzz, dtype=np.float64)
    M = K.shape[0]
    assert K.shape == (M, M)
    K = sym_jitter(K, lam)

    chosen = []
    diag = np.clip(np.diag(K).copy(), 1e-12, None)
    remaining = np.ones(M, dtype=bool)
    L = None

    for k in range(min(int(m_anchors), M)):
        if k == 0:
            i = int(np.argmax(diag))
            chosen.append(i)
            remaining[i] = False
            L = np.array([[np.sqrt(diag[i])]], dtype=np.float64)
            continue

        S = np.array(chosen, dtype=np.int64)
        Ks_all = K[np.ix_(S, np.arange(M))]     # (k,M)

        v = np.linalg.solve(L, Ks_all)          # (k,M)
        vn2 = np.sum(v * v, axis=0)             # (M,)
        s2 = diag - vn2
        s2 = np.where(remaining, s2, -np.inf)

        i = int(np.argmax(s2))
        if not np.isfinite(s2[i]) or s2[i] <= 1e-12:
            cand = np.where(remaining)[0]
            if len(cand) == 0:
                break
            i = int(cand[np.argmax(diag[cand])])
            s2_i = max(diag[i], 1e-12)
        else:
            s2_i = float(s2[i])

        chosen.append(i)
        remaining[i] = False

        kvec = K[np.ix_(S, [i])].reshape(-1, 1)  # (k,1)
        w = np.linalg.solve(L, kvec)             # (k,1)
        alpha = np.sqrt(max(s2_i, 1e-12))

        L_new = np.zeros((k + 1, k + 1), dtype=np.float64)
        L_new[:k, :k] = L
        L_new[k, :k] = w.reshape(-1)
        L_new[k, k] = alpha
        L = L_new

    return np.array(chosen, dtype=np.int64)

# ============================================================
# Online update builder (GLOBAL update step)
# ============================================================
def rebuild_osgpr_from_old_summary(
    model_old,
    X_new,
    Y_new,
    Z_new=None,
    iters=120,
    lr=0.02,
    noise=1e-4,
    freeze_kernel=False,
    clip_norm=10.0,
):
    """
    Build a NEW OSGPR_VFE model using:
      - old posterior summary extracted from model_old at its inducing Z_old
      - new executed batch (X_new, Y_new)
      - inducing set Z_new (defaults to model_old.Z; you may pass a refreshed Z here)

    Returns:
      model_new, train_time_sec, last_neg_obj
    """
    # old summary
    mu_old, Su_old, Kaa_old, Z_old = extract_summary_from_model(model_old)

    # inducing set for the new model
    if Z_new is None:
        Z_use = Z_old
    else:
        Z_use = np.asarray(Z_new, dtype=np.float64)

    # clone kernel to avoid variable-sharing surprises
    k_new = clone_kernel(model_old.kernel)

    m = OSGPR_VFE(
        data=(np.asarray(X_new, dtype=np.float64), np.asarray(Y_new, dtype=np.float64)),
        kernel=k_new,
        mu_old=mu_old, Su_old=Su_old, Kaa_old=Kaa_old, Z_old=Z_old,
        Z=Z_use,
    )
    m.likelihood.variance.assign(float(noise))

    if freeze_kernel:
        try:
            m.kernel.variance.trainable = False
            m.kernel.lengthscales.trainable = False
        except Exception:
            pass

    t_sec, neg = train_osgpr(m, iters=iters, lr=lr, clip_norm=clip_norm)
    m.build_predict_cache()
    return m, float(t_sec), float(neg)

print("✅ OSGPR core + helpers ready (Cell 3 aligned to pipeline)")


In [ ]:
# ===========================
# Cell 4 — Train INITIAL GLOBAL OSGPR models (CartPole, 4 outputs)
#   + GLOBAL evaluation plots (mean + uncertainty)
#   + Anchor reselection utilities (for the new pipeline)
#
# New pipeline alignment:
#   - Z_GLOBAL is FIXED capacity of global model
#   - After EVERY global update, we will:
#       (1) recompute ANCHOR_IDX from the UPDATED global model kernel on Z_GLOBAL
#       (2) choose tube-based subset + anchors -> build local predictors for MPPI
#
# After training this cell gives:
#   - m_dx, m_dxdot, m_dth, m_dthdot  (GLOBAL OSGPR models)
#   - Z_GLOBAL  (fixed)
#   - refresh_global_anchors()  -> updates ANCHOR_IDX using the current model kernel
#   - Plot helpers: slice + surface for any model (mean & std)
# ===========================

import numpy as np
import gpflow
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import plotly.graph_objects as go

SEED = 0
rng = np.random.default_rng(SEED)

# ----------------------------
# 0) Clean / finite filter
# ----------------------------
mask = finite_mask(X0, Ydx0, Ydxdot0, Ydth0, Ydthdot0)
X0f       = X0[mask]
Ydx0f     = Ydx0[mask]
Ydxdot0f  = Ydxdot0[mask]
Ydth0f    = Ydth0[mask]
Ydthdot0f = Ydthdot0[mask]
print("Data kept:", X0f.shape[0], "/", X0.shape[0])

# ----------------------------
# 1) Train/test split
# ----------------------------
N = X0f.shape[0]
perm = rng.permutation(N)
N_test = max(50, int(0.2 * N))
test_idx  = perm[:N_test]
train_idx = perm[N_test:]

Xtr = X0f[train_idx]
Xte = X0f[test_idx]

Ydx_tr,     Ydx_te     = Ydx0f[train_idx],     Ydx0f[test_idx]
Ydxdot_tr,  Ydxdot_te  = Ydxdot0f[train_idx],  Ydxdot0f[test_idx]
Ydth_tr,    Ydth_te    = Ydth0f[train_idx],    Ydth0f[test_idx]
Ydthdot_tr, Ydthdot_te = Ydthdot0f[train_idx], Ydthdot0f[test_idx]

print("Train:", Xtr.shape, " Test:", Xte.shape)

# ----------------------------
# 2) GLOBAL inducing set (fixed capacity) -- FIXED
# ----------------------------
M_GLOBAL = 256  # Target capacity (128/256 recommended)
N_avail  = Xtr.shape[0]

if N_avail >= M_GLOBAL:
    # Option A: We have enough data. Sample distinct points (standard).
    idxZ = rng.choice(N_avail, size=M_GLOBAL, replace=False)
    Z_GLOBAL = Xtr[idxZ].copy().astype(np.float64)
else:
    # Option B: Not enough data. Sample with replacement to fill buffer.
    print(f"⚠️ Warning: Train data ({N_avail}) < M_GLOBAL ({M_GLOBAL}). Sampling with replacement.")
    idxZ = rng.choice(N_avail, size=M_GLOBAL, replace=True)
    Z_GLOBAL = Xtr[idxZ].copy().astype(np.float64)

    # CRITICAL: Add jitter to duplicates.
    # If Z contains exact duplicates, Kzz is singular -> Cholesky fails.
    jitter_z = rng.standard_normal(Z_GLOBAL.shape) * 1e-6
    Z_GLOBAL += jitter_z

print("Z_GLOBAL:", Z_GLOBAL.shape)

# ----------------------------
# 3) Kernels (6D ARD SE)
# ----------------------------
def make_kernel_6d():
    # You can tune initial hyperparams; kernel may be frozen later in online loop for speed
    return gpflow.kernels.SquaredExponential(
        lengthscales=np.ones(6, dtype=np.float64),
        variance=1.0
    )

k_dx     = make_kernel_6d()
k_dxdot  = make_kernel_6d()
k_dth    = make_kernel_6d()
k_dthdot = make_kernel_6d()

# ----------------------------
# 4) Build + train helper
# ----------------------------
def build_and_train_global_model(kernel, X, Y, Z_init, name,
                                 iters=300, lr=0.02, noise=1e-4):
    # first model uses prior summary at Z_GLOBAL
    mu_old, Su_old, Kaa_old, Z_old = prior_summary(kernel, Z_init)

    m = OSGPR_VFE(
        data=(np.asarray(X, dtype=np.float64), np.asarray(Y, dtype=np.float64)),
        kernel=kernel,
        mu_old=mu_old, Su_old=Su_old, Kaa_old=Kaa_old, Z_old=Z_old,
        Z=Z_init
    )
    m.likelihood.variance.assign(float(noise))

    print(f"\nTraining {name} ...")
    t, neg = train_osgpr(m, iters=iters, lr=lr, clip_norm=10.0)
    print(f"{name} done | train={t:.2f}s | neg_obj={neg:.4f} | noise={float(m.likelihood.variance.numpy()):.2e}")

    m.build_predict_cache()
    return m

# ----------------------------
# 5) Train 4 global models
# ----------------------------
m_dx     = build_and_train_global_model(k_dx,     Xtr, Ydx_tr,     Z_GLOBAL, "dx",        iters=300, lr=0.02, noise=1e-4)
m_dxdot  = build_and_train_global_model(k_dxdot,  Xtr, Ydxdot_tr,  Z_GLOBAL, "dxdot",     iters=300, lr=0.02, noise=1e-4)
m_dth    = build_and_train_global_model(k_dth,    Xtr, Ydth_tr,    Z_GLOBAL, "dtheta",    iters=300, lr=0.02, noise=1e-4)
m_dthdot = build_and_train_global_model(k_dthdot, Xtr, Ydthdot_tr, Z_GLOBAL, "dthetadot", iters=300, lr=0.02, noise=1e-4)

print("\n✅ Global models trained + caches ready")

# ----------------------------
# 6) Numeric eval (RMSE on held-out)
# ----------------------------
def rmse_on_test(model, Xte, Yte):
    Xte = np.asarray(Xte, dtype=np.float64)
    y = np.asarray(Yte, dtype=np.float64).reshape(-1)
    if hasattr(model, "predict_f_cached"):
        mu, _ = model.predict_f_cached(Xte, full_cov=False)
    else:
        mu, _ = model.predict_f(Xte, full_cov=False)
    yhat = mu.numpy().reshape(-1)
    return float(np.sqrt(np.mean((yhat - y) ** 2)))

print("\nHeld-out RMSE:")
print("  dx        :", rmse_on_test(m_dx,     Xte, Ydx_te))
print("  dxdot     :", rmse_on_test(m_dxdot,  Xte, Ydxdot_te), "   <-- Δxdot")
print("  dtheta    :", rmse_on_test(m_dth,    Xte, Ydth_te))
print("  dthetadot :", rmse_on_test(m_dthdot, Xte, Ydthdot_te))

# ============================================================
# 7) Anchor reselection utilities (CRITICAL for new pipeline)
# ============================================================
def reselect_anchors_from_model(model, m_anchors=24, lam=1e-6):
    """
    Anchors chosen on K(Z_GLOBAL,Z_GLOBAL) using the model's CURRENT kernel hyperparams.
    This must be rerun after each global update if kernel changes OR if you want anchors to adapt.
    """
    Z = model.inducing_variable.Z.numpy().astype(np.float64)
    Kzz = model.kernel.K(Z).numpy()
    Kzz = sym_jitter(Kzz, lam)
    m_anchors = int(min(m_anchors, Z.shape[0]))
    idx = greedy_dopt_anchors_from_K(Kzz, m_anchors=m_anchors, lam=lam)
    return idx

ANCHOR_M = min(24, int(Z_GLOBAL.shape[0]))
ANCHOR_IDX = reselect_anchors_from_model(m_dxdot, m_anchors=ANCHOR_M, lam=1e-6)  # pick any of the 4 models
print("ANCHOR_IDX:", ANCHOR_IDX.shape, f"(anchors={len(ANCHOR_IDX)})")

def refresh_global_anchors(m_anchors=24):
    """
    Call this after global update (especially if you let kernel hyperparams move).
    Uses m_dxdot by default; you can change to m_dx etc.
    """
    global ANCHOR_IDX
    ANCHOR_IDX = reselect_anchors_from_model(m_dxdot, m_anchors=m_anchors, lam=1e-6)
    return ANCHOR_IDX

# ============================================================
# 8) Plot helpers (GLOBAL): mean + uncertainty
# ============================================================
def gp_predict_mu_std_fast(model, X):
    X = np.asarray(X, dtype=np.float64)
    if hasattr(model, "predict_f_cached"):
        mu_tf, var_tf = model.predict_f_cached(X, full_cov=False)
    else:
        mu_tf, var_tf = model.predict_f(X, full_cov=False)
    mu = mu_tf.numpy().reshape(-1)
    var = var_tf.numpy().reshape(-1)
    std = np.sqrt(np.maximum(var, 1e-12))
    return mu, std

def get_inducing_Z_np(model):
    return model.inducing_variable.Z.numpy().astype(np.float64)

# ---- Slice: sweep physical x, fixed xdot/theta/thetadot, compare u=+1/-1 ----
def plot_slice_x_two_actions_cartpole(
    model,
    X_train, y_train,
    title="Slice",
    y_label="Δy",
    xdot_fixed=0.0,
    theta_fixed=0.0,
    thetadot_fixed=0.0,
    x_min=-2.4, x_max=2.4,
    n_grid=260,
    u_list=(+1.0, -1.0),
    y_min=None, y_max=None,
    x_tick_step=0.6,
    y_tick_step=None,
    show_data=True,
    show_inducing=True,
    tol_xdot_feat=0.12,
    tol_theta=0.25,
    tol_w_feat=0.12,
    tol_u=0.10,
):
    x_grid = np.linspace(x_min, x_max, n_grid)
    Z = get_inducing_Z_np(model)

    curves = []
    auto_ymin, auto_ymax = +np.inf, -np.inf

    for u_fixed in u_list:
        X_query = np.vstack([
            state_to_features(x, xdot_fixed, theta_fixed, thetadot_fixed, u_fixed)
            for x in x_grid
        ]).astype(np.float64)
        mu, std = gp_predict_mu_std_fast(model, X_query)
        lo, hi = mu - 2 * std, mu + 2 * std
        curves.append((u_fixed, mu, std, lo, hi))
        auto_ymin, auto_ymax = min(auto_ymin, float(lo.min())), max(auto_ymax, float(hi.max()))

    if y_min is None: y_min = auto_ymin
    if y_max is None: y_max = auto_ymax

    plt.figure(figsize=(9, 5))

    X_train = np.asarray(X_train, dtype=np.float64)
    y_train = np.asarray(y_train, dtype=np.float64).reshape(-1)

    xdot_feat0 = np.tanh(xdot_fixed / 3.0)
    w_feat0    = np.tanh(thetadot_fixed / 8.0)

    for u_fixed, mu, std, lo, hi in curves:
        plt.plot(x_grid, mu, lw=2.5, label=f"mean (u={u_fixed:+.1f})")
        plt.fill_between(x_grid, lo, hi, alpha=0.18, label=f"±2σ (u={u_fixed:+.1f})")

        if show_data:
            th_train = np.arctan2(X_train[:, 2], X_train[:, 3])
            mask = (
                (np.abs(X_train[:, 1] - xdot_feat0) < tol_xdot_feat) &
                (np.abs(wrap_pi(th_train - theta_fixed)) < tol_theta) &
                (np.abs(X_train[:, 4] - w_feat0) < tol_w_feat) &
                (np.abs(X_train[:, 5] - float(u_fixed)) < tol_u)
            )
            if np.sum(mask) > 0:
                x_feat = np.clip(X_train[mask, 0], -0.999, 0.999)
                x_phys = 2.4 * np.arctanh(x_feat)
                plt.scatter(x_phys, y_train[mask], s=22, alpha=0.60,
                            label=f"data near slice (u≈{u_fixed:+.1f}, n={np.sum(mask)})")

        if show_inducing and (Z is not None):
            thZ = np.arctan2(Z[:, 2], Z[:, 3])
            maskZ = (
                (np.abs(Z[:, 1] - xdot_feat0) < tol_xdot_feat) &
                (np.abs(wrap_pi(thZ - theta_fixed)) < tol_theta) &
                (np.abs(Z[:, 4] - w_feat0) < tol_w_feat) &
                (np.abs(Z[:, 5] - float(u_fixed)) < tol_u)
            )
            if np.sum(maskZ) > 0:
                Zsel = Z[maskZ]
                muZ, _ = gp_predict_mu_std_fast(model, Zsel)
                x_feat = np.clip(Zsel[:, 0], -0.999, 0.999)
                x_phys = 2.4 * np.arctanh(x_feat)
                plt.scatter(x_phys, muZ, marker="x", s=70, linewidths=2.0,
                            label=f"inducing Z (u≈{u_fixed:+.1f}, M={np.sum(maskZ)})")

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel("Cart position x")
    plt.ylabel(y_label)

    ax = plt.gca()
    if x_tick_step is not None:
        ax.xaxis.set_major_locator(MultipleLocator(float(x_tick_step)))
    if y_tick_step is not None:
        ax.yaxis.set_major_locator(MultipleLocator(float(y_tick_step)))

    plt.grid(True, alpha=0.25)
    plt.title(title + f"\n(fixed: xdot={xdot_fixed:.2f}, theta={theta_fixed:.2f}, thetadot={thetadot_fixed:.2f})")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

# ---- 3D Surface: (x, xdot) -> mean Δ, colored by std ----
def plot_surface_x_xdot_mean_colored_by_std(
    model,
    title="3D Surface (interactive)",
    z_label="Δy",
    theta_fixed=0.0,
    thetadot_fixed=0.0,
    u_fixed=+1.0,
    x_min=-2.4, x_max=2.4,
    xdot_min=-3.0, xdot_max=3.0,
    n_grid=70,
    show_inducing=True,
    tol_theta=0.25,
    tol_w_feat=0.12,
    tol_u=0.10,
):
    x_grid = np.linspace(x_min, x_max, n_grid)
    xd_grid = np.linspace(xdot_min, xdot_max, n_grid)
    X, XD = np.meshgrid(x_grid, xd_grid)

    X_feat = np.vstack([
        state_to_features(x, xdot, theta_fixed, thetadot_fixed, u_fixed)
        for x, xdot in zip(X.ravel(), XD.ravel())
    ]).astype(np.float64)

    mu, std = gp_predict_mu_std_fast(model, X_feat)
    Mean = mu.reshape(X.shape)
    Std  = std.reshape(X.shape)

    surface = go.Surface(
        x=X, y=XD, z=Mean,
        surfacecolor=Std,
        colorscale="Viridis",
        colorbar=dict(title="Std"),
        opacity=0.95,
        showscale=True,
        name="surface"
    )
    traces = [surface]

    if show_inducing:
        Z = get_inducing_Z_np(model)
        thZ = np.arctan2(Z[:, 2], Z[:, 3])
        w_feat0 = np.tanh(thetadot_fixed / 8.0)
        maskZ = (
            (np.abs(wrap_pi(thZ - theta_fixed)) < tol_theta) &
            (np.abs(Z[:, 4] - w_feat0) < tol_w_feat) &
            (np.abs(Z[:, 5] - float(u_fixed)) < tol_u)
        )
        if np.sum(maskZ) > 0:
            Zsel = Z[maskZ]
            muZ, _ = gp_predict_mu_std_fast(model, Zsel)

            x_feat = np.clip(Zsel[:, 0], -0.999, 0.999)
            xdot_feat = np.clip(Zsel[:, 1], -0.999, 0.999)
            x_phys = 2.4 * np.arctanh(x_feat)
            xdot_phys = 3.0 * np.arctanh(xdot_feat)

            traces.append(
                go.Scatter3d(
                    x=x_phys, y=xdot_phys, z=muZ,
                    mode="markers",
                    marker=dict(size=4, color="red", opacity=0.95),
                    name=f"inducing Z (near θ,θdot,u) | M={np.sum(maskZ)}"
                )
            )

    fig = go.Figure(data=traces)
    fig.update_layout(
        title=f"{title} | fixed u={u_fixed:+.1f}, theta={theta_fixed:.2f}, thetadot={thetadot_fixed:.2f}",
        scene=dict(
            xaxis=dict(title="x", range=[x_min, x_max]),
            yaxis=dict(title="xdot", range=[xdot_min, xdot_max]),
            zaxis=dict(title=z_label),
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=650
    )
    fig.show()

print("\n✅ Cell 4 complete (trained global models + anchor refresh + plot helpers).")

In [ ]:
# ============================
# Cell 5 — Evaluate + Visualize GLOBAL GP (Reusable)
#
# What this cell provides:
#   ✅ RMSE for dx, dxdot, dtheta, dthetadot on Xte
#   ✅ Slice plot (mean ±2σ) for chosen output model (default: Δxdot)
#   ✅ 3D Surface mean colored by std (default: Δxdot over x, xdot)
#   ✅ NEW: 3D Surface of std alone (so you can SEE uncertainty shrink/grow)
#
# Assumes you ran:
#   Cell 4 (trained global models + plot helpers already defined)
#
# Expected variables available:
#   m_dx, m_dxdot, m_dth, m_dthdot
#   Xte, Ydx_te, Ydxdot_te, Ydth_te, Ydthdot_te
#   plot_slice_x_two_actions_cartpole()
#   plot_surface_x_xdot_mean_colored_by_std()
#   gp_predict_mu_std_fast()
# ============================

import numpy as np
import plotly.graph_objects as go

# ----------------------------
# 0) RMSE helper
# ----------------------------
def rmse_np(yhat, y):
    yhat = np.asarray(yhat).reshape(-1)
    y = np.asarray(y).reshape(-1)
    return float(np.sqrt(np.mean((yhat - y) ** 2)))

# ----------------------------
# 1) RMSE report
# ----------------------------
def print_global_rmse():
    mu_dx, _     = gp_predict_mu_std_fast(m_dx,     Xte)
    mu_dxdot, _  = gp_predict_mu_std_fast(m_dxdot,  Xte)
    mu_dth, _    = gp_predict_mu_std_fast(m_dth,    Xte)
    mu_dthdot, _ = gp_predict_mu_std_fast(m_dthdot, Xte)

    print("=== Test RMSE (global models) ===")
    print(f"dx        RMSE: {rmse_np(mu_dx,     Ydx_te):.6f}")
    print(f"dxdot     RMSE: {rmse_np(mu_dxdot,  Ydxdot_te):.6f}   (Δxdot / delta v)")
    print(f"dtheta    RMSE: {rmse_np(mu_dth,    Ydth_te):.6f}")
    print(f"dthetadot RMSE: {rmse_np(mu_dthdot, Ydthdot_te):.6f}")

# ----------------------------
# 2) Std-only surface (uncertainty surface)
# ----------------------------
def plot_surface_x_xdot_std_only(
    model,
    title="3D Std surface (uncertainty only)",
    theta_fixed=0.0,
    thetadot_fixed=0.0,
    u_fixed=+1.0,
    x_min=-2.4, x_max=2.4,
    xdot_min=-3.0, xdot_max=3.0,
    n_grid=70,
):
    x_grid = np.linspace(x_min, x_max, n_grid)
    xd_grid = np.linspace(xdot_min, xdot_max, n_grid)
    X, XD = np.meshgrid(x_grid, xd_grid)

    X_feat = np.vstack([
        state_to_features(x, xdot, theta_fixed, thetadot_fixed, u_fixed)
        for x, xdot in zip(X.ravel(), XD.ravel())
    ]).astype(np.float64)

    _, std = gp_predict_mu_std_fast(model, X_feat)
    Std = std.reshape(X.shape)

    fig = go.Figure(
        data=[
            go.Surface(
                x=X, y=XD, z=Std,
                colorscale="Viridis",
                colorbar=dict(title="Std"),
                opacity=0.98,
                showscale=True,
                name="std surface"
            )
        ]
    )
    fig.update_layout(
        title=f"{title} | fixed u={u_fixed:+.1f}, theta={theta_fixed:.2f}, thetadot={thetadot_fixed:.2f}",
        scene=dict(
            xaxis=dict(title="x", range=[x_min, x_max]),
            yaxis=dict(title="xdot", range=[xdot_min, xdot_max]),
            zaxis=dict(title="std"),
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=650
    )
    fig.show()

# ----------------------------
# 3) One-call global evaluation bundle (REUSABLE after updates)
# ----------------------------
def eval_and_plot_global(tag="GLOBAL (init)",
                         model_for_plots=None,
                         theta_fixed=0.0,
                         thetadot_fixed=0.0,
                         u_fixed=+1.0):
    """
    model_for_plots: choose which output model to visualize (default m_dxdot)
    """
    if model_for_plots is None:
        model_for_plots = m_dxdot

    print("\n==============================")
    print(f"GLOBAL EVAL: {tag}")
    print("==============================")
    print_global_rmse()

    # Slice: mean ±2σ, compare u=+1/-1
    plot_slice_x_two_actions_cartpole(
        model=model_for_plots,
        X_train=Xtr,
        y_train=Ydxdot_tr if (model_for_plots is m_dxdot) else Ydx_tr,
        title=f"{tag} slice: mean ±2σ (u=+1/-1) + inducing",
        y_label="Δxdot" if (model_for_plots is m_dxdot) else "Δx",
        xdot_fixed=0.0,
        theta_fixed=theta_fixed,
        thetadot_fixed=thetadot_fixed,
        x_min=-2.4, x_max=2.4,
        n_grid=260,
        u_list=(+1.0, -1.0),
    )

    # Surface: mean colored by std (your current favorite)
    plot_surface_x_xdot_mean_colored_by_std(
        model=model_for_plots,
        title=f"{tag} surface: mean colored by std (+ inducing)",
        z_label="Δxdot" if (model_for_plots is m_dxdot) else "Δx",
        theta_fixed=theta_fixed,
        thetadot_fixed=thetadot_fixed,
        u_fixed=u_fixed,
        x_min=-2.4, x_max=2.4,
        xdot_min=-3.0, xdot_max=3.0,
        n_grid=70,
        show_inducing=True,
    )

    # NEW: std-only surface (uncertainty itself)
    plot_surface_x_xdot_std_only(
        model=model_for_plots,
        title=f"{tag} surface: std only (uncertainty)",
        theta_fixed=theta_fixed,
        thetadot_fixed=thetadot_fixed,
        u_fixed=u_fixed,
        x_min=-2.4, x_max=2.4,
        xdot_min=-3.0, xdot_max=3.0,
        n_grid=70,
    )

# ----------------------------
# RUN ONCE FOR INITIAL GLOBAL
# ----------------------------
eval_and_plot_global(tag="GLOBAL (initial)", model_for_plots=m_dxdot, u_fixed=+1.0)


In [ ]:
# ===========================
# Cell 6 — MPPI + Online OSGPR (FIXED global inducing) + Local subset (PALSGP-style)
#   ✅ MULTI-RUN evaluation:
#       - N_RUNS = 3
#       - Each RUN has N_EPISODES_PER_RUN = 3 continuous episodes
#       - Models KEEP training across episodes inside a run
#       - BEFORE EACH RUN (including run 1): full reset to the same initial models + same initial Z_GLOBAL
#
#   ✅ Rendering:
#       - ONLY 1 render per RUN (episode 0), others no render
#       - BUT that 1 render is a SINGLE animation that stitches ALL episodes in the run
#
#   ✅ Timing curves (mean over runs):
#       - training time per timestep (non-cumulative)
#       - training time cumulative per timestep
#       - prediction time per timestep (MPPI planning time)
#       - wall time cumulative per timestep (EXCLUDING visualization)
#
#   ✅ Surfaces:
#       - Plot surface after EACH RUN (global + local using last idx_sub)
# ===========================

import time
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import gpflow
import tensorflow as tf
from scipy.stats import chi2
from gpflow.utilities import parameter_dict, multiple_assign

# ----------------------------
# Enforce float64 (per your pipeline going forward)
# ----------------------------
try:
    gpflow.config.set_default_float(np.float64)
except Exception:
    pass
DTYPE_TF = gpflow.default_float()
assert DTYPE_TF == tf.float64, f"Expected float64 default, got {DTYPE_TF}. Set gpflow default float to float64 in earlier cells."

# ----------------------------
# GPU sanity / enforcement
# ----------------------------
REQUIRE_GPU = False
LOG_DEVICE_PLACEMENT = False

print("TF built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs visible:", tf.config.list_physical_devices("GPU"))
print("Logical GPUs:", tf.config.list_logical_devices("GPU"))

if LOG_DEVICE_PLACEMENT:
    tf.debugging.set_log_device_placement(True)

if REQUIRE_GPU:
    gpus = tf.config.list_physical_devices("GPU")
    if len(gpus) == 0:
        raise RuntimeError("REQUIRE_GPU=True but no GPU is visible to TensorFlow. Fix CUDA/TF install or set REQUIRE_GPU=False.")

# ----------------------------
# Config (tune here)  (UNCHANGED: uses exactly your parameters)
# ----------------------------
HORIZON    = 40
K_SAMPLES  = 128
SIGMA      = 0.6
LAMBDA     = 1.0

UPDATE_EVERY = 30
ITERS_UPDATE = 200
LR_UPDATE    = 0.02
NOISE_UPDATE = 1e-4

M_GLOBAL = 256

M_SUB           = 48
DIVERSITY_FRAC  = 0.35

ANCHOR_M   = 12
ANCHOR_LAM = 1e-6

X_BAND      = 0.60
UPRIGHT_COS = 0.85
HOLD_STEPS  = 200

# Recording (visualization; excluded from runtime)
RECORD_RGB_DEFAULT   = True
RESIZE       = (720, 450)
FPS          = 10
FRAME_STRIDE = 2

# Exploration schedule
EXPLORE_STEPS = 1
UNC_W_MAX     = 15.0
UNC_W_MIN     = 0.0
CENTER_W      = 1.0
U_W           = 0.005
UPRIGHT_W     = 2.0

if "U_MIN" not in globals(): U_MIN = -1.0
if "U_MAX" not in globals(): U_MAX = +1.0

PLOT_EACH_UPDATE = False

LOCAL_REBUILD_EVERY   = 10
LOCAL_OVERLAP_THRESH  = 0.70

TUBE_ALPHA      = 0.99
TUBE_COV_EPS    = 1e-6
TUBE_TIME_BINS  = 16
TUBE_FALLBACK_CANDIDATES = 400

STD_CMIN_FIXED = 0.0
STD_FIXED_Q    = 0.5
STD_MODE       = "fixed"
STD_CMAX_FIXED = None

PLOT_X_MIN, PLOT_X_MAX = -2.4, 2.4
PLOT_V_MIN, PLOT_V_MAX = -3.0, 3.0
PLOT_N_GRID            = 60
PLOT_TH_FIXED          = 0.0
PLOT_THDOT_FIXED       = 0.0
PLOT_U_FIXED           = +1.0

STORE_GLOBAL_SURFACES = True
GLOBAL_SURF_HISTORY = []   # reset per RUN

# ----------------------------
# Safety: require dependencies from earlier cells
# ----------------------------
required = [
    "make_env", "obs_to_state", "wrap_pi", "state_to_features",
    "batch_state_to_features", "OSGPR_VFE", "train_osgpr",
    "extract_summary_from_model", "greedy_dopt_anchors_from_K",
    "clone_kernel",
]
missing = [k for k in required if k not in globals()]
if len(missing) > 0:
    raise NameError(f"Cell 6 missing required symbols from earlier cells: {missing}")

if "render_cartpole_frame_from_state" not in globals():
    print("⚠️ render_cartpole_frame_from_state not found. RGB recording will be disabled.")
    RECORD_RGB_DEFAULT = False

# ----------------------------
# Helper: sym_jitter
# ----------------------------
def sym_jitter(A, jitter=1e-6):
    A = np.asarray(A, dtype=np.float64)
    A = 0.5 * (A + A.T)
    return A + float(jitter) * np.eye(A.shape[0], dtype=np.float64)

# ----------------------------
# Helpers: feature -> physical (for plots)
# ----------------------------
def feat_x_to_phys(x_feat, x_scale=2.4):
    x_feat = np.asarray(x_feat, dtype=np.float64)
    x_feat = np.clip(x_feat, -0.999, 0.999)
    return x_scale * np.arctanh(x_feat)

def feat_xdot_to_phys(xdot_feat, v_scale=3.0):
    xdot_feat = np.asarray(xdot_feat, dtype=np.float64)
    xdot_feat = np.clip(xdot_feat, -0.999, 0.999)
    return v_scale * np.arctanh(xdot_feat)

# ----------------------------
# Precompute a fixed plot grid (same for all updates)
# ----------------------------
def build_fixed_plot_grid():
    xg = np.linspace(PLOT_X_MIN, PLOT_X_MAX, PLOT_N_GRID)
    vg = np.linspace(PLOT_V_MIN, PLOT_V_MAX, PLOT_N_GRID)
    X, V = np.meshgrid(xg, vg)

    Xfeat_grid = np.vstack([
        state_to_features(x, xdot, PLOT_TH_FIXED, PLOT_THDOT_FIXED, PLOT_U_FIXED)
        for x, xdot in zip(X.ravel(), V.ravel())
    ]).astype(np.float64)
    return X, V, Xfeat_grid

PLOT_XMESH, PLOT_VMESH, PLOT_XFEAT_GRID = build_fixed_plot_grid()

# ----------------------------
# Global predict helpers (mean/std)
# ----------------------------
def gp_predict_mu_std_fast(model, X):
    X = np.asarray(X, dtype=np.float64)
    if hasattr(model, "predict_f_cached"):
        mu_tf, var_tf = model.predict_f_cached(X, full_cov=False)
    else:
        mu_tf, var_tf = model.predict_f(X, full_cov=False)
    mu = mu_tf.numpy().reshape(-1)
    var = var_tf.numpy().reshape(-1)
    std = np.sqrt(np.maximum(var, 1e-12))
    return mu, std

def pred_mean_cached(model, Xfeat):
    Xfeat = np.asarray(Xfeat, dtype=np.float64)
    if hasattr(model, "predict_f_cached"):
        mu_tf, _ = model.predict_f_cached(Xfeat, full_cov=False)
    else:
        mu_tf, _ = model.predict_f(Xfeat, full_cov=False)
    return mu_tf.numpy().reshape(-1)

def make_global_predictors_bundle():
    return (
        lambda X: pred_mean_cached(m_dx, X),
        lambda X: pred_mean_cached(m_dxdot, X),
        lambda X: pred_mean_cached(m_dth, X),
        lambda X: pred_mean_cached(m_dthdot, X),
    )

# ----------------------------
# Unified std scale utilities
# ----------------------------
def update_unified_std_scale(std_list, q=0.99, mode="fixed"):
    global STD_CMAX_FIXED
    all_std = np.concatenate([np.asarray(s, dtype=np.float64).reshape(-1) for s in std_list if s is not None], axis=0)
    if all_std.size == 0:
        if STD_CMAX_FIXED is None:
            STD_CMAX_FIXED = 1.0
        return (STD_CMIN_FIXED, STD_CMAX_FIXED)

    cand = float(np.quantile(all_std, q))
    cand = max(cand, 1e-8)

    if STD_CMAX_FIXED is None:
        STD_CMAX_FIXED = cand
    elif mode == "grow_only":
        STD_CMAX_FIXED = max(STD_CMAX_FIXED, cand)

    return (STD_CMIN_FIXED, STD_CMAX_FIXED)

# ============================================================
# CLEAN multi-head Z_GLOBAL refit
# ============================================================
def refit_Z_global_multihead(Z_global, Xnew, M_global, lam=1e-6, mode="mean", normalize_traces=False):
    Z_global = np.asarray(Z_global, dtype=np.float64)
    Xnew = np.asarray(Xnew, dtype=np.float64)

    Z_cand = np.vstack([Z_global, Xnew]).astype(np.float64, copy=False)
    Zc_tf = tf.convert_to_tensor(Z_cand, dtype=tf.float64)

    K_dx     = m_dx.kernel.K(Zc_tf).numpy().astype(np.float64, copy=False)
    K_dxdot  = m_dxdot.kernel.K(Zc_tf).numpy().astype(np.float64, copy=False)
    K_dth    = m_dth.kernel.K(Zc_tf).numpy().astype(np.float64, copy=False)
    K_dthdot = m_dthdot.kernel.K(Zc_tf).numpy().astype(np.float64, copy=False)

    if normalize_traces:
        def _norm(K):
            tr = float(np.trace(K))
            return K / max(tr, 1e-12)
        K_dx, K_dxdot, K_dth, K_dthdot = map(_norm, [K_dx, K_dxdot, K_dth, K_dthdot])

    if mode == "sum":
        K_agg = (K_dx + K_dxdot + K_dth + K_dthdot)
    else:
        K_agg = (K_dx + K_dxdot + K_dth + K_dthdot) / 4.0

    idxZ = greedy_dopt_anchors_from_K(K_agg, m_anchors=int(M_global), lam=float(lam))
    Z_new = np.asarray(Z_cand[np.asarray(idxZ, dtype=np.int64)], dtype=np.float64)

    if Z_new.shape[0] != int(M_global):
        Z_new = Z_new[:int(M_global)].copy()
    return Z_new

# ----------------------------
# Online OSGPR update (Z fixed each update)
# ----------------------------
def osgpr_stream_update(model_old, X_new, Y_new, Z_new,
                        iters=150, lr=0.02, noise=1e-4,
                        freeze_kernel=False, clip_norm=10.0):
    X_new = np.asarray(X_new, dtype=np.float64)
    Y_new = np.asarray(Y_new, dtype=np.float64).reshape(-1, 1)

    mu_old, Su_old, Kaa_old, Z_old = extract_summary_from_model(model_old)
    Z_new = np.asarray(Z_new, dtype=np.float64)

    k_new = clone_kernel(model_old.kernel)

    m = OSGPR_VFE(
        data=(X_new, Y_new),
        kernel=k_new,
        mu_old=mu_old, Su_old=Su_old, Kaa_old=Kaa_old, Z_old=Z_old,
        Z=Z_new
    )
    m.likelihood.variance.assign(float(noise))

    if freeze_kernel:
        try:
            m.kernel.variance.trainable = False
            m.kernel.lengthscales.trainable = False
        except Exception:
            pass

    t_train, last_loss = train_osgpr(m, iters=iters, lr=lr, clip_norm=clip_norm)
    m.build_predict_cache()

    info = dict(
        train_seconds=float(t_train),
        last_neg_objective=float(last_loss),
        M_new=int(m.inducing_variable.num_inducing),
    )
    return m, info

# ============================================================
# Anchors via aggregated kernel on Z_GLOBAL (multi-head)
# ============================================================
def compute_anchor_idx_dopt_from_Zglobal_multihead(Z_global, m_anchors=18, lam=1e-6, normalize_traces=True):
    Z = np.asarray(Z_global, dtype=np.float64)
    Ztf = tf.convert_to_tensor(Z, dtype=tf.float64)

    K_dx     = m_dx.kernel.K(Ztf).numpy().astype(np.float64, copy=False)
    K_dxdot  = m_dxdot.kernel.K(Ztf).numpy().astype(np.float64, copy=False)
    K_dth    = m_dth.kernel.K(Ztf).numpy().astype(np.float64, copy=False)
    K_dthdot = m_dthdot.kernel.K(Ztf).numpy().astype(np.float64, copy=False)

    if normalize_traces:
        def _norm(K):
            tr = float(np.trace(K))
            return K / max(tr, 1e-12)
        K_dx, K_dxdot, K_dth, K_dthdot = map(_norm, [K_dx, K_dxdot, K_dth, K_dthdot])

    K_agg = (K_dx + K_dxdot + K_dth + K_dthdot) / 4.0
    return greedy_dopt_anchors_from_K(K_agg, m_anchors=int(m_anchors), lam=float(lam))

# ----------------------------
# Local subset predictor (NumPy SE-ARD) from global posterior at Z_sub
# ----------------------------
def se_ard_kernel_Kzx(Z, X, lengthscales, variance):
    Z = np.asarray(Z, dtype=np.float64)
    X = np.asarray(X, dtype=np.float64)
    ls = np.asarray(lengthscales, dtype=np.float64).reshape(1, -1)
    var = float(variance)

    Zs = Z / ls
    Xs = X / ls
    z2 = np.sum(Zs * Zs, axis=1, keepdims=True)
    x2 = np.sum(Xs * Xs, axis=1, keepdims=True).T
    zx = Zs @ Xs.T
    r2 = np.maximum(z2 + x2 - 2.0 * zx, 0.0)
    return var * np.exp(-0.5 * r2)

def build_subset_predictor_from_global(model_global, Z_global, idx_sub, jitter=1e-6):
    idx_sub = np.asarray(idx_sub, dtype=np.int64)

    Zg = np.asarray(Z_global, dtype=np.float64)
    Z = Zg[idx_sub].copy()

    mu_tf, Sig_tf = model_global.predict_f(Z, full_cov=True)
    m = mu_tf.numpy().reshape(-1, 1)
    S = Sig_tf.numpy()
    if S.ndim == 3:
        S = S[0]
    S = sym_jitter(S, 1e-6)

    ls  = model_global.kernel.lengthscales.numpy()
    var = model_global.kernel.variance.numpy()

    Kzz = se_ard_kernel_Kzx(Z, Z, ls, var)
    Kzz = sym_jitter(Kzz, jitter=jitter)
    L = np.linalg.cholesky(Kzz)

    y = np.linalg.solve(L, m)
    alpha = np.linalg.solve(L.T, y)

    def predict(Xfeat):
        Xfeat = np.asarray(Xfeat, dtype=np.float64)
        Kzx = se_ard_kernel_Kzx(Z, Xfeat, ls, var)

        W = np.linalg.solve(L, Kzx)
        W = np.linalg.solve(L.T, W)

        mu = (Kzx.T @ alpha).reshape(-1)

        kxx = float(var) * np.ones((Xfeat.shape[0],), dtype=np.float64)
        Qdiag = np.sum(Kzx * W, axis=0)
        SW = S @ W
        Sdiag = np.sum(W * SW, axis=0)

        v = kxx - Qdiag + Sdiag
        v = np.maximum(v, 1e-12)
        return mu, v

    return predict, Z

def make_subset_predictors_bundle(idx_sub):
    p_dx, _     = build_subset_predictor_from_global(m_dx,     Z_GLOBAL, idx_sub)
    p_dxdot, _  = build_subset_predictor_from_global(m_dxdot,  Z_GLOBAL, idx_sub)
    p_dth, _    = build_subset_predictor_from_global(m_dth,    Z_GLOBAL, idx_sub)
    p_dthdot, _ = build_subset_predictor_from_global(m_dthdot, Z_GLOBAL, idx_sub)
    return (
        lambda X: p_dx(X)[0],
        lambda X: p_dxdot(X)[0],
        lambda X: p_dth(X)[0],
        lambda X: p_dthdot(X)[0],
    )

def build_subset_uncertainty_fn(model_global, idx_sub):
    pred, _ = build_subset_predictor_from_global(model_global, Z_GLOBAL, idx_sub)
    def unc(Xfeat):
        _, v = pred(Xfeat)
        return v
    return unc

# ============================================================
# Chi-square Mahalanobis tube selection helpers
# ============================================================
def normalize_nonnegative_weights(w, eps=1e-12):
    w = np.asarray(w, dtype=np.float64).reshape(-1)
    w = np.maximum(w, 0.0)
    s = float(np.sum(w))
    if s < eps:
        return np.ones_like(w) / max(len(w), 1)
    return w / s

def compute_weighted_moments(rollout_inputs, rollout_weights):
    X = np.asarray(rollout_inputs, dtype=np.float64)  # (K,H,D)
    K, H, D = X.shape
    w = normalize_nonnegative_weights(rollout_weights).reshape(K, 1, 1)

    tube_mean = np.sum(w * X, axis=0)  # (H,D)
    Xc = X - tube_mean[None, :, :]
    tube_cov = np.einsum("khd,khe->hde", (w * Xc), Xc)  # (H,D,D)
    return tube_mean, tube_cov

def min_mahalanobis_and_argmin(points_Z, tube_mean, tube_cov, cov_eps=1e-6):
    Z = np.asarray(points_Z, dtype=np.float64)
    mu = np.asarray(tube_mean, dtype=np.float64)
    Sig = np.asarray(tube_cov, dtype=np.float64)

    M, D = Z.shape
    H = mu.shape[0]
    I = np.eye(D, dtype=np.float64)

    dmin = np.full((M,), np.inf, dtype=np.float64)
    tmin = np.zeros((M,), dtype=np.int64)

    for t in range(H):
        St = Sig[t] + float(cov_eps) * I
        L = np.linalg.cholesky(St)

        diff = (Z - mu[t:t+1, :]).T  # (D,M)
        y = np.linalg.solve(L, diff)
        y2 = np.linalg.solve(L.T, y)
        quad = np.sum(diff * y2, axis=0)  # (M,)

        mask = quad < dmin
        dmin[mask] = quad[mask]
        tmin[mask] = t

    return dmin, tmin

def split_even_quota(total, num_bins):
    num_bins = int(max(1, num_bins))
    base = total // num_bins
    rem = total - base * num_bins
    q = np.full((num_bins,), base, dtype=np.int64)
    if rem > 0:
        q[:rem] += 1
    return q

def chi2_radius_sq(alpha, D):
    alpha = float(alpha)
    D = int(D)
    alpha = min(max(alpha, 1e-6), 1.0 - 1e-12)
    return float(chi2.ppf(alpha, df=D))

def greedy_dopt_select_from_kernel(K, k, jitter=1e-6, log_eps=1e-300):
    K = np.asarray(K, dtype=np.float64)
    n = K.shape[0]
    k = int(min(k, n))
    if k <= 0:
        return np.zeros((0,), dtype=np.int64)

    diag = np.diag(K).copy() + float(jitter)
    chosen = []
    Lchol = None

    for t in range(k):
        if t == 0:
            safe = np.maximum(diag, log_eps)
            i = int(np.argmax(np.log(safe)))
            chosen.append(i)
            Lchol = np.sqrt(max(diag[i], log_eps)).reshape(1, 1)
            continue

        S = np.array(chosen, dtype=np.int64)

        K_S = K[:, S]
        v = np.linalg.solve(Lchol, K_S.T)
        sq = np.sum(v * v, axis=0)
        schur = diag - sq

        schur[S] = -np.inf

        schur_pos = np.maximum(schur, log_eps)
        scores = np.log(schur_pos)
        scores[S] = -np.inf

        i = int(np.argmax(scores))
        chosen.append(i)

        k_iS = K[i, S].reshape(1, -1)
        w = np.linalg.solve(Lchol, k_iS.T)
        alpha2 = diag[i] - float(np.sum(w * w))
        alpha2 = max(alpha2, log_eps)
        alpha = np.sqrt(alpha2)

        Lnew = np.zeros((t + 1, t + 1), dtype=np.float64)
        Lnew[:t, :t] = Lchol
        Lnew[t, :t] = w.reshape(-1)
        Lnew[t, t] = alpha
        Lchol = Lnew

    return np.array(chosen, dtype=np.int64)

def select_tube_subset(
    Z_global,
    kernel_for_dopt,
    rollout_inputs,
    rollout_weights,
    total_subset_size=64,
    anchor_idx=None,
    time_bins=16,
    cov_eps=1e-6,
    dopt_jitter=1e-6,
    fallback_candidates=400,
    alpha=0.99,
):
    Zg = np.asarray(Z_global, dtype=np.float64)
    M = Zg.shape[0]

    if anchor_idx is None:
        anchor_idx = np.zeros((0,), dtype=np.int64)
    anchor_idx = np.unique(np.asarray(anchor_idx, dtype=np.int64))
    anchor_idx = anchor_idx[(anchor_idx >= 0) & (anchor_idx < M)]

    tube_mean, tube_cov = compute_weighted_moments(rollout_inputs, rollout_weights)
    H, D = tube_mean.shape

    chi2_thr = chi2_radius_sq(alpha, D)

    dmin, tmin = min_mahalanobis_and_argmin(Zg, tube_mean, tube_cov, cov_eps=cov_eps)
    tube_candidates_idx = np.where(dmin <= chi2_thr)[0].astype(np.int64)

    remaining_budget = int(total_subset_size - anchor_idx.size)
    remaining_budget = max(0, remaining_budget)

    if tube_candidates_idx.size < remaining_budget:
        L = int(min(max(fallback_candidates, remaining_budget), M))
        tube_candidates_idx = np.argsort(dmin)[:L].astype(np.int64)

    tube_pool = tube_candidates_idx[~np.isin(tube_candidates_idx, anchor_idx)]
    tmin_pool = tmin[tube_pool]

    selected = set(int(i) for i in anchor_idx)

    if remaining_budget > 0 and tube_pool.size > 0:
        B = int(max(1, min(time_bins, H)))
        edges = np.linspace(0, H, B + 1, dtype=np.int64)
        quotas = split_even_quota(remaining_budget, B)

        for b in range(B):
            if len(selected) >= total_subset_size:
                break

            a, c = int(edges[b]), int(edges[b + 1])
            if c <= a:
                continue

            in_bin = (tmin_pool >= a) & (tmin_pool < c)
            bin_pool = tube_pool[in_bin]
            if bin_pool.size == 0:
                continue

            k_b = int(min(quotas[b], bin_pool.size, total_subset_size - len(selected)))
            if k_b <= 0:
                continue

            Zb = Zg[bin_pool]
            Kb = kernel_for_dopt.K(tf.convert_to_tensor(Zb, dtype=tf.float64)).numpy().astype(np.float64)
            pick = greedy_dopt_select_from_kernel(Kb, k=k_b, jitter=dopt_jitter)

            for gidx in bin_pool[pick]:
                selected.add(int(gidx))

    if len(selected) < total_subset_size:
        need = int(total_subset_size - len(selected))
        leftover = tube_pool[~np.isin(tube_pool, np.array(list(selected), dtype=np.int64))]
        if leftover.size > 0 and need > 0:
            Zl = Zg[leftover]
            Kl = kernel_for_dopt.K(tf.convert_to_tensor(Zl, dtype=tf.float64)).numpy().astype(np.float64)
            pick = greedy_dopt_select_from_kernel(Kl, k=min(need, leftover.size), jitter=dopt_jitter)
            for gidx in leftover[pick]:
                selected.add(int(gidx))

    if len(selected) < total_subset_size:
        need = int(total_subset_size - len(selected))
        remaining = np.setdiff1d(np.arange(M, dtype=np.int64), np.array(list(selected), dtype=np.int64), assume_unique=False)
        if remaining.size > 0:
            order = remaining[np.argsort(dmin[remaining])]
            for gidx in order[:need]:
                selected.add(int(gidx))

    subset_idx = np.array(sorted(selected), dtype=np.int64)[:total_subset_size]
    return subset_idx, tube_mean, tube_cov, tube_candidates_idx, chi2_thr

# ----------------------------
# MPPI (NumPy) helpers (unchanged)
# ----------------------------
def exploration_weight(t, explore_steps=EXPLORE_STEPS, w_max=UNC_W_MAX, w_min=UNC_W_MIN):
    if explore_steps <= 0:
        return float(w_min)
    a = np.clip(1.0 - float(t) / float(explore_steps), 0.0, 1.0)
    return float(w_min + (w_max - w_min) * a)

def stage_cost_cartpole(S, U, x_init, unc_bonus=None, unc_w=0.0):
    th = S[:, 2]
    x  = S[:, 0]
    c = (
        UPRIGHT_W * (1.0 - np.cos(th))
        + CENTER_W * ((x - x_init) ** 2)
        + U_W * (U ** 2)
    )
    if (unc_bonus is not None) and (unc_w > 0.0):
        c = c - float(unc_w) * np.asarray(unc_bonus, dtype=np.float64)
    return c

def terminal_cost_hold_like(S, x_init):
    th = S[:, 2]
    x  = S[:, 0]
    cT = 0.0 * th
    good = (np.cos(th) >= UPRIGHT_COS) & (np.abs(x - x_init) <= X_BAND)
    cT[good] -= 5.0
    return cT

def gp_dynamics_step_batch(S, U, pred_dx, pred_dxdot, pred_dth, pred_dthdot):
    Xfeat = batch_state_to_features(S, U)
    dx     = pred_dx(Xfeat)
    dxdot  = pred_dxdot(Xfeat)
    dth    = pred_dth(Xfeat)
    dthdot = pred_dthdot(Xfeat)

    S2 = np.empty_like(S)
    S2[:, 0] = S[:, 0] + dx
    S2[:, 1] = S[:, 1] + dxdot
    S2[:, 2] = wrap_pi(S[:, 2] + dth)
    S2[:, 3] = S[:, 3] + dthdot
    return S2, Xfeat

# ----------------------------
# GPU MPPI (TF) — predictor uses EXPLICIT Z_GLOBAL (FIXED)
# ----------------------------
@tf.function
def wrap_pi_tf(theta):
    two_pi = tf.constant(2.0 * np.pi, dtype=DTYPE_TF)
    pi = tf.constant(np.pi, dtype=DTYPE_TF)
    return tf.math.floormod(theta + pi, two_pi) - pi

@tf.function
def batch_state_to_features_tf(S, U):
    x = S[:, 0]
    xdot = S[:, 1]
    th = S[:, 2]
    thdot = S[:, 3]
    f0 = tf.tanh(x / tf.constant(2.4, dtype=DTYPE_TF))
    f1 = tf.tanh(xdot / tf.constant(3.0, dtype=DTYPE_TF))
    f2 = tf.sin(th)
    f3 = tf.cos(th)
    f4 = tf.tanh(thdot / tf.constant(8.0, dtype=DTYPE_TF))
    f5 = U
    return tf.stack([f0, f1, f2, f3, f4, f5], axis=1)

@tf.function
def se_ard_kernel_Kzx_tf(Z, X, lengthscales, variance):
    ls = tf.reshape(lengthscales, (1, -1))
    var = tf.cast(variance, DTYPE_TF)
    Zs = Z / ls
    Xs = X / ls
    z2 = tf.reduce_sum(Zs * Zs, axis=1, keepdims=True)
    x2 = tf.reduce_sum(Xs * Xs, axis=1, keepdims=True)
    zx = tf.matmul(Zs, Xs, transpose_b=True)
    r2 = tf.maximum(z2 + tf.transpose(x2) - 2.0 * zx, 0.0)
    return var * tf.exp(-0.5 * r2)

def build_subset_predictor_from_global_tf(model_global, Z_global, idx_sub):
    idx_sub = tf.convert_to_tensor(np.asarray(idx_sub, dtype=np.int32))
    Zg = tf.convert_to_tensor(np.asarray(Z_global, dtype=np.float64), dtype=DTYPE_TF)
    Z  = tf.gather(Zg, idx_sub, axis=0)

    muZ, SigZ = model_global.predict_f(Z, full_cov=True)
    m = muZ
    S = SigZ
    if len(S.shape) == 3:
        S = S[0]

    ls  = tf.cast(model_global.kernel.lengthscales, DTYPE_TF)
    var = tf.cast(model_global.kernel.variance, DTYPE_TF)

    Kzz = se_ard_kernel_Kzx_tf(Z, Z, ls, var)
    jitter = tf.cast(1e-6, DTYPE_TF)
    Kzz = 0.5 * (Kzz + tf.transpose(Kzz)) + jitter * tf.eye(tf.shape(Kzz)[0], dtype=DTYPE_TF)
    L = tf.linalg.cholesky(Kzz)

    y = tf.linalg.triangular_solve(L, m, lower=True)
    alpha = tf.linalg.triangular_solve(tf.transpose(L), y, lower=False)

    @tf.function
    def predict(Xfeat):
        Xfeat = tf.cast(Xfeat, DTYPE_TF)
        Kzx = se_ard_kernel_Kzx_tf(Z, Xfeat, ls, var)

        w1 = tf.linalg.triangular_solve(L, Kzx, lower=True)
        W  = tf.linalg.triangular_solve(tf.transpose(L), w1, lower=False)

        mu = tf.reshape(tf.matmul(Kzx, alpha, transpose_a=True), (-1,))

        kxx = tf.fill((tf.shape(Xfeat)[0],), tf.cast(var, DTYPE_TF))
        Qdiag = tf.reduce_sum(Kzx * W, axis=0)
        SW = tf.matmul(S, W)
        Sdiag = tf.reduce_sum(W * SW, axis=0)

        v = tf.maximum(kxx - Qdiag + Sdiag, tf.cast(1e-12, DTYPE_TF))
        return mu, v

    return predict, Z

@tf.function
def exploration_weight_tf(t):
    explore_steps_f = tf.cast(EXPLORE_STEPS, DTYPE_TF)
    t_f = tf.cast(t, DTYPE_TF)
    a = tf.clip_by_value(1.0 - t_f / tf.maximum(explore_steps_f, 1.0), 0.0, 1.0)
    return tf.cast(UNC_W_MIN, DTYPE_TF) + (tf.cast(UNC_W_MAX, DTYPE_TF) - tf.cast(UNC_W_MIN, DTYPE_TF)) * a

@tf.function
def stage_cost_cartpole_tf(S, U, x_init, unc_bonus=None, unc_w=0.0):
    th = S[:, 2]
    x  = S[:, 0]
    c = (
        tf.cast(UPRIGHT_W, DTYPE_TF) * (1.0 - tf.cos(th))
        + tf.cast(CENTER_W, DTYPE_TF) * tf.square(x - x_init)
        + tf.cast(U_W, DTYPE_TF) * tf.square(U)
    )
    if (unc_bonus is not None) and (unc_w > 0.0):
        c = c - tf.cast(unc_w, DTYPE_TF) * tf.cast(unc_bonus, DTYPE_TF)
    return c

@tf.function
def terminal_cost_hold_like_tf(S, x_init):
    th = S[:, 2]
    x  = S[:, 0]
    cT = tf.zeros_like(th)
    good = tf.logical_and(tf.cos(th) >= tf.cast(UPRIGHT_COS, DTYPE_TF),
                          tf.abs(x - x_init) <= tf.cast(X_BAND, DTYPE_TF))
    cT = tf.where(good, cT - tf.cast(5.0, DTYPE_TF), cT)
    return cT

def make_local_tf_predictors_bundle(idx_sub):
    pred_dx_tf, _     = build_subset_predictor_from_global_tf(m_dx,     Z_GLOBAL, idx_sub)
    pred_dxdot_tf, Zs = build_subset_predictor_from_global_tf(m_dxdot,  Z_GLOBAL, idx_sub)
    pred_dth_tf, _    = build_subset_predictor_from_global_tf(m_dth,    Z_GLOBAL, idx_sub)
    pred_dthdot_tf, _ = build_subset_predictor_from_global_tf(m_dthdot, Z_GLOBAL, idx_sub)

    @tf.function
    def dx_mu(X):     return pred_dx_tf(X)[0]
    @tf.function
    def dxdot_mu(X):  return pred_dxdot_tf(X)[0]
    @tf.function
    def dth_mu(X):    return pred_dth_tf(X)[0]
    @tf.function
    def thdot_mu(X):  return pred_dthdot_tf(X)[0]

    @tf.function
    def unc_var_dxdot(X):
        _, v = pred_dxdot_tf(X)
        return v

    return (dx_mu, dxdot_mu, dth_mu, thdot_mu), unc_var_dxdot, Zs

@tf.function
def gp_dynamics_step_batch_local_tf(S, U, pred_bundle_tf):
    pred_dx_tf, pred_dxdot_tf, pred_dth_tf, pred_dthdot_tf = pred_bundle_tf
    Xfeat = batch_state_to_features_tf(S, U)
    dx     = pred_dx_tf(Xfeat)
    dxdot  = pred_dxdot_tf(Xfeat)
    dth    = pred_dth_tf(Xfeat)
    dthdot = pred_dthdot_tf(Xfeat)

    S2 = tf.stack([
        S[:, 0] + dx,
        S[:, 1] + dxdot,
        wrap_pi_tf(S[:, 2] + dth),
        S[:, 3] + dthdot
    ], axis=1)
    return S2, Xfeat

@tf.function
def rollout_tube_features_local_tf(state0, u_seq, pred_bundle_tf):
    H = tf.shape(u_seq)[0]
    s = tf.identity(state0)
    tube = tf.TensorArray(dtype=DTYPE_TF, size=H)

    t = tf.constant(0)
    def cond(t, s, tube): return t < H
    def body(t, s, tube):
        u = u_seq[t]
        xfeat = batch_state_to_features_tf(tf.expand_dims(s, axis=0), tf.expand_dims(u, axis=0))[0]
        tube = tube.write(t, xfeat)
        s2, _ = gp_dynamics_step_batch_local_tf(tf.expand_dims(s, axis=0), tf.expand_dims(u, axis=0), pred_bundle_tf)
        s = s2[0]
        return t+1, s, tube

    _, _, tube = tf.while_loop(cond, body, [t, s, tube], parallel_iterations=1)
    return tube.stack()

@tf.function
def mppi_plan_gpu_local_tf(state0, x_init, u_mean0, t_global,
                          pred_bundle_tf, unc_fn_tf,
                          horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
                          base_seed=0):
    H = tf.cast(horizon, tf.int32)
    Kt = tf.cast(K, tf.int32)

    seed = tf.stack([tf.cast(base_seed, tf.int32), tf.cast(t_global, tf.int32)], axis=0)
    eps = tf.random.stateless_normal((Kt, H), seed=seed, mean=0.0, stddev=tf.cast(sigma, DTYPE_TF), dtype=DTYPE_TF)
    U = tf.clip_by_value(u_mean0[None, :] + eps, tf.cast(U_MIN, DTYPE_TF), tf.cast(U_MAX, DTYPE_TF))

    S = tf.tile(state0[None, :], [Kt, 1])
    total_cost = tf.zeros((Kt,), dtype=DTYPE_TF)

    Xta = tf.TensorArray(dtype=DTYPE_TF, size=H)
    unc_w = exploration_weight_tf(t_global)

    t = tf.constant(0, dtype=tf.int32)
    def cond(t, S, total_cost, Xta): return t < H
    def body(t, S, total_cost, Xta):
        Ut = U[:, t]
        S2, Xfeat = gp_dynamics_step_batch_local_tf(S, Ut, pred_bundle_tf)
        Xta = Xta.write(t, Xfeat)
        unc_bonus = unc_fn_tf(Xfeat)
        total_cost = total_cost + stage_cost_cartpole_tf(S, Ut, x_init=tf.cast(x_init, DTYPE_TF),
                                                         unc_bonus=unc_bonus, unc_w=unc_w)
        return t+1, S2, total_cost, Xta

    _, S, total_cost, Xta = tf.while_loop(cond, body, [t, S, total_cost, Xta], parallel_iterations=1)
    total_cost = total_cost + terminal_cost_hold_like_tf(S, x_init=tf.cast(x_init, DTYPE_TF))

    cmin = tf.reduce_min(total_cost)
    w = tf.exp(-(total_cost - cmin) / tf.cast(lam, DTYPE_TF))
    wsum = tf.reduce_sum(w) + tf.cast(1e-12, DTYPE_TF)

    u_mean = u_mean0 + tf.reduce_sum(w[:, None] * eps, axis=0) / wsum
    u_mean = tf.clip_by_value(u_mean, tf.cast(U_MIN, DTYPE_TF), tf.cast(U_MAX, DTYPE_TF))

    tubeX = rollout_tube_features_local_tf(state0, u_mean, pred_bundle_tf)
    XHKD = Xta.stack()
    Xroll = tf.transpose(XHKD, perm=[1, 0, 2])

    return u_mean[0], u_mean, tubeX, unc_w, Xroll, w

def mppi_plan_gpu_local(state, x_init, u_init, t_global, pred_bundle_tf, unc_fn_tf, base_seed=0):
    state0 = tf.convert_to_tensor(np.asarray(state, dtype=np.float64).reshape(4,), dtype=DTYPE_TF)
    x0     = tf.convert_to_tensor(float(x_init), dtype=DTYPE_TF)
    u0     = tf.convert_to_tensor(np.asarray(u_init, dtype=np.float64).reshape(-1,), dtype=DTYPE_TF)

    dev = "/GPU:0" if len(tf.config.list_logical_devices("GPU")) > 0 else "/CPU:0"
    if REQUIRE_GPU: dev = "/GPU:0"

    with tf.device(dev):
        u_first, u_mean, tubeX, unc_w, Xroll, w = mppi_plan_gpu_local_tf(
            state0, x0, u0, tf.convert_to_tensor(int(t_global), dtype=tf.int32),
            pred_bundle_tf=pred_bundle_tf,
            unc_fn_tf=unc_fn_tf,
            horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
            base_seed=int(base_seed)
        )

    return float(u_first.numpy()), u_mean.numpy(), tubeX.numpy(), float(unc_w.numpy()), Xroll.numpy(), w.numpy()

# ----------------------------
# Visualization helpers (kept)
# ----------------------------
def plot_surface_global_or_local_dxdot(model_like, is_local=False, Z_overlay=None,
                                      title="Surface", std_clim=None,
                                      Xmesh=PLOT_XMESH, Vmesh=PLOT_VMESH, Xfeat_grid=PLOT_XFEAT_GRID,
                                      show_colorbar=True):
    if not is_local:
        mu, std = gp_predict_mu_std_fast(model_like, Xfeat_grid)
        Mean = mu.reshape(Xmesh.shape)
        Std  = std.reshape(Xmesh.shape)
        Z = model_like.inducing_variable.Z.numpy()
    else:
        pred_local = model_like
        mu, var = pred_local(Xfeat_grid)
        Mean = mu.reshape(Xmesh.shape)
        Std  = np.sqrt(np.maximum(var, 1e-12)).reshape(Xmesh.shape)
        Z = Z_overlay

    cmin = cmax = None
    if std_clim is not None:
        cmin, cmax = float(std_clim[0]), float(std_clim[1])

    surface = go.Surface(
        x=Xmesh, y=Vmesh, z=Mean,
        surfacecolor=Std,
        colorscale="Viridis",
        cmin=cmin, cmax=cmax,
        colorbar=(dict(title="Std") if show_colorbar else None),
        showscale=bool(show_colorbar),
        opacity=0.95,
        name="surface"
    )
    fig = go.Figure(data=[surface])
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title="x", range=[PLOT_X_MIN, PLOT_X_MAX]),
            yaxis=dict(title="xdot", range=[PLOT_V_MIN, PLOT_V_MAX]),
            zaxis=dict(title="Δxdot"),
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=620
    )
    fig.show()

def success_hold_update(state, x_init, hold_count):
    x, xdot, th, thdot = state
    upright = (np.cos(th) >= UPRIGHT_COS)
    centered = (abs(x - x_init) <= X_BAND)
    hold_count = (hold_count + 1) if (upright and centered) else 0
    success = (hold_count >= HOLD_STEPS)
    return hold_count, success, upright, centered

# ============================================================
# NEW: Create ONE animation per RUN (stitches ep0+ep1+ep2 frames)
# ============================================================
def display_run_animation_from_frames(frames, fps=FPS, resize=RESIZE):
    if frames is None or len(frames) == 0:
        print("⚠️ No frames captured for this run.")
        return None

    fig = plt.figure(figsize=(resize[0]/100, resize[1]/100), dpi=100)
    plt.axis("off")
    im = plt.imshow(frames[0])

    def animate_fn(i):
        im.set_data(frames[i])
        return [im]

    ani = animation.FuncAnimation(
        fig, animate_fn,
        frames=len(frames),
        interval=1000 / float(fps),
        blit=True
    )
    plt.close(fig)
    html = HTML(ani.to_jshtml())
    display(html)
    return html

# ============================================================
# Single EPISODE runner (returns per-timestep timing arrays)
#   NOTE: model & Z_GLOBAL are global; this function MUTATES them (training)
#   CHANGE: does NOT display animation; returns frames if asked
# ============================================================
def run_one_episode_mppi_retrain_rgb_with_eval(
    max_steps=600, seed=0, start_down=True, verbose=True,
    use_gpu_mppi=True,
    warmup_mppi=True,
    record_rgb=True,
    t_offset_in_run=0,
    return_frames=False,  # NEW
):
    global m_dx, m_dxdot, m_dth, m_dthdot
    global Z_GLOBAL
    global STD_CMAX_FIXED

    env = make_env(render_mode=None, seed=seed, max_episode_steps=max_steps, start_down=start_down)
    obs, info = env.reset(seed=seed)
    s = np.array(obs_to_state(obs), dtype=np.float64)
    x_init = float(s[0])

    # Ensure Z_GLOBAL fixed size
    Z_GLOBAL = np.asarray(Z_GLOBAL, dtype=np.float64)
    if Z_GLOBAL.shape[0] != int(M_GLOBAL):
        if Z_GLOBAL.shape[0] > int(M_GLOBAL):
            Z_GLOBAL = Z_GLOBAL[:int(M_GLOBAL)].copy()
        else:
            raise ValueError(f"Z_GLOBAL has {Z_GLOBAL.shape[0]} points but M_GLOBAL={M_GLOBAL}.")

    # anchors
    ANCHOR_IDX = compute_anchor_idx_dopt_from_Zglobal_multihead(
        Z_GLOBAL, m_anchors=ANCHOR_M, lam=ANCHOR_LAM, normalize_traces=True
    )

    # init local
    u_mean = np.zeros((HORIZON,), dtype=np.float64)
    idx_sub = np.arange(min(M_SUB, M_GLOBAL), dtype=np.int64)

    pred_local_np = make_subset_predictors_bundle(idx_sub)
    unc_local_np  = build_subset_uncertainty_fn(m_dxdot, idx_sub)
    pred_local_tf, unc_local_tf, _ = make_local_tf_predictors_bundle(idx_sub)

    last_idx_sub = np.array(idx_sub, dtype=np.int64)
    last_local_rebuild_t = 0

    # timing arrays (length max_steps)
    pred_time_step = np.zeros((max_steps,), dtype=np.float64)   # MPPI planning time per step
    train_time_step = np.zeros((max_steps,), dtype=np.float64)  # training time per step (only nonzero on update steps)
    wall_excl_vis_cum = np.zeros((max_steps,), dtype=np.float64)

    # runtime excluding vis
    t_wall_start = time.perf_counter()
    vis_time_s = 0.0

    # Warmup TF MPPI (compile), excluded from pred_time_step but still counts into wall time (as requested)
    if use_gpu_mppi and warmup_mppi:
        _ = mppi_plan_gpu_local(
            state=s, x_init=x_init, u_init=u_mean, t_global=int(t_offset_in_run),
            pred_bundle_tf=pred_local_tf, unc_fn_tf=unc_local_tf,
            base_seed=seed
        )

    frames = []
    total_reward = 0.0
    hold_count = 0
    updates = 0

    Xbuf, ydx_buf, ydxdot_buf, ydth_buf, ydthdot_buf = [], [], [], [], []

    last_rollout_inputs = None
    last_rollout_weights = None

    for t in range(max_steps):
        t_global = int(t_offset_in_run + t)

        # ---------- MPPI plan (prediction time) ----------
        t0 = time.perf_counter()
        if use_gpu_mppi:
            u0, u_mean, tubeX, unc_w, Xroll, wroll = mppi_plan_gpu_local(
                state=s, x_init=x_init, u_init=u_mean, t_global=t_global,
                pred_bundle_tf=pred_local_tf,
                unc_fn_tf=unc_local_tf,
                base_seed=seed
            )
        else:
            raise RuntimeError("use_gpu_mppi=False path not wired in this multi-run cell (set use_gpu_mppi=True).")
        t1 = time.perf_counter()
        pred_time_step[t] = (t1 - t0)

        last_rollout_inputs = Xroll
        last_rollout_weights = wroll

        # ---------- tube subset selection ----------
        idx_sub_cand, _, _, _, _ = select_tube_subset(
            Z_global=Z_GLOBAL,
            kernel_for_dopt=m_dxdot.kernel,
            rollout_inputs=last_rollout_inputs,
            rollout_weights=last_rollout_weights,
            total_subset_size=M_SUB,
            anchor_idx=ANCHOR_IDX,
            time_bins=TUBE_TIME_BINS,
            cov_eps=TUBE_COV_EPS,
            dopt_jitter=1e-6,
            fallback_candidates=TUBE_FALLBACK_CANDIDATES,
            alpha=TUBE_ALPHA,
        )

        inter = np.intersect1d(last_idx_sub, idx_sub_cand)
        overlap = float(len(inter)) / float(len(idx_sub_cand)) if len(idx_sub_cand) > 0 else 1.0

        need_rebuild = ((t - last_local_rebuild_t) >= int(LOCAL_REBUILD_EVERY)) or (overlap < float(LOCAL_OVERLAP_THRESH))
        if need_rebuild:
            idx_sub = np.array(idx_sub_cand, dtype=np.int64)
            pred_local_np = make_subset_predictors_bundle(idx_sub)
            unc_local_np  = build_subset_uncertainty_fn(m_dxdot, idx_sub)
            pred_local_tf, unc_local_tf, _ = make_local_tf_predictors_bundle(idx_sub)

            last_idx_sub = np.array(idx_sub, dtype=np.int64)
            last_local_rebuild_t = int(t)
        else:
            idx_sub = last_idx_sub

        # ---------- env step ----------
        obs2, r, terminated, truncated, info = env.step(np.array([u0], dtype=np.float32))
        s2 = np.array(obs_to_state(obs2), dtype=np.float64)
        total_reward += float(r)

        respawned = bool(isinstance(info, dict) and info.get("respawned", False))
        if not respawned:
            Xbuf.append(state_to_features(s[0], s[1], s[2], s[3], float(u0)))
            ydx_buf.append([s2[0] - s[0]])
            ydxdot_buf.append([s2[1] - s[1]])
            ydth_buf.append([wrap_pi(s2[2] - s[2])])
            ydthdot_buf.append([s2[3] - s[3]])

        # ---------- render (excluded from wall) ----------
        if record_rgb and RECORD_RGB_DEFAULT and (t % FRAME_STRIDE == 0):
            tv0 = time.perf_counter()
            W, H = int(RESIZE[0]), int(RESIZE[1])
            frame = render_cartpole_frame_from_state(x=s2[0], theta=s2[2], x_threshold=2.4, W=W, H=H)
            frames.append(frame)
            vis_time_s += (time.perf_counter() - tv0)

        # ---------- success tracking ----------
        hold_count, success, upright, centered = success_hold_update(s2, x_init, hold_count)

        if verbose and (t % 50 == 0):
            print(f"[t_global={t_global:04d}] u0={u0:+.2f}  unc_w={unc_w:.2f}  upright={upright} centered={centered} hold={hold_count}/{HOLD_STEPS}")

        # ---------- UPDATE (training time) ----------
        if ((t + 1) % UPDATE_EVERY == 0) and (len(Xbuf) >= 10):
            updates += 1

            Xnew  = np.asarray(Xbuf, dtype=np.float64)
            ydx   = np.asarray(ydx_buf, dtype=np.float64)
            ydxd  = np.asarray(ydxdot_buf, dtype=np.float64)
            ydth  = np.asarray(ydth_buf, dtype=np.float64)
            ydthd = np.asarray(ydthdot_buf, dtype=np.float64)

            # refit Z_GLOBAL
            Z_GLOBAL = refit_Z_global_multihead(
                Z_global=Z_GLOBAL,
                Xnew=Xnew,
                M_global=M_GLOBAL,
                lam=ANCHOR_LAM,
                mode="mean",
                normalize_traces=False,
            )

            # train 4 heads (measure training time precisely)
            ttrain0 = time.perf_counter()
            m_dx,     info_dx     = osgpr_stream_update(m_dx,     Xnew, ydx,   Z_GLOBAL, iters=ITERS_UPDATE, lr=LR_UPDATE, noise=NOISE_UPDATE, freeze_kernel=False)
            m_dxdot,  info_dxdot  = osgpr_stream_update(m_dxdot,  Xnew, ydxd,  Z_GLOBAL, iters=ITERS_UPDATE, lr=LR_UPDATE, noise=NOISE_UPDATE, freeze_kernel=False)
            m_dth,    info_dth    = osgpr_stream_update(m_dth,    Xnew, ydth,  Z_GLOBAL, iters=ITERS_UPDATE, lr=LR_UPDATE, noise=NOISE_UPDATE, freeze_kernel=False)
            m_dthdot, info_dthdot = osgpr_stream_update(m_dthdot, Xnew, ydthd, Z_GLOBAL, iters=ITERS_UPDATE, lr=LR_UPDATE, noise=NOISE_UPDATE, freeze_kernel=False)
            ttrain1 = time.perf_counter()

            train_time_step[t] += float(ttrain1 - ttrain0)

            # clear buffers
            Xbuf, ydx_buf, ydxdot_buf, ydth_buf, ydthdot_buf = [], [], [], [], []

            # reselect anchors
            ANCHOR_IDX = compute_anchor_idx_dopt_from_Zglobal_multihead(
                Z_GLOBAL, m_anchors=ANCHOR_M, lam=ANCHOR_LAM, normalize_traces=True
            )

            # force rebuild subset using last rollout distribution
            idx_sub, _, _, _, _ = select_tube_subset(
                Z_global=Z_GLOBAL,
                kernel_for_dopt=m_dxdot.kernel,
                rollout_inputs=last_rollout_inputs,
                rollout_weights=last_rollout_weights,
                total_subset_size=M_SUB,
                anchor_idx=ANCHOR_IDX,
                time_bins=TUBE_TIME_BINS,
                cov_eps=TUBE_COV_EPS,
                dopt_jitter=1e-6,
                fallback_candidates=TUBE_FALLBACK_CANDIDATES,
                alpha=TUBE_ALPHA,
            )

            pred_local_np = make_subset_predictors_bundle(idx_sub)
            unc_local_np  = build_subset_uncertainty_fn(m_dxdot, idx_sub)
            pred_local_tf, unc_local_tf, _ = make_local_tf_predictors_bundle(idx_sub)

            last_idx_sub = np.array(idx_sub, dtype=np.int64)
            last_local_rebuild_t = int(t)

            # store global surfaces (excluded from timing; but we store only, no plot)
            if STORE_GLOBAL_SURFACES:
                muG, stdG = gp_predict_mu_std_fast(m_dxdot, PLOT_XFEAT_GRID)
                MeanG = muG.reshape(PLOT_XMESH.shape)
                StdG  = stdG.reshape(PLOT_XMESH.shape)
                GLOBAL_SURF_HISTORY.append(dict(update=int(t_global), Mean=MeanG, Std=StdG))
                _ = update_unified_std_scale([stdG.reshape(-1)], q=STD_FIXED_Q, mode=STD_MODE)

        # ---------- wall time cumulative excl vis ----------
        wall_excl_vis_cum[t] = max((time.perf_counter() - t_wall_start) - vis_time_s, 0.0)

        # advance
        s = s2

        # terminate
        if success or terminated or truncated:
            break

    env.close()

    steps = int(t + 1)

    stats = dict(
        total_reward=float(total_reward),
        steps=steps,
        updates=int(updates),
        frames=int(len(frames)),
        x_init=float(x_init),
        hold_steps=int(hold_count),
        Z_global_size=int(len(Z_GLOBAL)),
        std_cmax_fixed=float(STD_CMAX_FIXED) if STD_CMAX_FIXED is not None else None,
        vis_time_s=float(vis_time_s),
    )

    frames_out = frames if (return_frames and record_rgb and RECORD_RGB_DEFAULT) else None

    # Return per-step arrays trimmed to actual length + final idx_sub
    return stats, frames_out, idx_sub, pred_time_step[:steps], train_time_step[:steps], wall_excl_vis_cum[:steps]

# ============================================================
# MULTI-RUN driver  (same parameters as your code)
# ============================================================
N_RUNS = 3
N_EPISODES_PER_RUN = 3

MAX_STEPS_PER_EP = 1000
USE_GPU_MPPI = True
START_DOWN = True
VERBOSE = True

# Snapshot INITIAL models & initial Z_GLOBAL ONCE (used to reset before each RUN)
if "Z_GLOBAL" not in globals():
    raise NameError("Z_GLOBAL must exist before Cell 6. Define initial inducing points in earlier cells.")
Z_GLOBAL_INIT = np.asarray(Z_GLOBAL, dtype=np.float64).copy()

# Capture initial parameters for each model (safe reset)
params_dx_init     = parameter_dict(m_dx)
params_dxdot_init  = parameter_dict(m_dxdot)
params_dth_init    = parameter_dict(m_dth)
params_dthdot_init = parameter_dict(m_dthdot)

def reset_models_and_globals_for_fresh_run():
    global m_dx, m_dxdot, m_dth, m_dthdot
    global Z_GLOBAL
    global GLOBAL_SURF_HISTORY
    global STD_CMAX_FIXED

    # reset inducing points
    Z_GLOBAL = Z_GLOBAL_INIT.copy()
    if Z_GLOBAL.shape[0] != int(M_GLOBAL):
        if Z_GLOBAL.shape[0] > int(M_GLOBAL):
            Z_GLOBAL = Z_GLOBAL[:int(M_GLOBAL)].copy()
        else:
            raise ValueError(f"Z_GLOBAL_INIT has {Z_GLOBAL.shape[0]} < M_GLOBAL={M_GLOBAL}.")

    # restore model params
    multiple_assign(m_dx,     params_dx_init)
    multiple_assign(m_dxdot,  params_dxdot_init)
    multiple_assign(m_dth,    params_dth_init)
    multiple_assign(m_dthdot, params_dthdot_init)

    # rebuild caches
    try: m_dx.build_predict_cache()
    except Exception: pass
    try: m_dxdot.build_predict_cache()
    except Exception: pass
    try: m_dth.build_predict_cache()
    except Exception: pass
    try: m_dthdot.build_predict_cache()
    except Exception: pass

    # reset per-run histories/scales
    GLOBAL_SURF_HISTORY = []
    STD_CMAX_FIXED = None

# Storage for run-level metrics (arrays with variable lengths)
run_pred_time = []
run_train_time = []
run_wall_cum = []
run_rewards = []
run_steps_total = []
run_updates_total = []

for run in range(N_RUNS):
    print(f"\n==================== RUN {run+1}/{N_RUNS} (fresh reset) ====================")
    reset_models_and_globals_for_fresh_run()

    # run-level concatenated arrays
    pred_concat = []
    train_concat = []
    wall_concat = []
    t_offset = 0
    last_idx_sub = None

    # ONE render per RUN: only capture frames from EP0, but animation stitches EP0+EP1+EP2 by reusing EP0 frames only?
    # You asked: "combine all episodes into a single render for the run".
    # To do that while still "ONLY episode 0 renders", we capture frames ONLY in episode 0.
    # If you actually want visuals from ep1/ep2 too, set record_rgb=True for those eps as well.
    frames_run = []   # NEW: frames for the single RUN animation

    run_reward_sum = 0.0
    run_updates_sum = 0
    run_steps_sum = 0

    for ep in range(N_EPISODES_PER_RUN):
        record_rgb = (ep == 0)   # ✅ keep your constraint
        warmup_mppi = (ep == 0)  # warmup once per run

        print(f"\n--- RUN {run+1} EP {ep+1}/{N_EPISODES_PER_RUN} (t_offset={t_offset}) ---")

        stats_ep, frames_ep, idx_sub_ep, pred_t, train_t, wall_cum_t = run_one_episode_mppi_retrain_rgb_with_eval(
            max_steps=MAX_STEPS_PER_EP,
            seed=1000*run + ep,          # deterministic per run/ep
            start_down=START_DOWN,
            verbose=VERBOSE,
            use_gpu_mppi=USE_GPU_MPPI,
            warmup_mppi=warmup_mppi,
            record_rgb=record_rgb,
            t_offset_in_run=t_offset,
            return_frames=True,
        )

        # collect frames ONLY from EP0 (per your rendering rule)
        if record_rgb and (frames_ep is not None) and (len(frames_ep) > 0):
            frames_run.extend(frames_ep)

        last_idx_sub = idx_sub_ep

        pred_concat.append(pred_t)
        train_concat.append(train_t)
        wall_concat.append(wall_cum_t)

        run_reward_sum += stats_ep["total_reward"]
        run_updates_sum += stats_ep["updates"]
        run_steps_sum += stats_ep["steps"]

        t_offset += stats_ep["steps"]

    # concatenate (continuous timesteps across episodes in the run)
    pred_run = np.concatenate(pred_concat, axis=0)
    train_run = np.concatenate(train_concat, axis=0)
    wall_run = np.concatenate(wall_concat, axis=0)

    # recompute wall cumulative cleanly across concatenation (each episode’s wall_cum resets)
    wall_incr = np.zeros_like(wall_run)
    i0 = 0
    for ep_arr in wall_concat:
        ep_arr = np.asarray(ep_arr, dtype=np.float64)
        if ep_arr.size > 0:
            d = np.diff(np.concatenate([[0.0], ep_arr]))
            wall_incr[i0:i0+len(d)] = d
        i0 += len(ep_arr)
    wall_cum_run = np.cumsum(np.maximum(wall_incr, 0.0))

    # training cumulative
    train_cum_run = np.cumsum(train_run)

    # store run metrics
    run_pred_time.append(pred_run)
    run_train_time.append(train_run)
    run_wall_cum.append(wall_cum_run)
    run_rewards.append(run_reward_sum)
    run_steps_total.append(len(pred_run))
    run_updates_total.append(run_updates_sum)

    # ==========================
    # END OF RUN: SINGLE animation (EP0 frames) + surfaces
    # ==========================
    print(f"\n=== RUN {run+1}: single animation (episode 0 only, per your rule) ===")
    _ = display_run_animation_from_frames(frames_run, fps=FPS, resize=RESIZE)

    print(f"\n=== RUN {run+1}: final model surfaces ===")
    std_clim = (STD_CMIN_FIXED, STD_CMAX_FIXED if STD_CMAX_FIXED is not None else 1.0)

    plot_surface_global_or_local_dxdot(
        model_like=m_dxdot,
        is_local=False,
        title=f"RUN {run+1} FINAL — GLOBAL Δxdot (mean colored by Std)",
        std_clim=std_clim,
        show_colorbar=True
    )

    # OPTIONAL: local surface using last_idx_sub (you requested global + local)
    try:
        pred_local_dxdot, Zsub_overlay = build_subset_predictor_from_global(m_dxdot, Z_GLOBAL, last_idx_sub)
        plot_surface_global_or_local_dxdot(
            model_like=pred_local_dxdot,
            is_local=True,
            Z_overlay=Zsub_overlay,
            title=f"RUN {run+1} FINAL — LOCAL Δxdot (subset mean colored by Std)",
            std_clim=std_clim,
            show_colorbar=True
        )
    except Exception as e:
        print("⚠️ Local surface plot skipped due to error:", repr(e))

# ============================================================
# Aggregate across runs: mean curves (pad with NaN)
# ============================================================
maxT = int(max(run_steps_total))
def pad_to(arr, T):
    out = np.full((T,), np.nan, dtype=np.float64)
    out[:len(arr)] = arr
    return out

pred_mat  = np.vstack([pad_to(a, maxT) for a in run_pred_time])
train_mat = np.vstack([pad_to(a, maxT) for a in run_train_time])
wall_mat  = np.vstack([pad_to(a, maxT) for a in run_wall_cum])

# cumulative training per run then mean
train_cum_mat = np.vstack([pad_to(np.cumsum(a), maxT) for a in run_train_time])

pred_mean  = np.nanmean(pred_mat, axis=0)
train_mean = np.nanmean(train_mat, axis=0)
train_cum_mean = np.nanmean(train_cum_mat, axis=0)
wall_cum_mean  = np.nanmean(wall_mat, axis=0)

t_axis = np.arange(maxT)

# ============================================================
# Plots (mean over runs)
# ============================================================
plt.figure(figsize=(10, 3.2))
plt.plot(t_axis, train_mean)
plt.xlabel("timestep (within run, concatenated episodes)")
plt.ylabel("training time (s) per step")
plt.title("Mean training time per timestep (non-cumulative) over runs")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 3.2))
plt.plot(t_axis, train_cum_mean)
plt.xlabel("timestep (within run, concatenated episodes)")
plt.ylabel("cumulative training time (s)")
plt.title("Mean cumulative training time over runs")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 3.2))
plt.plot(t_axis, pred_mean)
plt.xlabel("timestep (within run, concatenated episodes)")
plt.ylabel("prediction time (s) per step (MPPI planning)")
plt.title("Mean prediction (MPPI) time per timestep over runs")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 3.2))
plt.plot(t_axis, wall_cum_mean)
plt.xlabel("timestep (within run, concatenated episodes)")
plt.ylabel("wall time cumulative (s) (EXCLUDING visualization)")
plt.title("Mean wall time cumulative over runs (no visualization)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

# ============================================================
# Summary stats
# ============================================================
print("\n==================== SUMMARY ====================")
for r in range(N_RUNS):
    print(f"RUN {r+1}: total_steps={run_steps_total[r]}, total_reward_sum={run_rewards[r]:.3f}, updates_sum={run_updates_total[r]}")
print("================================================\n")


In [ ]:
# ============================
# Cell 7 — BASELINE: Batch SVGP (4-head) + MPPI + Online retrain
#
# ✅ Matches your PLSSGP evaluation protocol:
#   - N_RUNS = 3
#   - Each run = 3 continuous episodes (model + dataset carry across episodes within run)
#   - BEFORE each run (including run 1): full reset (fresh models, fresh data, fresh Z)
#   - Same cost / success logic (upright+center hold)
#   - One render per run (episode 0 only)
#   - Logs per-timestep timings:
#       * train_time_step (non-cumulative)
#       * train_time_cum
#       * pred_time_step (MPPI time)
#       * wall_time_cum (excluding visualization)
#   - Plots MEAN curves across runs (nan-mean over variable lengths)
#   - Plots a GLOBAL surface after each run (Δxdot mean colored by std)
#
# Notes / assumptions:
#   - Uses GPflow SVGP (Gaussian likelihood) per head.
#   - Keeps inducing points size fixed at M_GLOBAL; updates variational params + hypers.
#   - Uses TF MPPI (GPU-capable) with GLOBAL predictors (no local subset / no tube).
# ============================

import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
import plotly.graph_objects as go
import gpflow
import tensorflow as tf

# -----------------------
# REQUIRED symbols from earlier cells
# -----------------------
required = [
    "make_env", "obs_to_state", "wrap_pi",
    "state_to_features", "batch_state_to_features",
]
missing = [k for k in required if k not in globals()]
if len(missing) > 0:
    raise NameError(f"SVGP Cell 7 missing required symbols from earlier cells: {missing}")

if "render_cartpole_frame_from_state" not in globals() and "render_cartpole_frame_from_state" not in globals():
    # your PLSSGP cell used render_cartpole_frame_from_state(...) (or render_cartpole_frame_from_state)
    # we'll handle both names
    pass

# -----------------------
# Dtypes (match your pipeline: float64)
# -----------------------
NP_DTYPE = np.float64
TF_DTYPE = gpflow.default_float()
if TF_DTYPE != tf.float64:
    print("⚠️ gpflow.default_float() is not float64. You said float64 throughout; set gpflow.config.set_default_float(np.float64) in Cell 1.")

# -----------------------
# Config defaults (prefer your globals if already defined)
# -----------------------
HORIZON    = int(globals().get("HORIZON", 50))
K_SAMPLES  = int(globals().get("K_SAMPLES", 256))
SIGMA      = float(globals().get("SIGMA", 0.6))
LAMBDA     = float(globals().get("LAMBDA", 1.0))

UPDATE_EVERY = int(globals().get("UPDATE_EVERY", 30))
ITERS_UPDATE = int(globals().get("ITERS_UPDATE", 150))
LR_UPDATE    = float(globals().get("LR_UPDATE", 0.02))
NOISE_UPDATE = float(globals().get("NOISE_UPDATE", 1e-4))

M_GLOBAL = int(globals().get("M_GLOBAL", 256))

# task/success (prefer your globals)
X_BAND      = float(globals().get("X_BAND", 0.70))
UPRIGHT_COS = float(globals().get("UPRIGHT_COS", 0.85))
HOLD_STEPS  = int(globals().get("HOLD_STEPS", 200))

# exploration schedule (prefer your globals)
EXPLORE_STEPS = int(globals().get("EXPLORE_STEPS", 200))
UNC_W_MAX     = float(globals().get("UNC_W_MAX", 15.0))
UNC_W_MIN     = float(globals().get("UNC_W_MIN", 0.0))
CENTER_W      = float(globals().get("CENTER_W", 1.0))
U_W           = float(globals().get("U_W", 0.005))
UPRIGHT_W     = float(globals().get("UPRIGHT_W", 2.0))

# action bounds (prefer earlier-cell globals)
U_MIN = float(globals().get("U_MIN", -1.0))
U_MAX = float(globals().get("U_MAX", +1.0))

# rendering
RECORD_RGB   = True
RESIZE       = globals().get("RESIZE", (720, 450))
FPS          = int(globals().get("FPS", 10))
FRAME_STRIDE = int(globals().get("FRAME_STRIDE", 2))

# multi-run protocol
N_RUNS = 3
EPISODES_PER_RUN = 3
MAX_STEPS_PER_EP = int(globals().get("MAX_STEPS_PER_EP", 1000))
START_DOWN = bool(globals().get("START_DOWN", True))
USE_GPU_MPPI = True
WARMUP_MPPI = True
VERBOSE = True

# -----------------------
# Fixed plot grid (for surface after each run)
# -----------------------
PLOT_X_MIN, PLOT_X_MAX = globals().get("PLOT_X_MIN", -2.4), globals().get("PLOT_X_MAX", 2.4)
PLOT_V_MIN, PLOT_V_MAX = globals().get("PLOT_V_MIN", -3.0), globals().get("PLOT_V_MAX", 3.0)
PLOT_N_GRID            = int(globals().get("PLOT_N_GRID", 60))
PLOT_TH_FIXED          = float(globals().get("PLOT_TH_FIXED", 0.0))
PLOT_THDOT_FIXED       = float(globals().get("PLOT_THDOT_FIXED", 0.0))
PLOT_U_FIXED           = float(globals().get("PLOT_U_FIXED", +1.0))

def build_fixed_plot_grid():
    xg = np.linspace(PLOT_X_MIN, PLOT_X_MAX, PLOT_N_GRID)
    vg = np.linspace(PLOT_V_MIN, PLOT_V_MAX, PLOT_N_GRID)
    X, V = np.meshgrid(xg, vg)
    Xfeat_grid = np.vstack([
        state_to_features(x, xdot, PLOT_TH_FIXED, PLOT_THDOT_FIXED, PLOT_U_FIXED)
        for x, xdot in zip(X.ravel(), V.ravel())
    ]).astype(np.float64)
    return X, V, Xfeat_grid

PLOT_XMESH, PLOT_VMESH, PLOT_XFEAT_GRID = build_fixed_plot_grid()

def gp_predict_mu_std(model, X):
    Xtf = tf.convert_to_tensor(np.asarray(X, dtype=np.float64), dtype=TF_DTYPE)
    mu_tf, var_tf = model.predict_f(Xtf, full_cov=False)
    mu = mu_tf.numpy().reshape(-1)
    var = var_tf.numpy().reshape(-1)
    std = np.sqrt(np.maximum(var, 1e-12))
    return mu, std

def plot_global_surface_dxdot(model_dxdot, title="GLOBAL Δxdot surface"):
    mu, std = gp_predict_mu_std(model_dxdot, PLOT_XFEAT_GRID)
    Mean = mu.reshape(PLOT_XMESH.shape)
    Std  = std.reshape(PLOT_XMESH.shape)

    fig = go.Figure(data=[
        go.Surface(
            x=PLOT_XMESH, y=PLOT_VMESH, z=Mean,
            surfacecolor=Std,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Std"),
            opacity=0.95
        )
    ])
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title="x", range=[PLOT_X_MIN, PLOT_X_MAX]),
            yaxis=dict(title="xdot", range=[PLOT_V_MIN, PLOT_V_MAX]),
            zaxis=dict(title="Δxdot"),
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=620
    )
    fig.show()

# -----------------------
# Success check (same logic as your PLSSGP cell)
# -----------------------
def success_hold_update(state, x_init, hold_count):
    x, xdot, th, thdot = state
    upright = (np.cos(th) >= UPRIGHT_COS)
    centered = (abs(x - x_init) <= X_BAND)
    hold_count = (hold_count + 1) if (upright and centered) else 0
    success = (hold_count >= HOLD_STEPS)
    return hold_count, success, upright, centered

# -----------------------
# Cost + exploration weight (same functional form as your PLSSGP cell)
# -----------------------
@tf.function
def exploration_weight_tf(t):
    explore_steps_f = tf.cast(EXPLORE_STEPS, TF_DTYPE)
    t_f = tf.cast(t, TF_DTYPE)
    a = tf.clip_by_value(1.0 - t_f / tf.maximum(explore_steps_f, 1.0), 0.0, 1.0)
    return tf.cast(UNC_W_MIN, TF_DTYPE) + (tf.cast(UNC_W_MAX, TF_DTYPE) - tf.cast(UNC_W_MIN, TF_DTYPE)) * a

@tf.function
def wrap_pi_tf(theta):
    two_pi = tf.constant(2.0 * np.pi, dtype=TF_DTYPE)
    pi = tf.constant(np.pi, dtype=TF_DTYPE)
    return tf.math.floormod(theta + pi, two_pi) - pi

@tf.function
def batch_state_to_features_tf(S, U):
    x = S[:, 0]
    xdot = S[:, 1]
    th = S[:, 2]
    thdot = S[:, 3]

    f0 = tf.tanh(x / tf.constant(2.4, dtype=TF_DTYPE))
    f1 = tf.tanh(xdot / tf.constant(3.0, dtype=TF_DTYPE))
    f2 = tf.sin(th)
    f3 = tf.cos(th)
    f4 = tf.tanh(thdot / tf.constant(8.0, dtype=TF_DTYPE))
    f5 = U
    return tf.stack([f0, f1, f2, f3, f4, f5], axis=1)

@tf.function
def stage_cost_cartpole_tf(S, U, x_init, unc_bonus=None, unc_w=0.0):
    th = S[:, 2]
    x  = S[:, 0]
    c = (
        tf.cast(UPRIGHT_W, TF_DTYPE) * (1.0 - tf.cos(th))
        + tf.cast(CENTER_W, TF_DTYPE) * tf.square(x - x_init)
        + tf.cast(U_W, TF_DTYPE) * tf.square(U)
    )
    if (unc_bonus is not None) and (unc_w > 0.0):
        c = c - tf.cast(unc_w, TF_DTYPE) * tf.cast(unc_bonus, TF_DTYPE)
    return c

@tf.function
def terminal_cost_hold_like_tf(S, x_init):
    th = S[:, 2]
    x  = S[:, 0]
    cT = tf.zeros_like(th)
    good = tf.logical_and(tf.cos(th) >= tf.cast(UPRIGHT_COS, TF_DTYPE),
                          tf.abs(x - x_init) <= tf.cast(X_BAND, TF_DTYPE))
    cT = tf.where(good, cT - tf.cast(5.0, TF_DTYPE), cT)
    return cT

# -----------------------
# SVGP model factory (4 heads)
# -----------------------
def make_svgp_head(Z_init, kernel, noise=1e-4):
    Z_init = np.asarray(Z_init, dtype=np.float64)
    inducing = gpflow.inducing_variables.InducingPoints(tf.convert_to_tensor(Z_init, dtype=TF_DTYPE))
    lik = gpflow.likelihoods.Gaussian(variance=float(noise))

    # q_mu/q_sqrt will be sized to inducing points (M,1)
    m = gpflow.models.SVGP(
        kernel=kernel,
        likelihood=lik,
        inducing_variable=inducing,
        num_latent_gps=1,
        whiten=True
    )
    return m

def make_four_head_svgp_models(Z_init):
    # IMPORTANT: use independent kernel objects for each head (no shared variables)
    # If you already have a kernel factory in earlier cells, you can swap it in here.
    def _base_kernel():
        # Default: SE-ARD over 6D feature space
        return gpflow.kernels.SquaredExponential(lengthscales=np.ones((6,), dtype=np.float64), variance=1.0)

    m_dx     = make_svgp_head(Z_init, _base_kernel(), noise=NOISE_UPDATE)
    m_dxdot  = make_svgp_head(Z_init, _base_kernel(), noise=NOISE_UPDATE)
    m_dth    = make_svgp_head(Z_init, _base_kernel(), noise=NOISE_UPDATE)
    m_dthdot = make_svgp_head(Z_init, _base_kernel(), noise=NOISE_UPDATE)
    return m_dx, m_dxdot, m_dth, m_dthdot

# -----------------------
# SVGP training step (batch retrain on ALL data so far in the run)
# -----------------------
def train_svgp_batch(model, X, Y, iters=150, lr=0.02, clip_norm=10.0):
    Xtf = tf.convert_to_tensor(np.asarray(X, dtype=np.float64), dtype=TF_DTYPE)
    Ytf = tf.convert_to_tensor(np.asarray(Y, dtype=np.float64).reshape(-1,1), dtype=TF_DTYPE)

    opt = tf.optimizers.Adam(learning_rate=float(lr))

    @tf.function
    def step():
        with tf.GradientTape() as tape:
            loss = -model.elbo((Xtf, Ytf))
        vars_ = model.trainable_variables
        grads = tape.gradient(loss, vars_)
        if clip_norm is not None:
            grads, _ = tf.clip_by_global_norm(grads, clip_norm)
        opt.apply_gradients(zip(grads, vars_))
        return loss

    t0 = time.perf_counter()
    last = None
    for _ in range(int(iters)):
        last = step()
    t_train = time.perf_counter() - t0
    return float(t_train), float(last.numpy() if last is not None else np.nan)

# -----------------------
# GLOBAL predictors for TF MPPI
# -----------------------
def make_global_tf_predictors_bundle_from_svgp(m_dx, m_dxdot, m_dth, m_dthdot):
    @tf.function
    def dx_mu(Xfeat):
        mu, _ = m_dx.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dxdot_mu(Xfeat):
        mu, _ = m_dxdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dth_mu(Xfeat):
        mu, _ = m_dth.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def thdot_mu(Xfeat):
        mu, _ = m_dthdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dxdot_var(Xfeat):
        _, v = m_dxdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(v, (-1,))

    return (dx_mu, dxdot_mu, dth_mu, thdot_mu), dxdot_var

@tf.function
def gp_dynamics_step_batch_tf(S, U, pred_bundle_tf):
    pred_dx_tf, pred_dxdot_tf, pred_dth_tf, pred_dthdot_tf = pred_bundle_tf
    Xfeat = batch_state_to_features_tf(S, U)
    dx     = pred_dx_tf(Xfeat)
    dxdot  = pred_dxdot_tf(Xfeat)
    dth    = pred_dth_tf(Xfeat)
    dthdot = pred_dthdot_tf(Xfeat)

    S2 = tf.stack([
        S[:, 0] + dx,
        S[:, 1] + dxdot,
        wrap_pi_tf(S[:, 2] + dth),
        S[:, 3] + dthdot
    ], axis=1)
    return S2, Xfeat

@tf.function
def rollout_tube_features_tf(state0, u_seq, pred_bundle_tf):
    # (kept for parity; baseline doesn't use tube selection)
    H = tf.shape(u_seq)[0]
    s = tf.identity(state0)
    tube = tf.TensorArray(dtype=TF_DTYPE, size=H)

    t = tf.constant(0)
    def cond(t, s, tube): return t < H

    def body(t, s, tube):
        u = u_seq[t]
        xfeat = batch_state_to_features_tf(tf.expand_dims(s, axis=0), tf.expand_dims(u, axis=0))[0]
        tube = tube.write(t, xfeat)
        s2, _ = gp_dynamics_step_batch_tf(tf.expand_dims(s, axis=0), tf.expand_dims(u, axis=0), pred_bundle_tf)
        return t+1, s2[0], tube

    _, _, tube = tf.while_loop(cond, body, [t, s, tube], parallel_iterations=1)
    return tube.stack()

@tf.function
def mppi_plan_gpu_global_tf(state0, x_init, u_mean0, t_global,
                           pred_bundle_tf, unc_var_fn_tf,
                           horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
                           base_seed=0):
    H = tf.cast(horizon, tf.int32)
    Kt = tf.cast(K, tf.int32)

    seed = tf.stack([tf.cast(base_seed, tf.int32), tf.cast(t_global, tf.int32)], axis=0)
    eps = tf.random.stateless_normal((Kt, H), seed=seed, mean=0.0, stddev=tf.cast(sigma, TF_DTYPE), dtype=TF_DTYPE)
    U = tf.clip_by_value(u_mean0[None, :] + eps, tf.cast(U_MIN, TF_DTYPE), tf.cast(U_MAX, TF_DTYPE))

    S = tf.tile(state0[None, :], [Kt, 1])
    total_cost = tf.zeros((Kt,), dtype=TF_DTYPE)

    unc_w = exploration_weight_tf(t_global)

    t = tf.constant(0, dtype=tf.int32)
    def cond(t, S, total_cost): return t < H

    def body(t, S, total_cost):
        Ut = U[:, t]
        S2, Xfeat = gp_dynamics_step_batch_tf(S, Ut, pred_bundle_tf)
        unc_bonus = unc_var_fn_tf(Xfeat)
        total_cost = total_cost + stage_cost_cartpole_tf(
            S, Ut, x_init=tf.cast(x_init, TF_DTYPE),
            unc_bonus=unc_bonus, unc_w=unc_w
        )
        return t+1, S2, total_cost

    _, S, total_cost = tf.while_loop(cond, body, [t, S, total_cost], parallel_iterations=1)
    total_cost = total_cost + terminal_cost_hold_like_tf(S, x_init=tf.cast(x_init, TF_DTYPE))

    cmin = tf.reduce_min(total_cost)
    w = tf.exp(-(total_cost - cmin) / tf.cast(lam, TF_DTYPE))
    wsum = tf.reduce_sum(w) + tf.cast(1e-12, TF_DTYPE)

    u_mean = u_mean0 + tf.reduce_sum(w[:, None] * eps, axis=0) / wsum
    u_mean = tf.clip_by_value(u_mean, tf.cast(U_MIN, TF_DTYPE), tf.cast(U_MAX, TF_DTYPE))

    tubeX = rollout_tube_features_tf(state0, u_mean, pred_bundle_tf)
    return u_mean[0], u_mean, tubeX, unc_w

def mppi_plan_gpu_global(state, x_init, u_init, t_global, pred_bundle_tf, unc_var_fn_tf, base_seed=0):
    state0 = tf.convert_to_tensor(np.asarray(state, dtype=np.float64).reshape(4,), dtype=TF_DTYPE)
    x0     = tf.convert_to_tensor(float(x_init), dtype=TF_DTYPE)
    u0     = tf.convert_to_tensor(np.asarray(u_init, dtype=np.float64).reshape(-1,), dtype=TF_DTYPE)

    dev = "/GPU:0" if len(tf.config.list_logical_devices("GPU")) > 0 else "/CPU:0"
    with tf.device(dev):
        u_first, u_mean, tubeX, unc_w = mppi_plan_gpu_global_tf(
            state0, x0, u0, tf.convert_to_tensor(int(t_global), dtype=tf.int32),
            pred_bundle_tf=pred_bundle_tf,
            unc_var_fn_tf=unc_var_fn_tf,
            horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
            base_seed=int(base_seed)
        )
    return float(u_first.numpy()), u_mean.numpy(), tubeX.numpy(), float(unc_w.numpy())

# -----------------------
# Rendering helper compatibility
# -----------------------
def _render_frame_from_state(s):
    W, H = int(RESIZE[0]), int(RESIZE[1])
    if "render_cartpole_frame_from_state" in globals():
        return render_cartpole_frame_from_state(x=s[0], theta=s[2], x_threshold=2.4, W=W, H=H)
    if "render_cartpole_frame_from_state" in globals():
        return render_cartpole_frame_from_state(x=s[0], theta=s[2], x_threshold=2.4, W=W, H=H)
    raise NameError("No render_cartpole_frame_from_state(...) found; set RECORD_RGB=False or define the renderer in earlier cells.")

# ============================
# RUNNER: one run = 3 episodes continuous
# ============================
def run_one_run_svgp(run_id, seed_base=0):
    # ---- FRESH RUN RESET (critical!) ----
    # Z init must exist from earlier cells
    if "Z_GLOBAL" not in globals():
        raise NameError("Z_GLOBAL must be defined in earlier cells (initial inducing points in feature space).")
    Z0 = np.asarray(Z_GLOBAL, dtype=np.float64)
    if Z0.shape[0] < M_GLOBAL:
        raise ValueError(f"Z_GLOBAL has {Z0.shape[0]} points but M_GLOBAL={M_GLOBAL}. Provide at least {M_GLOBAL} points.")
    Z0 = Z0[:M_GLOBAL].copy()

    # Fresh models per run
    m_dx, m_dxdot, m_dth, m_dthdot = make_four_head_svgp_models(Z0)

    # Fresh dataset per run (but accumulates across episodes within run)
    Xall, ydx_all, ydxdot_all, ydth_all, ydthdot_all = [], [], [], [], []

    # timing traces per timestep (for the whole run)
    train_time_step = []
    pred_time_step  = []
    wall_time_cum_excl_vis = []
    train_time_cum = []
    pred_time_cum  = []

    # 1 render per run (episode 0 only)
    frames = []
    html = None
    vis_time_s_total = 0.0

    # global cumulative counters
    t_global = 0
    total_reward_run = 0.0
    updates_run = 0

    # warmup compile once (optional)
    pred_bundle_tf, unc_var_fn_tf = make_global_tf_predictors_bundle_from_svgp(m_dx, m_dxdot, m_dth, m_dthdot)
    if USE_GPU_MPPI and WARMUP_MPPI:
        s_dummy = np.array([0.0, 0.0, np.pi, 0.0], dtype=np.float64)
        u_mean0 = np.zeros((HORIZON,), dtype=np.float64)
        _ = mppi_plan_gpu_global(s_dummy, x_init=0.0, u_init=u_mean0, t_global=0,
                                pred_bundle_tf=pred_bundle_tf, unc_var_fn_tf=unc_var_fn_tf,
                                base_seed=seed_base + 1000*run_id)

    t_wall_start = time.perf_counter()
    wall_excl_vis = 0.0
    train_cum = 0.0
    pred_cum  = 0.0

    # ---- 3 continuous episodes ----
    for ep in range(EPISODES_PER_RUN):
        seed = int(seed_base + 1000 * run_id + ep)
        env = make_env(render_mode=None, seed=seed, max_episode_steps=MAX_STEPS_PER_EP, start_down=START_DOWN)
        obs, info = env.reset(seed=seed)
        s = np.array(obs_to_state(obs), dtype=np.float64)
        x_init = float(s[0])

        u_mean = np.zeros((HORIZON,), dtype=np.float64)
        hold_count = 0
        record_this_ep = (ep == 0) and RECORD_RGB

        if VERBOSE:
            print(f"\n[SVGP] Run {run_id+1}/{N_RUNS} | Episode {ep+1}/{EPISODES_PER_RUN} | seed={seed}")

        for t_ep in range(MAX_STEPS_PER_EP):
            # ---- prediction / planning (time this) ----
            t0 = time.perf_counter()
            u0, u_mean, tubeX, unc_w = mppi_plan_gpu_global(
                state=s, x_init=x_init, u_init=u_mean, t_global=t_global,
                pred_bundle_tf=pred_bundle_tf, unc_var_fn_tf=unc_var_fn_tf,
                base_seed=seed_base + 1000*run_id
            )
            t1 = time.perf_counter()
            pred_dt = (t1 - t0)
            pred_cum += pred_dt

            # ---- step env ----
            obs2, r, terminated, truncated, info = env.step(np.array([u0], dtype=np.float32))
            s2 = np.array(obs_to_state(obs2), dtype=np.float64)
            total_reward_run += float(r)

            # buffer data (skip respawned if your env uses it)
            respawned = bool(isinstance(info, dict) and info.get("respawned", False))
            if not respawned:
                Xall.append(state_to_features(s[0], s[1], s[2], s[3], float(u0)))
                ydx_all.append([s2[0] - s[0]])
                ydxdot_all.append([s2[1] - s[1]])
                ydth_all.append([wrap_pi(s2[2] - s[2])])
                ydthdot_all.append([s2[3] - s[3]])

            # ---- render (excluded from wall_excl_vis) ----
            if record_this_ep and (t_global % FRAME_STRIDE == 0):
                tv0 = time.perf_counter()
                try:
                    frames.append(_render_frame_from_state(s2))
                except Exception:
                    pass
                vis_time_s_total += (time.perf_counter() - tv0)

            # ---- success ----
            hold_count, success, upright, centered = success_hold_update(s2, x_init, hold_count)

            if VERBOSE and (t_global % 50 == 0):
                print(f"[t={t_global:04d}] u0={u0:+.2f} unc_w={unc_w:.2f} upright={upright} centered={centered} hold={hold_count}/{HOLD_STEPS}")

            # ---- online retrain (batch SVGP on ALL data so far) ----
            train_dt = 0.0
            if ((t_global + 1) % UPDATE_EVERY == 0) and (len(Xall) >= 10):
                updates_run += 1

                Xnp   = np.asarray(Xall, dtype=np.float64)
                ydx   = np.asarray(ydx_all, dtype=np.float64)
                ydxd  = np.asarray(ydxdot_all, dtype=np.float64)
                ydth  = np.asarray(ydth_all, dtype=np.float64)
                ydthd = np.asarray(ydthdot_all, dtype=np.float64)

                # train each head (time total training)
                tt0 = time.perf_counter()
                t_dx,   _ = train_svgp_batch(m_dx,     Xnp, ydx,   iters=ITERS_UPDATE, lr=LR_UPDATE)
                t_v,    _ = train_svgp_batch(m_dxdot,  Xnp, ydxd,  iters=ITERS_UPDATE, lr=LR_UPDATE)
                t_th,   _ = train_svgp_batch(m_dth,    Xnp, ydth,  iters=ITERS_UPDATE, lr=LR_UPDATE)
                t_thd,  _ = train_svgp_batch(m_dthdot, Xnp, ydthd, iters=ITERS_UPDATE, lr=LR_UPDATE)
                train_dt = time.perf_counter() - tt0

                train_cum += train_dt

                # refresh TF predictor bundle (same objects, but safe)
                pred_bundle_tf, unc_var_fn_tf = make_global_tf_predictors_bundle_from_svgp(m_dx, m_dxdot, m_dth, m_dthdot)

            # ---- timing logs (per timestep in the run) ----
            total_wall = time.perf_counter() - t_wall_start
            wall_excl_vis = max(total_wall - vis_time_s_total, 0.0)

            train_time_step.append(float(train_dt))
            pred_time_step.append(float(pred_dt))
            train_time_cum.append(float(train_cum))
            pred_time_cum.append(float(pred_cum))
            wall_time_cum_excl_vis.append(float(wall_excl_vis))

            # advance
            s = s2
            t_global += 1

            # stop episode
            if success:
                if VERBOSE:
                    print(f"✅ SUCCESS at global t={t_global-1} (held {HOLD_STEPS} steps)")
                break
            if terminated or truncated:
                if VERBOSE:
                    print(f"Episode ended at global t={t_global-1} (terminated={terminated}, truncated={truncated})")
                break

        env.close()

    # ---- one render per run ----
    if RECORD_RGB and (len(frames) > 0):
        tv0 = time.perf_counter()
        fig = plt.figure(figsize=(RESIZE[0]/100, RESIZE[1]/100), dpi=100)
        plt.axis("off")
        im = plt.imshow(frames[0])

        def animate_fn(i):
            im.set_data(frames[i])
            return [im]

        ani = animation.FuncAnimation(fig, animate_fn, frames=len(frames),
                                      interval=1000 / float(FPS), blit=True)
        plt.close(fig)
        html = HTML(ani.to_jshtml())
        display(html)
        vis_time_s_total += (time.perf_counter() - tv0)

    run_stats = dict(
        run_id=int(run_id),
        total_reward=float(total_reward_run),
        total_steps=int(len(train_time_step)),
        updates=int(updates_run),
        vis_time_s=float(vis_time_s_total),
        wall_excl_vis_s=float(wall_time_cum_excl_vis[-1] if len(wall_time_cum_excl_vis)>0 else 0.0),
        train_cum_s=float(train_time_cum[-1] if len(train_time_cum)>0 else 0.0),
        pred_cum_s=float(pred_time_cum[-1] if len(pred_time_cum)>0 else 0.0),
    )

    traces = dict(
        train_time_step=np.asarray(train_time_step, dtype=np.float64),
        train_time_cum=np.asarray(train_time_cum, dtype=np.float64),
        pred_time_step=np.asarray(pred_time_step, dtype=np.float64),
        pred_time_cum=np.asarray(pred_time_cum, dtype=np.float64),
        wall_time_cum_excl_vis=np.asarray(wall_time_cum_excl_vis, dtype=np.float64),
    )

    models = dict(m_dx=m_dx, m_dxdot=m_dxdot, m_dth=m_dth, m_dthdot=m_dthdot)

    return run_stats, traces, models, html

# ============================
# MULTI-RUN DRIVER (3 runs)
# ============================
all_stats = []
all_traces = []
all_models = []

for r in range(N_RUNS):
    stats_r, traces_r, models_r, html_r = run_one_run_svgp(run_id=r, seed_base=0)
    all_stats.append(stats_r)
    all_traces.append(traces_r)
    all_models.append(models_r)

    print(f"\n[SVGP] Run {r+1} stats:", stats_r)

    # surface after each run
    plot_global_surface_dxdot(
        models_r["m_dxdot"],
        title=f"SVGP baseline — GLOBAL Δxdot after Run {r+1} (mean colored by std)"
    )

# ============================
# MEAN PLOTS ACROSS RUNS (nan-mean over variable lengths)
# ============================
def nanpad_to(T, L):
    out = np.full((L,), np.nan, dtype=np.float64)
    n = min(L, len(T))
    out[:n] = T[:n]
    return out

maxL = max([len(tr["train_time_step"]) for tr in all_traces]) if len(all_traces)>0 else 0
if maxL == 0:
    raise RuntimeError("No timesteps were recorded (did the env terminate immediately?).")

train_step_mat = np.vstack([nanpad_to(tr["train_time_step"], maxL) for tr in all_traces])
train_cum_mat  = np.vstack([nanpad_to(tr["train_time_cum"], maxL) for tr in all_traces])
pred_step_mat  = np.vstack([nanpad_to(tr["pred_time_step"], maxL) for tr in all_traces])
wall_cum_mat   = np.vstack([nanpad_to(tr["wall_time_cum_excl_vis"], maxL) for tr in all_traces])

train_step_mean = np.nanmean(train_step_mat, axis=0)
train_cum_mean  = np.nanmean(train_cum_mat, axis=0)
pred_step_mean  = np.nanmean(pred_step_mat, axis=0)
wall_cum_mean   = np.nanmean(wall_cum_mat, axis=0)

t_axis = np.arange(maxL)

plt.figure(figsize=(10,4))
plt.plot(t_axis, train_step_mean)
plt.title("SVGP baseline — mean TRAIN time per timestep (non-cumulative) across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("train time at step (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, train_cum_mean)
plt.title("SVGP baseline — mean TRAIN time cumulative across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("cumulative train time (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, pred_step_mean)
plt.title("SVGP baseline — mean PRED/MPPI time per timestep across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("prediction/planning time at step (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, wall_cum_mean)
plt.title("SVGP baseline — mean WALL time cumulative (excluding visualization) across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("cumulative wall time excl. vis (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

print("\n[SVGP baseline] Summary across runs:")
for s in all_stats:
    print(s)


In [ ]:
# ============================
# Cell — BASELINE: Exact GP (4-head) + MPPI + Online retrain (batch on ALL data so far)
#   Mirrors your SVGP baseline protocol/logs.
# ============================

import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
import plotly.graph_objects as go
import gpflow
import tensorflow as tf

# -----------------------
# REQUIRED symbols from earlier cells
# -----------------------
required = [
    "make_env", "obs_to_state", "wrap_pi",
    "state_to_features", "batch_state_to_features",
]
missing = [k for k in required if k not in globals()]
if len(missing) > 0:
    raise NameError(f"ExactGP cell missing required symbols from earlier cells: {missing}")

# -----------------------
# Dtypes (float64 everywhere)
# -----------------------
NP_DTYPE = np.float64
TF_DTYPE = gpflow.default_float()
if TF_DTYPE != tf.float64:
    print("⚠️ gpflow.default_float() is not float64. Set gpflow.config.set_default_float(np.float64) in Cell 1.")

# -----------------------
# Config defaults (prefer your globals if already defined)
# -----------------------
HORIZON    = int(globals().get("HORIZON", 50))
K_SAMPLES  = int(globals().get("K_SAMPLES", 256))
SIGMA      = float(globals().get("SIGMA", 0.6))
LAMBDA     = float(globals().get("LAMBDA", 1.0))

UPDATE_EVERY = int(globals().get("UPDATE_EVERY", 30))

# For Exact GP, DO NOT use "iters=150" like Adam loops.
# Use a small number of L-BFGS iterations per update, otherwise it will be too slow.
EXACT_SCIPY_ITERS = int(globals().get("EXACT_SCIPY_ITERS", 20))

NOISE_UPDATE = float(globals().get("NOISE_UPDATE", 1e-4))

# task/success
X_BAND      = float(globals().get("X_BAND", 0.70))
UPRIGHT_COS = float(globals().get("UPRIGHT_COS", 0.85))
HOLD_STEPS  = int(globals().get("HOLD_STEPS", 200))

# exploration schedule
EXPLORE_STEPS = int(globals().get("EXPLORE_STEPS", 200))
UNC_W_MAX     = float(globals().get("UNC_W_MAX", 15.0))
UNC_W_MIN     = float(globals().get("UNC_W_MIN", 0.0))
CENTER_W      = float(globals().get("CENTER_W", 1.0))
U_W           = float(globals().get("U_W", 0.005))
UPRIGHT_W     = float(globals().get("UPRIGHT_W", 2.0))

# action bounds
U_MIN = float(globals().get("U_MIN", -1.0))
U_MAX = float(globals().get("U_MAX", +1.0))

# rendering
RECORD_RGB   = True
RESIZE       = globals().get("RESIZE", (720, 450))
FPS          = int(globals().get("FPS", 10))
FRAME_STRIDE = int(globals().get("FRAME_STRIDE", 2))

# multi-run protocol
N_RUNS = 3
EPISODES_PER_RUN = 3
MAX_STEPS_PER_EP = int(globals().get("MAX_STEPS_PER_EP", 1000))
START_DOWN = bool(globals().get("START_DOWN", True))
USE_GPU_MPPI = True
WARMUP_MPPI = True
VERBOSE = True

# -----------------------
# Fixed plot grid (for surface after each run)
# -----------------------
PLOT_X_MIN, PLOT_X_MAX = globals().get("PLOT_X_MIN", -2.4), globals().get("PLOT_X_MAX", 2.4)
PLOT_V_MIN, PLOT_V_MAX = globals().get("PLOT_V_MIN", -3.0), globals().get("PLOT_V_MAX", 3.0)
PLOT_N_GRID            = int(globals().get("PLOT_N_GRID", 60))
PLOT_TH_FIXED          = float(globals().get("PLOT_TH_FIXED", 0.0))
PLOT_THDOT_FIXED       = float(globals().get("PLOT_THDOT_FIXED", 0.0))
PLOT_U_FIXED           = float(globals().get("PLOT_U_FIXED", +1.0))

def build_fixed_plot_grid():
    xg = np.linspace(PLOT_X_MIN, PLOT_X_MAX, PLOT_N_GRID)
    vg = np.linspace(PLOT_V_MIN, PLOT_V_MAX, PLOT_N_GRID)
    X, V = np.meshgrid(xg, vg)
    Xfeat_grid = np.vstack([
        state_to_features(x, xdot, PLOT_TH_FIXED, PLOT_THDOT_FIXED, PLOT_U_FIXED)
        for x, xdot in zip(X.ravel(), V.ravel())
    ]).astype(np.float64)
    return X, V, Xfeat_grid

PLOT_XMESH, PLOT_VMESH, PLOT_XFEAT_GRID = build_fixed_plot_grid()

def gp_predict_mu_std(model, X):
    Xtf = tf.convert_to_tensor(np.asarray(X, dtype=np.float64), dtype=TF_DTYPE)
    mu_tf, var_tf = model.predict_f(Xtf, full_cov=False)
    mu = mu_tf.numpy().reshape(-1)
    var = var_tf.numpy().reshape(-1)
    std = np.sqrt(np.maximum(var, 1e-12))
    return mu, std

def plot_global_surface_dxdot(model_dxdot, title="GLOBAL Δxdot surface (Exact GP)"):
    mu, std = gp_predict_mu_std(model_dxdot, PLOT_XFEAT_GRID)
    Mean = mu.reshape(PLOT_XMESH.shape)
    Std  = std.reshape(PLOT_XMESH.shape)

    fig = go.Figure(data=[
        go.Surface(
            x=PLOT_XMESH, y=PLOT_VMESH, z=Mean,
            surfacecolor=Std,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Std"),
            opacity=0.95
        )
    ])
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title="x", range=[PLOT_X_MIN, PLOT_X_MAX]),
            yaxis=dict(title="xdot", range=[PLOT_V_MIN, PLOT_V_MAX]),
            zaxis=dict(title="Δxdot"),
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=620
    )
    fig.show()

# -----------------------
# Success check (same logic as your other cells)
# -----------------------
def success_hold_update(state, x_init, hold_count):
    x, xdot, th, thdot = state
    upright = (np.cos(th) >= UPRIGHT_COS)
    centered = (abs(x - x_init) <= X_BAND)
    hold_count = (hold_count + 1) if (upright and centered) else 0
    success = (hold_count >= HOLD_STEPS)
    return hold_count, success, upright, centered

# -----------------------
# Cost + exploration weight (same functional form as your SVGP cell)
# -----------------------
@tf.function
def exploration_weight_tf(t):
    explore_steps_f = tf.cast(EXPLORE_STEPS, TF_DTYPE)
    t_f = tf.cast(t, TF_DTYPE)
    a = tf.clip_by_value(1.0 - t_f / tf.maximum(explore_steps_f, 1.0), 0.0, 1.0)
    return tf.cast(UNC_W_MIN, TF_DTYPE) + (tf.cast(UNC_W_MAX, TF_DTYPE) - tf.cast(UNC_W_MIN, TF_DTYPE)) * a

@tf.function
def wrap_pi_tf(theta):
    two_pi = tf.constant(2.0 * np.pi, dtype=TF_DTYPE)
    pi = tf.constant(np.pi, dtype=TF_DTYPE)
    return tf.math.floormod(theta + pi, two_pi) - pi

@tf.function
def batch_state_to_features_tf(S, U):
    x = S[:, 0]
    xdot = S[:, 1]
    th = S[:, 2]
    thdot = S[:, 3]

    f0 = tf.tanh(x / tf.constant(2.4, dtype=TF_DTYPE))
    f1 = tf.tanh(xdot / tf.constant(3.0, dtype=TF_DTYPE))
    f2 = tf.sin(th)
    f3 = tf.cos(th)
    f4 = tf.tanh(thdot / tf.constant(8.0, dtype=TF_DTYPE))
    f5 = U
    return tf.stack([f0, f1, f2, f3, f4, f5], axis=1)

@tf.function
def stage_cost_cartpole_tf(S, U, x_init, unc_bonus=None, unc_w=0.0):
    th = S[:, 2]
    x  = S[:, 0]
    c = (
        tf.cast(UPRIGHT_W, TF_DTYPE) * (1.0 - tf.cos(th))
        + tf.cast(CENTER_W, TF_DTYPE) * tf.square(x - x_init)
        + tf.cast(U_W, TF_DTYPE) * tf.square(U)
    )
    if (unc_bonus is not None) and (unc_w > 0.0):
        c = c - tf.cast(unc_w, TF_DTYPE) * tf.cast(unc_bonus, TF_DTYPE)
    return c

@tf.function
def terminal_cost_hold_like_tf(S, x_init):
    th = S[:, 2]
    x  = S[:, 0]
    cT = tf.zeros_like(th)
    good = tf.logical_and(tf.cos(th) >= tf.cast(UPRIGHT_COS, TF_DTYPE),
                          tf.abs(x - x_init) <= tf.cast(X_BAND, TF_DTYPE))
    cT = tf.where(good, cT - tf.cast(5.0, TF_DTYPE), cT)
    return cT

# -----------------------
# Exact GP model factory (4 heads, GPR)
# -----------------------
def make_gpr_head(X0, Y0, kernel=None, noise=1e-4):
    if kernel is None:
        kernel = gpflow.kernels.SquaredExponential(lengthscales=np.ones((6,), dtype=np.float64), variance=1.0)

    Xtf = tf.convert_to_tensor(np.asarray(X0, dtype=np.float64), dtype=TF_DTYPE)
    Ytf = tf.convert_to_tensor(np.asarray(Y0, dtype=np.float64).reshape(-1,1), dtype=TF_DTYPE)

    m = gpflow.models.GPR(data=(Xtf, Ytf), kernel=kernel, mean_function=None)
    m.likelihood.variance.assign(float(noise))
    return m

def make_four_head_gpr_models(X0, ydx0, ydxdot0, ydth0, ydthdot0):
    def _k():
        return gpflow.kernels.SquaredExponential(lengthscales=np.ones((6,), dtype=np.float64), variance=1.0)
    m_dx     = make_gpr_head(X0, ydx0,     kernel=_k(), noise=NOISE_UPDATE)
    m_dxdot  = make_gpr_head(X0, ydxdot0,  kernel=_k(), noise=NOISE_UPDATE)
    m_dth    = make_gpr_head(X0, ydth0,    kernel=_k(), noise=NOISE_UPDATE)
    m_dthdot = make_gpr_head(X0, ydthdot0, kernel=_k(), noise=NOISE_UPDATE)
    return m_dx, m_dxdot, m_dth, m_dthdot

def set_gpr_data(model, X, Y):
    Xtf = tf.convert_to_tensor(np.asarray(X, dtype=np.float64), dtype=TF_DTYPE)
    Ytf = tf.convert_to_tensor(np.asarray(Y, dtype=np.float64).reshape(-1,1), dtype=TF_DTYPE)
    model.data = (Xtf, Ytf)

def train_gpr_scipy(model, maxiter=50):
    # Optimize kernel hypers + noise (you can freeze things if desired)
    opt = gpflow.optimizers.Scipy()
    t0 = time.perf_counter()
    res = opt.minimize(
        model.training_loss,
        variables=model.trainable_variables,
        method="L-BFGS-B",
        options=dict(maxiter=int(maxiter), disp=False),
    )
    t_train = time.perf_counter() - t0
    loss_val = float(model.training_loss().numpy())
    return float(t_train), float(loss_val)

# -----------------------
# GLOBAL predictors for TF MPPI (Exact GP)
# -----------------------
def make_global_tf_predictors_bundle_from_gpr(m_dx, m_dxdot, m_dth, m_dthdot):
    @tf.function
    def dx_mu(Xfeat):
        mu, _ = m_dx.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dxdot_mu(Xfeat):
        mu, _ = m_dxdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dth_mu(Xfeat):
        mu, _ = m_dth.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dthdot_mu(Xfeat):
        mu, _ = m_dthdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dxdot_var(Xfeat):
        _, v = m_dxdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(v, (-1,))

    return (dx_mu, dxdot_mu, dth_mu, dthdot_mu), dxdot_var

@tf.function
def gp_dynamics_step_batch_tf(S, U, pred_bundle_tf):
    pred_dx_tf, pred_dxdot_tf, pred_dth_tf, pred_dthdot_tf = pred_bundle_tf
    Xfeat = batch_state_to_features_tf(S, U)
    dx     = pred_dx_tf(Xfeat)
    dxdot  = pred_dxdot_tf(Xfeat)
    dth    = pred_dth_tf(Xfeat)
    dthdot = pred_dthdot_tf(Xfeat)

    S2 = tf.stack([
        S[:, 0] + dx,
        S[:, 1] + dxdot,
        wrap_pi_tf(S[:, 2] + dth),
        S[:, 3] + dthdot
    ], axis=1)
    return S2, Xfeat

@tf.function
def rollout_tube_features_tf(state0, u_seq, pred_bundle_tf):
    H = tf.shape(u_seq)[0]
    s = tf.identity(state0)
    tube = tf.TensorArray(dtype=TF_DTYPE, size=H)

    t = tf.constant(0)
    def cond(t, s, tube): return t < H

    def body(t, s, tube):
        u = u_seq[t]
        xfeat = batch_state_to_features_tf(tf.expand_dims(s, axis=0), tf.expand_dims(u, axis=0))[0]
        tube = tube.write(t, xfeat)
        s2, _ = gp_dynamics_step_batch_tf(tf.expand_dims(s, axis=0), tf.expand_dims(u, axis=0), pred_bundle_tf)
        return t+1, s2[0], tube

    _, _, tube = tf.while_loop(cond, body, [t, s, tube], parallel_iterations=1)
    return tube.stack()

@tf.function
def mppi_plan_gpu_global_tf(state0, x_init, u_mean0, t_global,
                           pred_bundle_tf, unc_var_fn_tf,
                           horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
                           base_seed=0):
    H = tf.cast(horizon, tf.int32)
    Kt = tf.cast(K, tf.int32)

    seed = tf.stack([tf.cast(base_seed, tf.int32), tf.cast(t_global, tf.int32)], axis=0)
    eps = tf.random.stateless_normal((Kt, H), seed=seed, mean=0.0, stddev=tf.cast(sigma, TF_DTYPE), dtype=TF_DTYPE)
    U = tf.clip_by_value(u_mean0[None, :] + eps, tf.cast(U_MIN, TF_DTYPE), tf.cast(U_MAX, TF_DTYPE))

    S = tf.tile(state0[None, :], [Kt, 1])
    total_cost = tf.zeros((Kt,), dtype=TF_DTYPE)

    unc_w = exploration_weight_tf(t_global)

    t = tf.constant(0, dtype=tf.int32)
    def cond(t, S, total_cost): return t < H

    def body(t, S, total_cost):
        Ut = U[:, t]
        S2, Xfeat = gp_dynamics_step_batch_tf(S, Ut, pred_bundle_tf)
        unc_bonus = unc_var_fn_tf(Xfeat)
        total_cost = total_cost + stage_cost_cartpole_tf(
            S, Ut, x_init=tf.cast(x_init, TF_DTYPE),
            unc_bonus=unc_bonus, unc_w=unc_w
        )
        return t+1, S2, total_cost

    _, S, total_cost = tf.while_loop(cond, body, [t, S, total_cost], parallel_iterations=1)
    total_cost = total_cost + terminal_cost_hold_like_tf(S, x_init=tf.cast(x_init, TF_DTYPE))

    cmin = tf.reduce_min(total_cost)
    w = tf.exp(-(total_cost - cmin) / tf.cast(lam, TF_DTYPE))
    wsum = tf.reduce_sum(w) + tf.cast(1e-12, TF_DTYPE)

    u_mean = u_mean0 + tf.reduce_sum(w[:, None] * eps, axis=0) / wsum
    u_mean = tf.clip_by_value(u_mean, tf.cast(U_MIN, TF_DTYPE), tf.cast(U_MAX, TF_DTYPE))

    tubeX = rollout_tube_features_tf(state0, u_mean, pred_bundle_tf)
    return u_mean[0], u_mean, tubeX, unc_w

def mppi_plan_gpu_global(state, x_init, u_init, t_global, pred_bundle_tf, unc_var_fn_tf, base_seed=0):
    state0 = tf.convert_to_tensor(np.asarray(state, dtype=np.float64).reshape(4,), dtype=TF_DTYPE)
    x0     = tf.convert_to_tensor(float(x_init), dtype=TF_DTYPE)
    u0     = tf.convert_to_tensor(np.asarray(u_init, dtype=np.float64).reshape(-1,), dtype=TF_DTYPE)

    dev = "/GPU:0" if len(tf.config.list_logical_devices("GPU")) > 0 else "/CPU:0"
    with tf.device(dev):
        u_first, u_mean, tubeX, unc_w = mppi_plan_gpu_global_tf(
            state0, x0, u0, tf.convert_to_tensor(int(t_global), dtype=tf.int32),
            pred_bundle_tf=pred_bundle_tf,
            unc_var_fn_tf=unc_var_fn_tf,
            horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
            base_seed=int(base_seed)
        )
    return float(u_first.numpy()), u_mean.numpy(), tubeX.numpy(), float(unc_w.numpy())

# -----------------------
# Rendering helper compatibility
# -----------------------
def _render_frame_from_state(s):
    W, H = int(RESIZE[0]), int(RESIZE[1])
    if "render_cartpole_frame_from_state" in globals():
        return render_cartpole_frame_from_state(x=s[0], theta=s[2], x_threshold=2.4, W=W, H=H)
    raise NameError("No render_cartpole_frame_from_state(...) found; set RECORD_RGB=False or define the renderer earlier.")

# ============================
# RUNNER: one run = 3 episodes continuous
# ============================
def run_one_run_exactgp(run_id, seed_base=0):
    # Seed data warm start: if you already have X0/Y0 from earlier cells, use them; else create tiny dummy.
    if "X0" in globals() and "Ydx0" in globals():
        X0 = np.asarray(globals()["X0"], dtype=np.float64)
        ydx0 = np.asarray(globals()["Ydx0"], dtype=np.float64).reshape(-1,1)
        ydxdot0 = np.asarray(globals()["Ydxdot0"], dtype=np.float64).reshape(-1,1)
        ydth0 = np.asarray(globals()["Ydth0"], dtype=np.float64).reshape(-1,1)
        ydthdot0 = np.asarray(globals()["Ydthdot0"], dtype=np.float64).reshape(-1,1)
    else:
        # minimal seed to instantiate models
        X0 = np.zeros((5, 6), dtype=np.float64)
        ydx0 = np.zeros((5, 1), dtype=np.float64)
        ydxdot0 = np.zeros((5, 1), dtype=np.float64)
        ydth0 = np.zeros((5, 1), dtype=np.float64)
        ydthdot0 = np.zeros((5, 1), dtype=np.float64)

    # Fresh models per run
    m_dx, m_dxdot, m_dth, m_dthdot = make_four_head_gpr_models(X0, ydx0, ydxdot0, ydth0, ydthdot0)

    # Dataset per run (accumulates across episodes)
    Xall, ydx_all, ydxdot_all, ydth_all, ydthdot_all = [], [], [], [], []
    # Optionally start with seed
    for i in range(X0.shape[0]):
        Xall.append(X0[i])
        ydx_all.append([float(ydx0[i])])
        ydxdot_all.append([float(ydxdot0[i])])
        ydth_all.append([float(ydth0[i])])
        ydthdot_all.append([float(ydthdot0[i])])

    train_time_step = []
    pred_time_step  = []
    wall_time_cum_excl_vis = []
    train_time_cum = []
    pred_time_cum  = []

    frames = []
    html = None
    vis_time_s_total = 0.0

    t_global = 0
    total_reward_run = 0.0
    updates_run = 0

    pred_bundle_tf, unc_var_fn_tf = make_global_tf_predictors_bundle_from_gpr(m_dx, m_dxdot, m_dth, m_dthdot)

    # Warmup compile once
    if USE_GPU_MPPI and WARMUP_MPPI:
        s_dummy = np.array([0.0, 0.0, np.pi, 0.0], dtype=np.float64)
        u_mean0 = np.zeros((HORIZON,), dtype=np.float64)
        _ = mppi_plan_gpu_global(
            s_dummy, x_init=0.0, u_init=u_mean0, t_global=0,
            pred_bundle_tf=pred_bundle_tf, unc_var_fn_tf=unc_var_fn_tf,
            base_seed=seed_base + 1000*run_id
        )

    t_wall_start = time.perf_counter()
    train_cum = 0.0
    pred_cum  = 0.0

    for ep in range(EPISODES_PER_RUN):
        seed = int(seed_base + 1000 * run_id + ep)
        env = make_env(render_mode=None, seed=seed, max_episode_steps=MAX_STEPS_PER_EP, start_down=START_DOWN)
        obs, info = env.reset(seed=seed)
        s = np.array(obs_to_state(obs), dtype=np.float64)
        x_init = float(s[0])

        u_mean = np.zeros((HORIZON,), dtype=np.float64)
        hold_count = 0
        record_this_ep = (ep == 0) and RECORD_RGB

        if VERBOSE:
            print(f"\n[ExactGP] Run {run_id+1}/{N_RUNS} | Episode {ep+1}/{EPISODES_PER_RUN} | seed={seed}")

        for t_ep in range(MAX_STEPS_PER_EP):
            # planning time
            t0 = time.perf_counter()
            u0, u_mean, tubeX, unc_w = mppi_plan_gpu_global(
                state=s, x_init=x_init, u_init=u_mean, t_global=t_global,
                pred_bundle_tf=pred_bundle_tf, unc_var_fn_tf=unc_var_fn_tf,
                base_seed=seed_base + 1000*run_id
            )
            pred_dt = time.perf_counter() - t0
            pred_cum += pred_dt

            # env step
            obs2, r, terminated, truncated, info = env.step(np.array([u0], dtype=np.float32))
            s2 = np.array(obs_to_state(obs2), dtype=np.float64)
            total_reward_run += float(r)

            # collect (skip respawns)
            respawned = bool(isinstance(info, dict) and info.get("respawned", False))
            if not respawned:
                Xall.append(state_to_features(s[0], s[1], s[2], s[3], float(u0)))
                ydx_all.append([s2[0] - s[0]])
                ydxdot_all.append([s2[1] - s[1]])
                ydth_all.append([wrap_pi(s2[2] - s[2])])
                ydthdot_all.append([s2[3] - s[3]])

            # render
            if record_this_ep and (t_global % FRAME_STRIDE == 0):
                tv0 = time.perf_counter()
                try:
                    frames.append(_render_frame_from_state(s2))
                except Exception:
                    pass
                vis_time_s_total += (time.perf_counter() - tv0)

            # success
            hold_count, success, upright, centered = success_hold_update(s2, x_init, hold_count)

            if VERBOSE and (t_global % 50 == 0):
                print(f"[t={t_global:04d}] u0={u0:+.2f} unc_w={unc_w:.2f} upright={upright} centered={centered} hold={hold_count}/{HOLD_STEPS}")

            # online retrain (Exact GP on ALL data so far)
            train_dt = 0.0
            if ((t_global + 1) % UPDATE_EVERY == 0) and (len(Xall) >= 20):
                updates_run += 1

                Xnp   = np.asarray(Xall, dtype=np.float64)
                ydx   = np.asarray(ydx_all, dtype=np.float64)
                ydxd  = np.asarray(ydxdot_all, dtype=np.float64)
                ydth  = np.asarray(ydth_all, dtype=np.float64)
                ydthd = np.asarray(ydthdot_all, dtype=np.float64)

                tt0 = time.perf_counter()

                set_gpr_data(m_dx,     Xnp, ydx)
                set_gpr_data(m_dxdot,  Xnp, ydxd)
                set_gpr_data(m_dth,    Xnp, ydth)
                set_gpr_data(m_dthdot, Xnp, ydthd)

                # (optional) keep likelihood noise fixed
                m_dx.likelihood.variance.assign(float(NOISE_UPDATE))
                m_dxdot.likelihood.variance.assign(float(NOISE_UPDATE))
                m_dth.likelihood.variance.assign(float(NOISE_UPDATE))
                m_dthdot.likelihood.variance.assign(float(NOISE_UPDATE))

                # optimize hypers
                train_gpr_scipy(m_dx,     maxiter=EXACT_SCIPY_ITERS)
                train_gpr_scipy(m_dxdot,  maxiter=EXACT_SCIPY_ITERS)
                train_gpr_scipy(m_dth,    maxiter=EXACT_SCIPY_ITERS)
                train_gpr_scipy(m_dthdot, maxiter=EXACT_SCIPY_ITERS)

                train_dt = time.perf_counter() - tt0
                train_cum += train_dt

                pred_bundle_tf, unc_var_fn_tf = make_global_tf_predictors_bundle_from_gpr(m_dx, m_dxdot, m_dth, m_dthdot)

            # timing logs
            total_wall = time.perf_counter() - t_wall_start
            wall_excl_vis = max(total_wall - vis_time_s_total, 0.0)

            train_time_step.append(float(train_dt))
            pred_time_step.append(float(pred_dt))
            train_time_cum.append(float(train_cum))
            pred_time_cum.append(float(pred_cum))
            wall_time_cum_excl_vis.append(float(wall_excl_vis))

            # advance
            s = s2
            t_global += 1

            if success:
                if VERBOSE:
                    print(f"✅ SUCCESS at global t={t_global-1} (held {HOLD_STEPS} steps)")
                break
            if terminated or truncated:
                if VERBOSE:
                    print(f"Episode ended at global t={t_global-1} (terminated={terminated}, truncated={truncated})")
                break

        env.close()

    # one render per run
    if RECORD_RGB and (len(frames) > 0):
        tv0 = time.perf_counter()
        fig = plt.figure(figsize=(RESIZE[0]/100, RESIZE[1]/100), dpi=100)
        plt.axis("off")
        im = plt.imshow(frames[0])

        def animate_fn(i):
            im.set_data(frames[i])
            return [im]

        ani = animation.FuncAnimation(fig, animate_fn, frames=len(frames),
                                      interval=1000 / float(FPS), blit=True)
        plt.close(fig)
        html = HTML(ani.to_jshtml())
        display(html)
        vis_time_s_total += (time.perf_counter() - tv0)

    run_stats = dict(
        run_id=int(run_id),
        total_reward=float(total_reward_run),
        total_steps=int(len(train_time_step)),
        updates=int(updates_run),
        vis_time_s=float(vis_time_s_total),
        wall_excl_vis_s=float(wall_time_cum_excl_vis[-1] if len(wall_time_cum_excl_vis)>0 else 0.0),
        train_cum_s=float(train_time_cum[-1] if len(train_time_cum)>0 else 0.0),
        pred_cum_s=float(pred_time_cum[-1] if len(pred_time_cum)>0 else 0.0),
    )

    traces = dict(
        train_time_step=np.asarray(train_time_step, dtype=np.float64),
        train_time_cum=np.asarray(train_time_cum, dtype=np.float64),
        pred_time_step=np.asarray(pred_time_step, dtype=np.float64),
        pred_time_cum=np.asarray(pred_time_cum, dtype=np.float64),
        wall_time_cum_excl_vis=np.asarray(wall_time_cum_excl_vis, dtype=np.float64),
    )

    models = dict(m_dx=m_dx, m_dxdot=m_dxdot, m_dth=m_dth, m_dthdot=m_dthdot)

    return run_stats, traces, models, html

# ============================
# MULTI-RUN DRIVER
# ============================
all_stats = []
all_traces = []
all_models = []

for r in range(N_RUNS):
    stats_r, traces_r, models_r, html_r = run_one_run_exactgp(run_id=r, seed_base=0)
    all_stats.append(stats_r)
    all_traces.append(traces_r)
    all_models.append(models_r)

    print(f"\n[ExactGP] Run {r+1} stats:", stats_r)

    plot_global_surface_dxdot(
        models_r["m_dxdot"],
        title=f"Exact GP baseline — GLOBAL Δxdot after Run {r+1} (mean colored by std)"
    )

# ============================
# MEAN PLOTS ACROSS RUNS
# ============================
def nanpad_to(T, L):
    out = np.full((L,), np.nan, dtype=np.float64)
    n = min(L, len(T))
    out[:n] = T[:n]
    return out

maxL = max([len(tr["train_time_step"]) for tr in all_traces]) if len(all_traces)>0 else 0
if maxL == 0:
    raise RuntimeError("No timesteps were recorded.")

train_step_mat = np.vstack([nanpad_to(tr["train_time_step"], maxL) for tr in all_traces])
train_cum_mat  = np.vstack([nanpad_to(tr["train_time_cum"], maxL) for tr in all_traces])
pred_step_mat  = np.vstack([nanpad_to(tr["pred_time_step"], maxL) for tr in all_traces])
wall_cum_mat   = np.vstack([nanpad_to(tr["wall_time_cum_excl_vis"], maxL) for tr in all_traces])

train_step_mean = np.nanmean(train_step_mat, axis=0)
train_cum_mean  = np.nanmean(train_cum_mat, axis=0)
pred_step_mean  = np.nanmean(pred_step_mat, axis=0)
wall_cum_mean   = np.nanmean(wall_cum_mat, axis=0)

t_axis = np.arange(maxL)

plt.figure(figsize=(10,4))
plt.plot(t_axis, train_step_mean)
plt.title("Exact GP baseline — mean TRAIN time per timestep (non-cumulative) across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("train time at step (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, train_cum_mean)
plt.title("Exact GP baseline — mean TRAIN time cumulative across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("cumulative train time (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, pred_step_mean)
plt.title("Exact GP baseline — mean PRED/MPPI time per timestep across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("prediction/planning time at step (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, wall_cum_mean)
plt.title("Exact GP baseline — mean WALL time cumulative (excluding visualization) across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("cumulative wall time excl. vis (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

print("\n[Exact GP baseline] Summary across runs:")
for s in all_stats:
    print(s)


In [ ]:
# ============================
# Cell 9 — BASELINE: Pure Streaming OSGPR-VFE (4-head) + MPPI (GLOBAL predictors)
#
# ✅ Pure OSGPR-VFE baseline meaning:
#   - No local subset, no tube, no anchors
#   - Fixed inducing size: Z0 = Z_GLOBAL[:M_GLOBAL] for the whole run
#   - Stream update every UPDATE_EVERY using ONLY NEW data since last update (buffer-only)
#
# ✅ Protocol matches your SVGP/PLSSGP evaluation style:
#   - N_RUNS = 3
#   - Each run = EPISODES_PER_RUN continuous episodes (models carry across episodes in a run)
#   - BEFORE each run: full reset to the SAME initial OSGPR models + SAME Z0
#   - One render per run (episode 0 only)
#   - Logs per-timestep timings:
#       * train_time_step (non-cumulative)
#       * train_time_cum
#       * pred_time_step (MPPI planning time)
#       * wall_time_cum_excl_vis (excluding visualization)
#   - Plots mean curves across runs (nan-mean)
#   - Plots a GLOBAL surface after each run (Δxdot mean colored by std)
# ============================

import time
import copy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML, display
import plotly.graph_objects as go
import tensorflow as tf
import gpflow

# -----------------------
# REQUIRED symbols from earlier cells
# -----------------------
required = [
    "make_env", "obs_to_state", "wrap_pi",
    "state_to_features", "batch_state_to_features",
    "render_cartpole_frame_from_state",
    "Z_GLOBAL",
    "osgpr_stream_update",
    "m_dx", "m_dxdot", "m_dth", "m_dthdot",  # we will use these as the "reset template"
]
missing = [k for k in required if k not in globals()]
if len(missing) > 0:
    raise NameError(f"Cell 9 missing required symbols from earlier cells: {missing}")

# -----------------------
# Dtypes (float64 throughout)
# -----------------------
NP_DTYPE = np.float64
TF_DTYPE = gpflow.default_float()
if TF_DTYPE != tf.float64:
    print("⚠️ gpflow.default_float() is not float64. If you want float64 everywhere, set gpflow.config.set_default_float(np.float64) in Cell 1.")

# -----------------------
# Config defaults (prefer your globals if already defined)
# -----------------------
HORIZON    = int(globals().get("HORIZON", 50))
K_SAMPLES  = int(globals().get("K_SAMPLES", 256))
SIGMA      = float(globals().get("SIGMA", 0.6))
LAMBDA     = float(globals().get("LAMBDA", 1.0))

UPDATE_EVERY = int(globals().get("UPDATE_EVERY", 30))
OSGPR_ITERS  = int(globals().get("OSGPR_ITERS", 150))
OSGPR_LR     = float(globals().get("OSGPR_LR", 0.02))
NOISE_UPDATE = float(globals().get("NOISE_UPDATE", 1e-4))

M_GLOBAL = int(globals().get("M_GLOBAL", 256))

# task/success
X_BAND      = float(globals().get("X_BAND", 0.70))
UPRIGHT_COS = float(globals().get("UPRIGHT_COS", 0.85))
HOLD_STEPS  = int(globals().get("HOLD_STEPS", 200))

# exploration schedule (same as your SVGP baseline)
EXPLORE_STEPS = int(globals().get("EXPLORE_STEPS", 200))
UNC_W_MAX     = float(globals().get("UNC_W_MAX", 15.0))
UNC_W_MIN     = float(globals().get("UNC_W_MIN", 0.0))
CENTER_W      = float(globals().get("CENTER_W", 1.0))
U_W           = float(globals().get("U_W", 0.005))
UPRIGHT_W     = float(globals().get("UPRIGHT_W", 2.0))

# action bounds
U_MIN = float(globals().get("U_MIN", -1.0))
U_MAX = float(globals().get("U_MAX", +1.0))

# rendering
RECORD_RGB   = True
RESIZE       = globals().get("RESIZE", (720, 450))
FPS          = int(globals().get("FPS", 10))
FRAME_STRIDE = int(globals().get("FRAME_STRIDE", 2))

# multi-run protocol
N_RUNS = 3
EPISODES_PER_RUN = 3
MAX_STEPS_PER_EP = int(globals().get("MAX_STEPS_PER_EP", 1000))
START_DOWN = bool(globals().get("START_DOWN", True))
USE_GPU_MPPI = True
WARMUP_MPPI = True
VERBOSE = True

# Optional: whether to freeze hypers in OSGPR updates
OSGPR_FREEZE_HYPERS = bool(globals().get("OSGPR_FREEZE_HYPERS", True))

# -----------------------
# Fixed plot grid (for surface after each run)
# -----------------------
PLOT_X_MIN, PLOT_X_MAX = globals().get("PLOT_X_MIN", -2.4), globals().get("PLOT_X_MAX", 2.4)
PLOT_V_MIN, PLOT_V_MAX = globals().get("PLOT_V_MIN", -3.0), globals().get("PLOT_V_MAX", 3.0)
PLOT_N_GRID            = int(globals().get("PLOT_N_GRID", 60))
PLOT_TH_FIXED          = float(globals().get("PLOT_TH_FIXED", 0.0))
PLOT_THDOT_FIXED       = float(globals().get("PLOT_THDOT_FIXED", 0.0))
PLOT_U_FIXED           = float(globals().get("PLOT_U_FIXED", +1.0))

def build_fixed_plot_grid():
    xg = np.linspace(PLOT_X_MIN, PLOT_X_MAX, PLOT_N_GRID)
    vg = np.linspace(PLOT_V_MIN, PLOT_V_MAX, PLOT_N_GRID)
    X, V = np.meshgrid(xg, vg)
    Xfeat_grid = np.vstack([
        state_to_features(x, xdot, PLOT_TH_FIXED, PLOT_THDOT_FIXED, PLOT_U_FIXED)
        for x, xdot in zip(X.ravel(), V.ravel())
    ]).astype(np.float64)
    return X, V, Xfeat_grid

PLOT_XMESH, PLOT_VMESH, PLOT_XFEAT_GRID = build_fixed_plot_grid()

def gp_predict_mu_std(model, X):
    Xtf = tf.convert_to_tensor(np.asarray(X, dtype=np.float64), dtype=TF_DTYPE)
    mu_tf, var_tf = model.predict_f(Xtf, full_cov=False)
    mu = mu_tf.numpy().reshape(-1)
    var = var_tf.numpy().reshape(-1)
    std = np.sqrt(np.maximum(var, 1e-12))
    return mu, std

def plot_global_surface_dxdot(model_dxdot, title="OSGPR baseline — GLOBAL Δxdot surface"):
    mu, std = gp_predict_mu_std(model_dxdot, PLOT_XFEAT_GRID)
    Mean = mu.reshape(PLOT_XMESH.shape)
    Std  = std.reshape(PLOT_XMESH.shape)

    fig = go.Figure(data=[
        go.Surface(
            x=PLOT_XMESH, y=PLOT_VMESH, z=Mean,
            surfacecolor=Std,
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Std"),
            opacity=0.95
        )
    ])
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title="x", range=[PLOT_X_MIN, PLOT_X_MAX]),
            yaxis=dict(title="xdot", range=[PLOT_V_MIN, PLOT_V_MAX]),
            zaxis=dict(title="Δxdot"),
        ),
        margin=dict(l=0, r=0, b=0, t=50),
        height=620
    )
    fig.show()

# -----------------------
# Success check (same logic)
# -----------------------
def success_hold_update(state, x_init, hold_count):
    x, xdot, th, thdot = state
    upright = (np.cos(th) >= UPRIGHT_COS)
    centered = (abs(x - x_init) <= X_BAND)
    hold_count = (hold_count + 1) if (upright and centered) else 0
    success = (hold_count >= HOLD_STEPS)
    return hold_count, success, upright, centered

# -----------------------
# Cost + exploration weight (same as your SVGP baseline)
# -----------------------
@tf.function
def exploration_weight_tf(t):
    explore_steps_f = tf.cast(EXPLORE_STEPS, TF_DTYPE)
    t_f = tf.cast(t, TF_DTYPE)
    a = tf.clip_by_value(1.0 - t_f / tf.maximum(explore_steps_f, 1.0), 0.0, 1.0)
    return tf.cast(UNC_W_MIN, TF_DTYPE) + (tf.cast(UNC_W_MAX, TF_DTYPE) - tf.cast(UNC_W_MIN, TF_DTYPE)) * a

@tf.function
def wrap_pi_tf(theta):
    two_pi = tf.constant(2.0 * np.pi, dtype=TF_DTYPE)
    pi = tf.constant(np.pi, dtype=TF_DTYPE)
    return tf.math.floormod(theta + pi, two_pi) - pi

@tf.function
def batch_state_to_features_tf(S, U):
    x = S[:, 0]
    xdot = S[:, 1]
    th = S[:, 2]
    thdot = S[:, 3]

    f0 = tf.tanh(x / tf.constant(2.4, dtype=TF_DTYPE))
    f1 = tf.tanh(xdot / tf.constant(3.0, dtype=TF_DTYPE))
    f2 = tf.sin(th)
    f3 = tf.cos(th)
    f4 = tf.tanh(thdot / tf.constant(8.0, dtype=TF_DTYPE))
    f5 = U
    return tf.stack([f0, f1, f2, f3, f4, f5], axis=1)

@tf.function
def stage_cost_cartpole_tf(S, U, x_init, unc_bonus=None, unc_w=0.0):
    th = S[:, 2]
    x  = S[:, 0]
    c = (
        tf.cast(UPRIGHT_W, TF_DTYPE) * (1.0 - tf.cos(th))
        + tf.cast(CENTER_W, TF_DTYPE) * tf.square(x - x_init)
        + tf.cast(U_W, TF_DTYPE) * tf.square(U)
    )
    if (unc_bonus is not None) and (unc_w > 0.0):
        c = c - tf.cast(unc_w, TF_DTYPE) * tf.cast(unc_bonus, TF_DTYPE)
    return c

@tf.function
def terminal_cost_hold_like_tf(S, x_init):
    th = S[:, 2]
    x  = S[:, 0]
    cT = tf.zeros_like(th)
    good = tf.logical_and(tf.cos(th) >= tf.cast(UPRIGHT_COS, TF_DTYPE),
                          tf.abs(x - x_init) <= tf.cast(X_BAND, TF_DTYPE))
    cT = tf.where(good, cT - tf.cast(5.0, TF_DTYPE), cT)
    return cT

# -----------------------
# GLOBAL predictors for TF MPPI
# -----------------------
def make_global_tf_predictors_bundle_from_models(m_dx, m_dxdot, m_dth, m_dthdot):
    @tf.function
    def dx_mu(Xfeat):
        mu, _ = m_dx.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dxdot_mu(Xfeat):
        mu, _ = m_dxdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dth_mu(Xfeat):
        mu, _ = m_dth.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dthdot_mu(Xfeat):
        mu, _ = m_dthdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(mu, (-1,))

    @tf.function
    def dxdot_var(Xfeat):
        _, v = m_dxdot.predict_f(Xfeat, full_cov=False)
        return tf.reshape(v, (-1,))

    return (dx_mu, dxdot_mu, dth_mu, dthdot_mu), dxdot_var

@tf.function
def gp_dynamics_step_batch_tf(S, U, pred_bundle_tf):
    pred_dx_tf, pred_dxdot_tf, pred_dth_tf, pred_dthdot_tf = pred_bundle_tf
    Xfeat = batch_state_to_features_tf(S, U)
    dx     = pred_dx_tf(Xfeat)
    dxdot  = pred_dxdot_tf(Xfeat)
    dth    = pred_dth_tf(Xfeat)
    dthdot = pred_dthdot_tf(Xfeat)

    S2 = tf.stack([
        S[:, 0] + dx,
        S[:, 1] + dxdot,
        wrap_pi_tf(S[:, 2] + dth),
        S[:, 3] + dthdot
    ], axis=1)
    return S2, Xfeat

@tf.function
def mppi_plan_gpu_global_tf(state0, x_init, u_mean0, t_global,
                           pred_bundle_tf, unc_var_fn_tf,
                           horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
                           base_seed=0):
    H = tf.cast(horizon, tf.int32)
    Kt = tf.cast(K, tf.int32)

    seed = tf.stack([tf.cast(base_seed, tf.int32), tf.cast(t_global, tf.int32)], axis=0)
    eps = tf.random.stateless_normal((Kt, H), seed=seed, mean=0.0, stddev=tf.cast(sigma, TF_DTYPE), dtype=TF_DTYPE)
    U = tf.clip_by_value(u_mean0[None, :] + eps, tf.cast(U_MIN, TF_DTYPE), tf.cast(U_MAX, TF_DTYPE))

    S = tf.tile(state0[None, :], [Kt, 1])
    total_cost = tf.zeros((Kt,), dtype=TF_DTYPE)

    unc_w = exploration_weight_tf(t_global)

    t = tf.constant(0, dtype=tf.int32)
    def cond(t, S, total_cost): return t < H

    def body(t, S, total_cost):
        Ut = U[:, t]
        S2, Xfeat = gp_dynamics_step_batch_tf(S, Ut, pred_bundle_tf)
        unc_bonus = unc_var_fn_tf(Xfeat)
        total_cost = total_cost + stage_cost_cartpole_tf(
            S, Ut, x_init=tf.cast(x_init, TF_DTYPE),
            unc_bonus=unc_bonus, unc_w=unc_w
        )
        return t+1, S2, total_cost

    _, S, total_cost = tf.while_loop(cond, body, [t, S, total_cost], parallel_iterations=1)
    total_cost = total_cost + terminal_cost_hold_like_tf(S, x_init=tf.cast(x_init, TF_DTYPE))

    cmin = tf.reduce_min(total_cost)
    w = tf.exp(-(total_cost - cmin) / tf.cast(lam, TF_DTYPE))
    wsum = tf.reduce_sum(w) + tf.cast(1e-12, TF_DTYPE)

    u_mean = u_mean0 + tf.reduce_sum(w[:, None] * eps, axis=0) / wsum
    u_mean = tf.clip_by_value(u_mean, tf.cast(U_MIN, TF_DTYPE), tf.cast(U_MAX, TF_DTYPE))
    return u_mean[0], u_mean, unc_w

def mppi_plan_gpu_global(state, x_init, u_init, t_global, pred_bundle_tf, unc_var_fn_tf, base_seed=0):
    state0 = tf.convert_to_tensor(np.asarray(state, dtype=np.float64).reshape(4,), dtype=TF_DTYPE)
    x0     = tf.convert_to_tensor(float(x_init), dtype=TF_DTYPE)
    u0     = tf.convert_to_tensor(np.asarray(u_init, dtype=np.float64).reshape(-1,), dtype=TF_DTYPE)

    dev = "/GPU:0" if len(tf.config.list_logical_devices("GPU")) > 0 else "/CPU:0"
    with tf.device(dev):
        u_first, u_mean, unc_w = mppi_plan_gpu_global_tf(
            state0, x0, u0, tf.convert_to_tensor(int(t_global), dtype=tf.int32),
            pred_bundle_tf=pred_bundle_tf,
            unc_var_fn_tf=unc_var_fn_tf,
            horizon=HORIZON, K=K_SAMPLES, sigma=SIGMA, lam=LAMBDA,
            base_seed=int(base_seed)
        )
    return float(u_first.numpy()), u_mean.numpy(), float(unc_w.numpy())

# -----------------------
# Rendering helper
# -----------------------
def _render_frame_from_state(s):
    W, H = int(RESIZE[0]), int(RESIZE[1])
    return render_cartpole_frame_from_state(x=s[0], theta=s[2], x_threshold=2.4, W=W, H=H)

# -----------------------
# OSGPR update wrapper (fixed Z)
# -----------------------
def osgpr_update_fixedZ(model_old, Xnew, Ynew, Z0, iters, lr, freeze_hypers):
    return osgpr_stream_update(
        model_old,
        np.asarray(Xnew, dtype=np.float64),
        np.asarray(Ynew, dtype=np.float64),
        np.asarray(Z0, dtype=np.float64),
        iters=int(iters),
        lr=float(lr),
        freeze_hypers=bool(freeze_hypers),
    )

# ============================
# IMPORTANT: snapshot "initial models" NOW as the reset template
#   This guarantees each run starts from the same parameters.
# ============================
_BASE_m_dx     = copy.deepcopy(m_dx)
_BASE_m_dxdot  = copy.deepcopy(m_dxdot)
_BASE_m_dth    = copy.deepcopy(m_dth)
_BASE_m_dthdot = copy.deepcopy(m_dthdot)

# ============================
# RUNNER: one run = 3 continuous episodes
# ============================
def run_one_run_osgpr_pure(run_id, seed_base=0):
    # fixed inducing
    Zg = np.asarray(Z_GLOBAL, dtype=np.float64)
    if Zg.shape[0] < M_GLOBAL:
        raise ValueError(f"Z_GLOBAL has {Zg.shape[0]} points but M_GLOBAL={M_GLOBAL}. Provide at least {M_GLOBAL} points.")
    Z0 = Zg[:M_GLOBAL].copy()

    # reset models for this run
    m_dx     = copy.deepcopy(_BASE_m_dx)
    m_dxdot  = copy.deepcopy(_BASE_m_dxdot)
    m_dth    = copy.deepcopy(_BASE_m_dth)
    m_dthdot = copy.deepcopy(_BASE_m_dthdot)

    # fixed noise (optional): keep identical across methods
    try:
        m_dx.likelihood.variance.assign(float(NOISE_UPDATE))
        m_dxdot.likelihood.variance.assign(float(NOISE_UPDATE))
        m_dth.likelihood.variance.assign(float(NOISE_UPDATE))
        m_dthdot.likelihood.variance.assign(float(NOISE_UPDATE))
    except Exception:
        pass

    # per-update buffer (streaming)
    Xbuf, ydx_buf, ydxdot_buf, ydth_buf, ydthdot_buf = [], [], [], [], []

    # timing traces per timestep
    train_time_step = []
    pred_time_step  = []
    wall_time_cum_excl_vis = []
    train_time_cum = []
    pred_time_cum  = []

    # one render per run (episode 0 only)
    frames = []
    html = None
    vis_time_s_total = 0.0

    # global counters
    t_global = 0
    total_reward_run = 0.0
    updates_run = 0

    # predictor bundle
    pred_bundle_tf, unc_var_fn_tf = make_global_tf_predictors_bundle_from_models(m_dx, m_dxdot, m_dth, m_dthdot)

    # warmup (compile)
    if USE_GPU_MPPI and WARMUP_MPPI:
        s_dummy = np.array([0.0, 0.0, np.pi, 0.0], dtype=np.float64)
        u_mean0 = np.zeros((HORIZON,), dtype=np.float64)
        _ = mppi_plan_gpu_global(
            s_dummy, x_init=0.0, u_init=u_mean0, t_global=0,
            pred_bundle_tf=pred_bundle_tf, unc_var_fn_tf=unc_var_fn_tf,
            base_seed=seed_base + 1000*run_id
        )

    t_wall_start = time.perf_counter()
    train_cum = 0.0
    pred_cum  = 0.0

    for ep in range(EPISODES_PER_RUN):
        seed = int(seed_base + 1000 * run_id + ep)
        env = make_env(render_mode=None, seed=seed, max_episode_steps=MAX_STEPS_PER_EP, start_down=START_DOWN)
        obs, info = env.reset(seed=seed)
        s = np.array(obs_to_state(obs), dtype=np.float64)
        x_init = float(s[0])

        u_mean = np.zeros((HORIZON,), dtype=np.float64)
        hold_count = 0
        record_this_ep = (ep == 0) and RECORD_RGB

        if VERBOSE:
            print(f"\n[OSGPR-PURE] Run {run_id+1}/{N_RUNS} | Episode {ep+1}/{EPISODES_PER_RUN} | seed={seed}")

        for t_ep in range(MAX_STEPS_PER_EP):
            # ---- planning time ----
            t0 = time.perf_counter()
            u0, u_mean, unc_w = mppi_plan_gpu_global(
                state=s, x_init=x_init, u_init=u_mean, t_global=t_global,
                pred_bundle_tf=pred_bundle_tf, unc_var_fn_tf=unc_var_fn_tf,
                base_seed=seed_base + 1000*run_id
            )
            pred_dt = time.perf_counter() - t0
            pred_cum += pred_dt

            # ---- env step ----
            obs2, r, terminated, truncated, info = env.step(np.array([u0], dtype=np.float32))
            s2 = np.array(obs_to_state(obs2), dtype=np.float64)
            total_reward_run += float(r)

            # ---- collect buffer (skip respawned) ----
            respawned = bool(isinstance(info, dict) and info.get("respawned", False))
            if not respawned:
                Xbuf.append(state_to_features(s[0], s[1], s[2], s[3], float(u0)))
                ydx_buf.append([s2[0] - s[0]])
                ydxdot_buf.append([s2[1] - s[1]])
                ydth_buf.append([wrap_pi(s2[2] - s[2])])
                ydthdot_buf.append([s2[3] - s[3]])

            # ---- render (excluded) ----
            if record_this_ep and (t_global % FRAME_STRIDE == 0):
                tv0 = time.perf_counter()
                try:
                    frames.append(_render_frame_from_state(s2))
                except Exception:
                    pass
                vis_time_s_total += (time.perf_counter() - tv0)

            # ---- success ----
            hold_count, success, upright, centered = success_hold_update(s2, x_init, hold_count)
            if VERBOSE and (t_global % 50 == 0):
                print(f"[t={t_global:04d}] u0={u0:+.2f} unc_w={unc_w:.2f} upright={upright} centered={centered} hold={hold_count}/{HOLD_STEPS}")

            # ---- streaming OSGPR update (buffer only) ----
            train_dt = 0.0
            if ((t_global + 1) % UPDATE_EVERY == 0) and (len(Xbuf) >= 10):
                updates_run += 1

                Xnew = np.asarray(Xbuf, dtype=np.float64)
                ydx  = np.asarray(ydx_buf, dtype=np.float64)
                ydxd = np.asarray(ydxdot_buf, dtype=np.float64)
                ydth = np.asarray(ydth_buf, dtype=np.float64)
                ydthd= np.asarray(ydthdot_buf, dtype=np.float64)

                tt0 = time.perf_counter()
                m_dx     = osgpr_update_fixedZ(m_dx,     Xnew, ydx,  Z0, iters=OSGPR_ITERS, lr=OSGPR_LR, freeze_hypers=OSGPR_FREEZE_HYPERS)
                m_dxdot  = osgpr_update_fixedZ(m_dxdot,  Xnew, ydxd, Z0, iters=OSGPR_ITERS, lr=OSGPR_LR, freeze_hypers=OSGPR_FREEZE_HYPERS)
                m_dth    = osgpr_update_fixedZ(m_dth,    Xnew, ydth, Z0, iters=OSGPR_ITERS, lr=OSGPR_LR, freeze_hypers=OSGPR_FREEZE_HYPERS)
                m_dthdot = osgpr_update_fixedZ(m_dthdot, Xnew, ydthd,Z0, iters=OSGPR_ITERS, lr=OSGPR_LR, freeze_hypers=OSGPR_FREEZE_HYPERS)
                train_dt = time.perf_counter() - tt0
                train_cum += train_dt

                # clear buffers
                Xbuf.clear(); ydx_buf.clear(); ydxdot_buf.clear(); ydth_buf.clear(); ydthdot_buf.clear()

                # refresh predictor bundle
                pred_bundle_tf, unc_var_fn_tf = make_global_tf_predictors_bundle_from_models(m_dx, m_dxdot, m_dth, m_dthdot)

            # ---- timing logs ----
            total_wall = time.perf_counter() - t_wall_start
            wall_excl_vis = max(total_wall - vis_time_s_total, 0.0)

            train_time_step.append(float(train_dt))
            pred_time_step.append(float(pred_dt))
            train_time_cum.append(float(train_cum))
            pred_time_cum.append(float(pred_cum))
            wall_time_cum_excl_vis.append(float(wall_excl_vis))

            # advance
            s = s2
            t_global += 1

            if success:
                if VERBOSE:
                    print(f"✅ SUCCESS at global t={t_global-1} (held {HOLD_STEPS} steps)")
                break
            if terminated or truncated:
                if VERBOSE:
                    print(f"Episode ended at global t={t_global-1} (terminated={terminated}, truncated={truncated})")
                break

        env.close()

    # ---- one render per run ----
    if RECORD_RGB and (len(frames) > 0):
        tv0 = time.perf_counter()
        fig = plt.figure(figsize=(RESIZE[0]/100, RESIZE[1]/100), dpi=100)
        plt.axis("off")
        im = plt.imshow(frames[0])

        def animate_fn(i):
            im.set_data(frames[i])
            return [im]

        ani = animation.FuncAnimation(fig, animate_fn, frames=len(frames),
                                      interval=1000 / float(FPS), blit=True)
        plt.close(fig)
        html = HTML(ani.to_jshtml())
        display(html)
        vis_time_s_total += (time.perf_counter() - tv0)

    run_stats = dict(
        run_id=int(run_id),
        total_reward=float(total_reward_run),
        total_steps=int(len(train_time_step)),
        updates=int(updates_run),
        vis_time_s=float(vis_time_s_total),
        wall_excl_vis_s=float(wall_time_cum_excl_vis[-1] if len(wall_time_cum_excl_vis)>0 else 0.0),
        train_cum_s=float(train_time_cum[-1] if len(train_time_cum)>0 else 0.0),
        pred_cum_s=float(pred_time_cum[-1] if len(pred_time_cum)>0 else 0.0),
    )

    traces = dict(
        train_time_step=np.asarray(train_time_step, dtype=np.float64),
        train_time_cum=np.asarray(train_time_cum, dtype=np.float64),
        pred_time_step=np.asarray(pred_time_step, dtype=np.float64),
        pred_time_cum=np.asarray(pred_time_cum, dtype=np.float64),
        wall_time_cum_excl_vis=np.asarray(wall_time_cum_excl_vis, dtype=np.float64),
    )

    models = dict(m_dx=m_dx, m_dxdot=m_dxdot, m_dth=m_dth, m_dthdot=m_dthdot)
    return run_stats, traces, models, html

# ============================
# MULTI-RUN DRIVER
# ============================
all_stats = []
all_traces = []
all_models = []

for r in range(N_RUNS):
    stats_r, traces_r, models_r, html_r = run_one_run_osgpr_pure(run_id=r, seed_base=0)
    all_stats.append(stats_r)
    all_traces.append(traces_r)
    all_models.append(models_r)

    print(f"\n[OSGPR-PURE] Run {r+1} stats:", stats_r)

    # surface after each run
    plot_global_surface_dxdot(
        models_r["m_dxdot"],
        title=f"OSGPR-PURE baseline — GLOBAL Δxdot after Run {r+1} (mean colored by std)"
    )

# ============================
# MEAN PLOTS ACROSS RUNS (nan-mean over variable lengths)
# ============================
def nanpad_to(T, L):
    out = np.full((L,), np.nan, dtype=np.float64)
    n = min(L, len(T))
    out[:n] = T[:n]
    return out

maxL = max([len(tr["train_time_step"]) for tr in all_traces]) if len(all_traces)>0 else 0
if maxL == 0:
    raise RuntimeError("No timesteps were recorded (did the env terminate immediately?).")

train_step_mat = np.vstack([nanpad_to(tr["train_time_step"], maxL) for tr in all_traces])
train_cum_mat  = np.vstack([nanpad_to(tr["train_time_cum"], maxL) for tr in all_traces])
pred_step_mat  = np.vstack([nanpad_to(tr["pred_time_step"], maxL) for tr in all_traces])
wall_cum_mat   = np.vstack([nanpad_to(tr["wall_time_cum_excl_vis"], maxL) for tr in all_traces])

train_step_mean = np.nanmean(train_step_mat, axis=0)
train_cum_mean  = np.nanmean(train_cum_mat, axis=0)
pred_step_mean  = np.nanmean(pred_step_mat, axis=0)
wall_cum_mean   = np.nanmean(wall_cum_mat, axis=0)

t_axis = np.arange(maxL)

plt.figure(figsize=(10,4))
plt.plot(t_axis, train_step_mean)
plt.title("OSGPR-PURE baseline — mean TRAIN time per timestep (non-cumulative) across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("train time at step (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, train_cum_mean)
plt.title("OSGPR-PURE baseline — mean TRAIN time cumulative across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("cumulative train time (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, pred_step_mean)
plt.title("OSGPR-PURE baseline — mean PRED/MPPI time per timestep across runs")
plt.xlabel("global timestep (within run)")
plt.ylabel("prediction/planning time at step (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,4))
plt.plot(t_axis, wall_cum_mean)
plt.title("OSGPR-PURE baseline — mean WALL time cumulative (excluding visualization)")
plt.xlabel("global timestep (within run)")
plt.ylabel("cumulative wall time excl. vis (s)")
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

print("\n[OSGPR-PURE baseline] Summary across runs:")
for s in all_stats:
    print(s)
